Resolution modification

In [2]:
from PIL import Image,ImageEnhance
from dotenv import load_dotenv 
import boto3
import os
import numpy as np
import math

In [3]:

dotenv_path = os.path.join("..",'.env')
load_dotenv(dotenv_path)

#Connect to AWS Ressource, Credentials are optionals as we are accessing from the same AWS user
client=boto3.client('rekognition',aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("REGION_NAME"))
s3 = boto3.client('s3',
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("REGION_NAME")
)

#Define bucket name to access
bucket_name = os.getenv("BUCKET_NAME")

#Folder of real classroom photos
classroomFolder=os.path.join("..","ImagesRekognition","RealClassroomPhotos")

In [4]:

def calculate_iou(box1, box2):

    name1,x1, y1, w1, h1 = box1
    name2, x2, y2, w2, h2 = box2
    
    x_intersect_1 = max(x1, x2)
    y_intersect_1 = max(y1, y2)
    x_intersect_2 = min(x1 + w1, x2 + w2)
    y_intersect_2 = min(y1 + h1, y2 + h2)
    
    if x_intersect_2 <= x_intersect_1 or y_intersect_2 <= y_intersect_1:
        return 0.0
    
    area_intersect = (x_intersect_2 - x_intersect_1) * (y_intersect_2 - y_intersect_1)
    area_box1 = w1 * h1
    area_box2 = w2 * h2
    
    area_union = area_box1 + area_box2 - area_intersect
    
    iou = area_intersect / area_union
    
    return iou


Load Face gallery and obtain bounding boxes and names only

In [5]:
true=True
faceGalleryArray=[
{"classroom1.jpeg156424":{"filename":"classroom1.jpeg","size":156424,"regions":[{"shape_attributes":{"name":"rect","x":910,"y":348,"width":21,"height":27},"region_attributes":{"name":"AlbaQuiroz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":733,"y":402,"width":29,"height":34},"region_attributes":{"name":"AlejandroCasallas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":344,"y":485,"width":38,"height":44},"region_attributes":{"name":"AlejandroVelasco","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":612,"y":337,"width":20,"height":23},"region_attributes":{"name":"AlexanderDominguez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":164,"y":481,"width":35,"height":38},"region_attributes":{"name":"AndresAnillo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":567,"y":548,"width":57,"height":58},"region_attributes":{"name":"CarlosBornacelly","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":403,"y":582,"width":59,"height":78},"region_attributes":{"name":"DavidChaparro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":693,"y":332,"width":20,"height":22},"region_attributes":{"name":"EstebanToro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":15,"y":506,"width":35,"height":40},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1189,"y":330,"width":31,"height":31},"region_attributes":{"name":"GiohanOlivares","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":122,"y":538,"width":38,"height":55},"region_attributes":{"name":"JesusCotes","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":440,"y":368,"width":24,"height":26},"region_attributes":{"name":"JoseVilla","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":806,"y":460,"width":47,"height":55},"region_attributes":{"name":"ManuelRangel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1158,"y":366,"width":26,"height":33},"region_attributes":{"name":"MariaBerdugo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1023,"y":399,"width":30,"height":37},"region_attributes":{"name":"MariaLopez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":717,"y":350,"width":22,"height":27},"region_attributes":{"name":"MauricioDeLaHoz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":547,"y":431,"width":37,"height":44},"region_attributes":{"name":"SergioFonseca","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":372,"y":427,"width":28,"height":33},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":616,"y":374,"width":30,"height":37},"region_attributes":{"name":"JuanMolina","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom2.jpeg152579":{"filename":"classroom2.jpeg","size":152579,"regions":[{"shape_attributes":{"name":"rect","x":910,"y":347,"width":25,"height":28},"region_attributes":{"name":"AlbaQuiroz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":729,"y":394,"width":31,"height":35},"region_attributes":{"name":"AlejandroCasallas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":340,"y":481,"width":37,"height":46},"region_attributes":{"name":"AlejandroVelasco","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":617,"y":336,"width":20,"height":25},"region_attributes":{"name":"AlexanderDominguez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":160,"y":481,"width":31,"height":37},"region_attributes":{"name":"AndresAnillo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":580,"y":541,"width":55,"height":65},"region_attributes":{"name":"CarlosBornacelly","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":428,"y":580,"width":51,"height":76},"region_attributes":{"name":"DavidChaparro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":693,"y":333,"width":20,"height":20},"region_attributes":{"name":"EstebanToro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":28,"y":504,"width":29,"height":38},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1185,"y":328,"width":27,"height":32},"region_attributes":{"name":"GiohanOlivares","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":171,"y":538,"width":40,"height":57},"region_attributes":{"name":"JesusCotes","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":443,"y":366,"width":23,"height":27},"region_attributes":{"name":"JoseVilla","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":809,"y":459,"width":46,"height":53},"region_attributes":{"name":"ManuelRangel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1126,"y":375,"width":25,"height":33},"region_attributes":{"name":"MariaBerdugo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1039,"y":395,"width":25,"height":36},"region_attributes":{"name":"MariaLopez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":720,"y":350,"width":20,"height":30},"region_attributes":{"name":"MauricioDeLaHoz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":542,"y":428,"width":38,"height":43},"region_attributes":{"name":"SergioFonseca","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":373,"y":425,"width":27,"height":32},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":613,"y":370,"width":35,"height":39},"region_attributes":{"name":"JuanMolina","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom3.jpg155001":{"filename":"classroom3.jpg","size":155001,"regions":[{"shape_attributes":{"name":"rect","x":526,"y":587,"width":59,"height":70},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":605,"y":499,"width":37,"height":47},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":771,"y":537,"width":43,"height":51},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":795,"y":452,"width":31,"height":39},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1156,"y":408,"width":23,"height":32},"region_attributes":{"name":"HabidAbdala","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1107,"y":430,"width":26,"height":32},"region_attributes":{"name":"AngieValencia","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1026,"y":479,"width":31,"height":37},"region_attributes":{"name":"AllysonSalom","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":356,"y":707,"width":66,"height":80},"region_attributes":{"name":"DylanDeOro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":60,"y":790,"width":76,"height":90},"region_attributes":{"name":"JuanLozano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom4.jpg156291":{"filename":"classroom4.jpg","size":156291,"regions":[{"shape_attributes":{"name":"rect","x":552,"y":592,"width":48,"height":62},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":609,"y":500,"width":35,"height":49},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":770,"y":538,"width":39,"height":51},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":795,"y":453,"width":28,"height":39},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1023,"y":480,"width":31,"height":37},"region_attributes":{"name":"AllysonSalom","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1108,"y":431,"width":25,"height":33},"region_attributes":{"name":"AngieValencia","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1155,"y":410,"width":25,"height":30},"region_attributes":{"name":"HabidAbdala","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom5.jpg156524":{"filename":"classroom5.jpg","size":156524,"regions":[{"shape_attributes":{"name":"rect","x":551,"y":592,"width":49,"height":59},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":769,"y":540,"width":42,"height":49},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":607,"y":500,"width":36,"height":45},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":795,"y":452,"width":29,"height":41},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1020,"y":480,"width":33,"height":39},"region_attributes":{"name":"AllysonSalom","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1108,"y":431,"width":26,"height":34},"region_attributes":{"name":"AngieValencia","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1156,"y":410,"width":25,"height":30},"region_attributes":{"name":"HabidAbdala","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom6.jpg153504":{"filename":"classroom6.jpg","size":153504,"regions":[{"shape_attributes":{"name":"rect","x":59,"y":845,"width":67,"height":74},"region_attributes":{"name":"MayraCarreno","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":221,"y":721,"width":58,"height":72},"region_attributes":{"name":"JuanSanchez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":434,"y":701,"width":50,"height":60},"region_attributes":{"name":"JuanQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":584,"y":657,"width":47,"height":47},"region_attributes":{"name":"SamirBarcelo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":772,"y":616,"width":42,"height":45},"region_attributes":{"name":"EdwinUtria","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":976,"y":539,"width":29,"height":37},"region_attributes":{"name":"OctavioMorales","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1077,"y":516,"width":26,"height":30},"region_attributes":{"name":"MariaFerrer","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":957,"y":472,"width":25,"height":28},"region_attributes":{"name":"SteevenBallena","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":903,"y":501,"width":23,"height":29},"region_attributes":{"name":"BrayanRubiano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":811,"y":526,"width":21,"height":29},"region_attributes":{"name":"AndreaQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":704,"y":532,"width":31,"height":34},"region_attributes":{"name":"JuanHernandez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":574,"y":567,"width":40,"height":49},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":343,"y":557,"width":27,"height":29},"region_attributes":{"name":"AndresFabregas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":243,"y":581,"width":30,"height":31},"region_attributes":{"name":"GabrielaBecerra","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":290,"y":520,"width":21,"height":26},"region_attributes":{"name":"AliRada","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":193,"y":553,"width":24,"height":26},"region_attributes":{"name":"LucianaDeLaRosa","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":155,"y":576,"width":31,"height":36},"region_attributes":{"name":"AndresNarvaez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom7.jpg155011":{"filename":"classroom7.jpg","size":155011,"regions":[{"shape_attributes":{"name":"rect","x":46,"y":849,"width":64,"height":79},"region_attributes":{"name":"MayraCarreno","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":219,"y":724,"width":58,"height":71},"region_attributes":{"name":"JuanSanchez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":430,"y":696,"width":55,"height":65},"region_attributes":{"name":"JuanQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":589,"y":661,"width":46,"height":49},"region_attributes":{"name":"SamirBarcelo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":771,"y":612,"width":44,"height":54},"region_attributes":{"name":"EdwinUtria","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":969,"y":539,"width":32,"height":39},"region_attributes":{"name":"OctavioMorales","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1069,"y":518,"width":29,"height":33},"region_attributes":{"name":"MariaFerrer","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":954,"y":475,"width":25,"height":28},"region_attributes":{"name":"SteevenBallena","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":898,"y":501,"width":27,"height":29},"region_attributes":{"name":"BrayanRubiano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":807,"y":527,"width":24,"height":29},"region_attributes":{"name":"AndreaQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":575,"y":569,"width":38,"height":47},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":706,"y":531,"width":31,"height":34},"region_attributes":{"name":"JuanHernandez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":343,"y":557,"width":27,"height":31},"region_attributes":{"name":"AndresFabregas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":290,"y":505,"width":24,"height":29},"region_attributes":{"name":"AliRada","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":244,"y":579,"width":28,"height":31},"region_attributes":{"name":"GabrielaBecerra","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":186,"y":555,"width":24,"height":26},"region_attributes":{"name":"LucianaDeLaRosa","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":134,"y":574,"width":31,"height":38},"region_attributes":{"name":"AndresNarvaez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom01001.jpg150320":{"filename":"classroom01001.jpg","size":150320,"regions":[{"shape_attributes":{"name":"rect","x":57,"y":806,"width":63,"height":74},"region_attributes":{"name":"JuanLozano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":426,"y":726,"width":44,"height":63},"region_attributes":{"name":"DylanDeOro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":568,"y":630,"width":62,"height":69},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":775,"y":555,"width":38,"height":50},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":609,"y":519,"width":41,"height":49},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":796,"y":465,"width":29,"height":44},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1090,"y":499,"width":27,"height":36},"region_attributes":{"name":"AllysonSalom","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1158,"y":416,"width":20,"height":32},"region_attributes":{"name":"HabidAbdala","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1144,"y":436,"width":19,"height":38},"region_attributes":{"name":"AngieValencia","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom10110.jpg149420":{"filename":"classroom10110.jpg","size":149420,"regions":[{"shape_attributes":{"name":"rect","x":67,"y":814,"width":53,"height":82},"region_attributes":{"name":"MayraCarreno","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":623,"y":650,"width":46,"height":55},"region_attributes":{"name":"SamirBarcelo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":128,"y":590,"width":30,"height":35},"region_attributes":{"name":"AndresNarvaez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":162,"y":570,"width":26,"height":29},"region_attributes":{"name":"LucianaDeLaRosa","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":223,"y":585,"width":25,"height":30},"region_attributes":{"name":"GabrielaBecerra","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":296,"y":541,"width":24,"height":28},"region_attributes":{"name":"AliRada","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":388,"y":577,"width":25,"height":31},"region_attributes":{"name":"AndresFabregas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":576,"y":567,"width":34,"height":51},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":814,"y":523,"width":23,"height":32},"region_attributes":{"name":"AndreaQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":882,"y":516,"width":23,"height":31},"region_attributes":{"name":"BrayanRubiano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":932,"y":494,"width":24,"height":30},"region_attributes":{"name":"SteevenBallena","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1078,"y":520,"width":26,"height":31},"region_attributes":{"name":"MariaFerrer","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1022,"y":566,"width":28,"height":28},"region_attributes":{"name":"OctavioMorales","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom28.jpg154093":{"filename":"classroom28.jpg","size":154093,"regions":[{"shape_attributes":{"name":"rect","x":22,"y":550,"width":31,"height":47},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":326,"y":507,"width":28,"height":35},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":136,"y":699,"width":56,"height":66},"region_attributes":{"name":"DavidChaparro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":92,"y":592,"width":45,"height":55},"region_attributes":{"name":"SergioFonseca","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":584,"y":607,"width":50,"height":60},"region_attributes":{"name":"CarlosBornacelly","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":515,"y":510,"width":42,"height":48},"region_attributes":{"name":"JesusCotes","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1145,"y":404,"width":25,"height":35},"region_attributes":{"name":"GiohanOlivares","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1006,"y":479,"width":33,"height":38},"region_attributes":{"name":"MariaLopez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":798,"y":471,"width":28,"height":35},"region_attributes":{"name":"AlbaQuiroz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":625,"y":438,"width":25,"height":33},"region_attributes":{"name":"JuanMolina","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":431,"y":432,"width":23,"height":28},"region_attributes":{"name":"JoseVilla","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":901,"y":583,"width":50,"height":47},"region_attributes":{"name":"ManuelRangel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}

]



In [74]:
true=True
faceGallery=[
    faceGalleryArray[9]
]
tuples=[]
m=0
for element in faceGallery:
    i=1
    filteredFaceGalleryObject=element[next(iter(element))]["regions"]
    tuplesnthPhoto=[]
    for element in filteredFaceGalleryObject:
        region=element["shape_attributes"]
        identity=element["region_attributes"]
        tuplesnthPhoto.append((identity["name"],region["x"],region["y"], region["width"], region["height"]))
        i=i+1
    tuples.append(tuplesnthPhoto)
    m=m+1

groundTruthNames=[]
for element in tuples[0]:
    groundTruthNames.append(element[0])


Reduce image's resolution until reaching Rekognition's limit

In [ ]:
selectedCourse="Robotics"
student_photos_path=os.path.join("..","ImagesRekognition",f"Students{selectedCourse}")
camera_frame=os.path.join("..","ImagesRekognition","RealClassroomPhotos","classroom11100.jpg")
camera_photo= Image.open(camera_frame)
cameraWidth,cameraHeight=camera_photo.size

with open(camera_frame, 'rb') as image_file:
    camera_frame_bytes = image_file.read()
results=[]

i=0
for student in os.listdir(student_photos_path):
    if student.split(".")[0] in groundTruthNames:
        if student.split(".")[1]=="jpeg":
            imageQuality=100
        elif student.split(".")[1]=="jpg":
            imageQuality=95
        multiplier=1
        #Define path to individual photo
        student_id_photo=os.path.join("..","imagesRekognition",f"Students{selectedCourse}",student)
        #Obtain image's dimensions in order to know if its size is valid for the minimal rekognition's requirements
        baseImage=Image.open(student_id_photo)
        initialWidth, initialHeight=baseImage.size
        initialBox=True

        while ((initialWidth/multiplier)>80 and (initialHeight/multiplier)>80):
            #Take base image, resize and save it according to a multiplier
            finalWidth=initialWidth/multiplier
            finalHeight=initialHeight/multiplier
            student_id_photo_resized=baseImage.resize((int(finalWidth),int(finalHeight)))
            student_id_photo_resized.save(f'{student}_scaled_{finalHeight}x{finalWidth}.jpg',quality=imageQuality)
            student_id_photo_resized_path=f'{student}_scaled_{finalHeight}x{finalWidth}.jpg'
            size=os.path.getsize(student_id_photo_resized_path)
            TPSimilarityAverage=0
            FPSimilarityAverage=0
            FPCounter=0
            TP=TN=FN=FP=0
            #Convert to bytes the resixed image
            with open(student_id_photo_resized_path, 'rb') as image_file:
                student_id_photo_resized_bytes = image_file.read()

            #Look for the resized students photo in the group photo, reduce threshold to obtain similarity info about all faces detected


            responseRecognition=client.compare_faces(
                        SourceImage={'Bytes': student_id_photo_resized_bytes},
                        TargetImage={'Bytes': camera_frame_bytes},
                        SimilarityThreshold=0
                    )
            
            #Save the corresponding face gallery coordinates in order to compare
            if (initialBox):
                for element in tuples[0]:
                    if element[0]==student.split(".")[0]:
                        trueBoundingBoxWithId=(student,element[1]/cameraWidth,element[2]/cameraHeight,
                                               element[3]/cameraWidth,element[4]/cameraHeight)
                        break

            initialBox=False

            print(f"Results for resolution {finalWidth}x{finalHeight}")

            for face in responseRecognition["FaceMatches"]:
                partialBB=face["Face"]["BoundingBox"]
                IoU=calculate_iou(trueBoundingBoxWithId, ("Name",partialBB["Left"],partialBB["Top"],partialBB["Width"],partialBB["Height"]))
                if (IoU>=0.5):
                    TPSimilarityAverage=TPSimilarityAverage+float(face["Similarity"])
                    if(face["Similarity"]>80):
                        TP=TP+1
                    print(f"{student.split(".")[0]} recognized in {partialBB} with a similarity of {face["Similarity"]}")
                else:
                    FPSimilarityAverage=FPSimilarityAverage+float(face["Similarity"])
                    FPCounter=FPCounter+1
                    if face["Similarity"]>80:
                        FP=FP+1
                        print(f"Error occurred, the model mismatched a face in {partialBB} with {face["Similarity"]}")
                    else:
                        TN=TN+1
            if (TP==0):
                FN=1
            results.append({"Name":student.split(".")[0],"Size": size,"Width": finalWidth, "Height": finalHeight, "TP Similarity": TPSimilarityAverage,
                            "Others Similarity Average": FPSimilarityAverage/FPCounter,"TP":TP,"TN":TN,"FP": FP,"FN":FN})
            multiplier=multiplier*2
    else:
        print(f"{student.split(".")[0]} not in ground truth array ")

    i=i+1


Save partial results

In [32]:
resolutionControlResults=[
[{'Name': 'AlbaQuiroz',
  'Size': 1480511,
  'Width': 2640.0,
  'Height': 1980.0,
  'TP Similarity': 88.90058135986328,
  'Others Similarity Average': 2.46839035153389,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlbaQuiroz',
  'Size': 411929,
  'Width': 1320.0,
  'Height': 990.0,
  'TP Similarity': 88.87728118896484,
  'Others Similarity Average': 2.348551980778575,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlbaQuiroz',
  'Size': 109834,
  'Width': 660.0,
  'Height': 495.0,
  'TP Similarity': 85.59800720214844,
  'Others Similarity Average': 2.098415069282055,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlbaQuiroz',
  'Size': 31881,
  'Width': 330.0,
  'Height': 247.5,
  'TP Similarity': 90.67321014404297,
  'Others Similarity Average': 2.9759058967232703,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlbaQuiroz',
  'Size': 10426,
  'Width': 165.0,
  'Height': 123.75,
  'TP Similarity': 94.1820297241211,
  'Others Similarity Average': 3.3088589154183863,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Size': 3009512,
  'Width': 3392.0,
  'Height': 2544.0,
  'TP Similarity': 99.59100341796875,
  'Others Similarity Average': 1.116161481104791,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Size': 503790,
  'Width': 1696.0,
  'Height': 1272.0,
  'TP Similarity': 99.63220977783203,
  'Others Similarity Average': 1.0466341201215983,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Size': 103610,
  'Width': 848.0,
  'Height': 636.0,
  'TP Similarity': 99.55142974853516,
  'Others Similarity Average': 0.9126466266810894,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Size': 27419,
  'Width': 424.0,
  'Height': 318.0,
  'TP Similarity': 99.6233139038086,
  'Others Similarity Average': 1.1585255263373255,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Size': 8691,
  'Width': 212.0,
  'Height': 159.0,
  'TP Similarity': 99.765869140625,
  'Others Similarity Average': 1.4512335427105427,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Size': 2493504,
  'Width': 4080.0,
  'Height': 2296.0,
  'TP Similarity': 99.83390045166016,
  'Others Similarity Average': 0.9993078446947038,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Size': 670114,
  'Width': 2040.0,
  'Height': 1148.0,
  'TP Similarity': 99.88066101074219,
  'Others Similarity Average': 1.0361146867275237,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Size': 178216,
  'Width': 1020.0,
  'Height': 574.0,
  'TP Similarity': 99.880859375,
  'Others Similarity Average': 1.1565723842941225,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Size': 47146,
  'Width': 510.0,
  'Height': 287.0,
  'TP Similarity': 99.91187286376953,
  'Others Similarity Average': 1.745438533090055,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Size': 13775,
  'Width': 255.0,
  'Height': 143.5,
  'TP Similarity': 99.91368103027344,
  'Others Similarity Average': 1.631252614222467,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Size': 681421,
  'Width': 1728.0,
  'Height': 2304.0,
  'TP Similarity': 95.24468994140625,
  'Others Similarity Average': 3.114847409725189,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Size': 252306,
  'Width': 864.0,
  'Height': 1152.0,
  'TP Similarity': 93.95336151123047,
  'Others Similarity Average': 3.061367241293192,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Size': 62181,
  'Width': 432.0,
  'Height': 576.0,
  'TP Similarity': 93.93663024902344,
  'Others Similarity Average': 3.005732050538063,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Size': 18347,
  'Width': 216.0,
  'Height': 288.0,
  'TP Similarity': 97.13996887207031,
  'Others Similarity Average': 3.359985103458166,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Size': 6506,
  'Width': 108.0,
  'Height': 144.0,
  'TP Similarity': 98.57781219482422,
  'Others Similarity Average': 3.7353317491710185,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Size': 2210501,
  'Width': 1448.0,
  'Height': 2573.0,
  'TP Similarity': 92.88935089111328,
  'Others Similarity Average': 1.937134314328432,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Size': 563068,
  'Width': 724.0,
  'Height': 1286.5,
  'TP Similarity': 93.37411499023438,
  'Others Similarity Average': 1.9045796804130077,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Size': 147200,
  'Width': 362.0,
  'Height': 643.25,
  'TP Similarity': 95.51879119873047,
  'Others Similarity Average': 2.436876054853201,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Size': 42406,
  'Width': 181.0,
  'Height': 321.625,
  'TP Similarity': 97.00713348388672,
  'Others Similarity Average': 2.5822086304426195,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Size': 13126,
  'Width': 90.5,
  'Height': 160.8125,
  'TP Similarity': 97.75584411621094,
  'Others Similarity Average': 2.7143202677369116,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 2656147,
  'Width': 1815.0,
  'Height': 2420.0,
  'TP Similarity': 99.98616027832031,
  'Others Similarity Average': 3.7006253257393835,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 678168,
  'Width': 907.5,
  'Height': 1210.0,
  'TP Similarity': 99.98738861083984,
  'Others Similarity Average': 3.5045509047806265,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 167717,
  'Width': 453.75,
  'Height': 605.0,
  'TP Similarity': 99.98947143554688,
  'Others Similarity Average': 3.444971077889204,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 43922,
  'Width': 226.875,
  'Height': 302.5,
  'TP Similarity': 99.99185180664062,
  'Others Similarity Average': 3.151020758599043,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 12822,
  'Width': 113.4375,
  'Height': 151.25,
  'TP Similarity': 99.98652648925781,
  'Others Similarity Average': 3.881474418193102,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Size': 406218,
  'Width': 960.0,
  'Height': 1280.0,
  'TP Similarity': 99.97025299072266,
  'Others Similarity Average': 1.6272360134869814,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Size': 176177,
  'Width': 480.0,
  'Height': 640.0,
  'TP Similarity': 99.97370910644531,
  'Others Similarity Average': 1.5945897918194532,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Size': 46878,
  'Width': 240.0,
  'Height': 320.0,
  'TP Similarity': 99.99266052246094,
  'Others Similarity Average': 2.089319349452853,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Size': 13242,
  'Width': 120.0,
  'Height': 160.0,
  'TP Similarity': 99.99169921875,
  'Others Similarity Average': 1.930557368323207,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'EstebanToro',
  'Size': 406528,
  'Width': 1158.0,
  'Height': 1544.0,
  'TP Similarity': 46.63393783569336,
  'Others Similarity Average': 2.2701166439801455,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'EstebanToro',
  'Size': 197793,
  'Width': 579.0,
  'Height': 772.0,
  'TP Similarity': 43.257606506347656,
  'Others Similarity Average': 2.3949098315089943,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'EstebanToro',
  'Size': 61009,
  'Width': 289.5,
  'Height': 386.0,
  'TP Similarity': 71.0179672241211,
  'Others Similarity Average': 3.9986398931592704,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'EstebanToro',
  'Size': 18113,
  'Width': 144.75,
  'Height': 193.0,
  'TP Similarity': 83.1950912475586,
  'Others Similarity Average': 5.57758966833353,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Size': 499704,
  'Width': 1560.0,
  'Height': 2104.0,
  'TP Similarity': 89.69254302978516,
  'Others Similarity Average': 0.9426450707018376,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Size': 218030,
  'Width': 780.0,
  'Height': 1052.0,
  'TP Similarity': 90.51416778564453,
  'Others Similarity Average': 0.9379294890910387,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Size': 65930,
  'Width': 390.0,
  'Height': 526.0,
  'TP Similarity': 97.17056274414062,
  'Others Similarity Average': 1.3138406869024037,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Size': 20995,
  'Width': 195.0,
  'Height': 263.0,
  'TP Similarity': 98.06564331054688,
  'Others Similarity Average': 1.3007290121167898,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Size': 6987,
  'Width': 97.5,
  'Height': 131.5,
  'TP Similarity': 99.67396545410156,
  'Others Similarity Average': 2.9415776036679744,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 1753278,
  'Width': 1448.0,
  'Height': 2573.0,
  'TP Similarity': 98.1666030883789,
  'Others Similarity Average': 3.113985112309456,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 501727,
  'Width': 724.0,
  'Height': 1286.5,
  'TP Similarity': 98.16162872314453,
  'Others Similarity Average': 2.9970393002033235,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 145044,
  'Width': 362.0,
  'Height': 643.25,
  'TP Similarity': 98.90632629394531,
  'Others Similarity Average': 3.6663398489356043,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 43406,
  'Width': 181.0,
  'Height': 321.625,
  'TP Similarity': 98.75383758544922,
  'Others Similarity Average': 5.166873200237751,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 13749,
  'Width': 90.5,
  'Height': 160.8125,
  'TP Similarity': 99.59549713134766,
  'Others Similarity Average': 6.05631552785635,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Size': 153406,
  'Width': 900.0,
  'Height': 1600.0,
  'TP Similarity': 99.7214126586914,
  'Others Similarity Average': 1.0196609620004893,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Size': 76288,
  'Width': 450.0,
  'Height': 800.0,
  'TP Similarity': 99.84765625,
  'Others Similarity Average': 1.004794950503856,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Size': 23481,
  'Width': 225.0,
  'Height': 400.0,
  'TP Similarity': 99.94078826904297,
  'Others Similarity Average': 1.349409263394773,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Size': 7635,
  'Width': 112.5,
  'Height': 200.0,
  'TP Similarity': 99.96001434326172,
  'Others Similarity Average': 1.8340937487781048,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Size': 2494917,
  'Width': 1815.0,
  'Height': 2420.0,
  'TP Similarity': 94.66946411132812,
  'Others Similarity Average': 2.4856734005734324,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Size': 648495,
  'Width': 907.5,
  'Height': 1210.0,
  'TP Similarity': 95.1860580444336,
  'Others Similarity Average': 2.4531746277585627,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Size': 168647,
  'Width': 453.75,
  'Height': 605.0,
  'TP Similarity': 94.4015884399414,
  'Others Similarity Average': 2.2673211505636575,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Size': 46372,
  'Width': 226.875,
  'Height': 302.5,
  'TP Similarity': 96.17891693115234,
  'Others Similarity Average': 2.8334007896482944,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Size': 14192,
  'Width': 113.4375,
  'Height': 151.25,
  'TP Similarity': 97.4831771850586,
  'Others Similarity Average': 3.578752299956977,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 1309286,
  'Width': 2448.0,
  'Height': 2448.0,
  'TP Similarity': 93.04896545410156,
  'Others Similarity Average': 1.9102924808859825,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 283578,
  'Width': 1224.0,
  'Height': 1224.0,
  'TP Similarity': 92.34402465820312,
  'Others Similarity Average': 2.2933120910078286,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 69647,
  'Width': 612.0,
  'Height': 612.0,
  'TP Similarity': 92.72358703613281,
  'Others Similarity Average': 2.090443298779428,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 20186,
  'Width': 306.0,
  'Height': 306.0,
  'TP Similarity': 92.7055435180664,
  'Others Similarity Average': 2.508882932923734,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 6921,
  'Width': 153.0,
  'Height': 153.0,
  'TP Similarity': 94.6355972290039,
  'Others Similarity Average': 3.2718039494007827,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Size': 56945,
  'Width': 709.0,
  'Height': 945.0,
  'TP Similarity': 99.93087005615234,
  'Others Similarity Average': 4.026283391565085,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Size': 28130,
  'Width': 354.5,
  'Height': 472.5,
  'TP Similarity': 99.9192123413086,
  'Others Similarity Average': 4.797521563619375,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Size': 10527,
  'Width': 177.25,
  'Height': 236.25,
  'TP Similarity': 99.94037628173828,
  'Others Similarity Average': 5.11220781058073,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Size': 4245,
  'Width': 88.625,
  'Height': 118.125,
  'TP Similarity': 99.92469787597656,
  'Others Similarity Average': 6.5124119311571125,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaBerdugo',
  'Size': 295314,
  'Width': 682.0,
  'Height': 910.0,
  'TP Similarity': 91.10781860351562,
  'Others Similarity Average': 3.066073393076658,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaBerdugo',
  'Size': 95993,
  'Width': 341.0,
  'Height': 455.0,
  'TP Similarity': 92.25030517578125,
  'Others Similarity Average': 2.7400142408907415,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaBerdugo',
  'Size': 28116,
  'Width': 170.5,
  'Height': 227.5,
  'TP Similarity': 95.22795104980469,
  'Others Similarity Average': 2.8407340236008167,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaBerdugo',
  'Size': 8931,
  'Width': 85.25,
  'Height': 113.75,
  'TP Similarity': 97.25487518310547,
  'Others Similarity Average': 2.627590428292751,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 1234792,
  'Width': 2640.0,
  'Height': 1980.0,
  'TP Similarity': 99.13067626953125,
  'Others Similarity Average': 1.3595963045023383,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 320733,
  'Width': 1320.0,
  'Height': 990.0,
  'TP Similarity': 99.21927642822266,
  'Others Similarity Average': 1.3524278529919684,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 88227,
  'Width': 660.0,
  'Height': 495.0,
  'TP Similarity': 99.23494720458984,
  'Others Similarity Average': 1.2860210153274239,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 25172,
  'Width': 330.0,
  'Height': 247.5,
  'TP Similarity': 99.68449401855469,
  'Others Similarity Average': 1.5238568065688014,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 8658,
  'Width': 165.0,
  'Height': 123.75,
  'TP Similarity': 99.86618041992188,
  'Others Similarity Average': 1.5352014377713203,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MauricioDeLaHoz',
  'Size': 805111,
  'Width': 1200.0,
  'Height': 1600.0,
  'TP Similarity': 86.22069549560547,
  'Others Similarity Average': 1.68244486823678,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MauricioDeLaHoz',
  'Size': 307452,
  'Width': 600.0,
  'Height': 800.0,
  'TP Similarity': 85.17274475097656,
  'Others Similarity Average': 1.6740729995071888,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MauricioDeLaHoz',
  'Size': 91433,
  'Width': 300.0,
  'Height': 400.0,
  'TP Similarity': 88.38228607177734,
  'Others Similarity Average': 1.9132840916514398,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MauricioDeLaHoz',
  'Size': 27666,
  'Width': 150.0,
  'Height': 200.0,
  'TP Similarity': 87.17445373535156,
  'Others Similarity Average': 1.9520578861236573,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 1139778,
  'Width': 1440.0,
  'Height': 2559.0,
  'TP Similarity': 99.54988098144531,
  'Others Similarity Average': 2.263835819810629,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 523623,
  'Width': 720.0,
  'Height': 1279.5,
  'TP Similarity': 99.52462768554688,
  'Others Similarity Average': 2.2391146156936883,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 146653,
  'Width': 360.0,
  'Height': 639.75,
  'TP Similarity': 99.45360565185547,
  'Others Similarity Average': 2.0209351809695364,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 42367,
  'Width': 180.0,
  'Height': 319.875,
  'TP Similarity': 99.58377838134766,
  'Others Similarity Average': 2.170742596685886,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 12835,
  'Width': 90.0,
  'Height': 159.9375,
  'TP Similarity': 99.79711151123047,
  'Others Similarity Average': 3.432893993705511,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 543971,
  'Width': 1740.0,
  'Height': 2319.0,
  'TP Similarity': 98.95976257324219,
  'Others Similarity Average': 1.1744868079200388,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 236961,
  'Width': 870.0,
  'Height': 1159.5,
  'TP Similarity': 98.88408660888672,
  'Others Similarity Average': 1.2100826004520058,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 62035,
  'Width': 435.0,
  'Height': 579.75,
  'TP Similarity': 98.80642700195312,
  'Others Similarity Average': 1.7245452165603639,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 17872,
  'Width': 217.5,
  'Height': 289.875,
  'TP Similarity': 98.79564666748047,
  'Others Similarity Average': 1.5794075606390834,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 6081,
  'Width': 108.75,
  'Height': 144.9375,
  'TP Similarity': 99.1836929321289,
  'Others Similarity Average': 2.143058940768242,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0}]
    ,
[{'Name': 'AlbaQuiroz',
  'Size': 1480511,
  'Width': 2640.0,
  'Height': 1980.0,
  'TP Similarity': 83.20394897460938,
  'Others Similarity Average': 1.1827100519090892,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlbaQuiroz',
  'Size': 411929,
  'Width': 1320.0,
  'Height': 990.0,
  'TP Similarity': 82.75366973876953,
  'Others Similarity Average': 1.0727916441857814,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlbaQuiroz',
  'Size': 109834,
  'Width': 660.0,
  'Height': 495.0,
  'TP Similarity': 80.27983093261719,
  'Others Similarity Average': 1.0389086041599511,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlbaQuiroz',
  'Size': 31881,
  'Width': 330.0,
  'Height': 247.5,
  'TP Similarity': 87.13426208496094,
  'Others Similarity Average': 1.3756762444972992,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlbaQuiroz',
  'Size': 10426,
  'Width': 165.0,
  'Height': 123.75,
  'TP Similarity': 94.67363739013672,
  'Others Similarity Average': 1.6057271648198366,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Size': 3009512,
  'Width': 3392.0,
  'Height': 2544.0,
  'TP Similarity': 85.64579772949219,
  'Others Similarity Average': 1.1528775874525308,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Size': 503790,
  'Width': 1696.0,
  'Height': 1272.0,
  'TP Similarity': 86.24542236328125,
  'Others Similarity Average': 1.1427059032022953,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Size': 103610,
  'Width': 848.0,
  'Height': 636.0,
  'TP Similarity': 85.27627563476562,
  'Others Similarity Average': 1.0195453561842442,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Size': 27419,
  'Width': 424.0,
  'Height': 318.0,
  'TP Similarity': 86.31083679199219,
  'Others Similarity Average': 1.3421583257615566,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Size': 8691,
  'Width': 212.0,
  'Height': 159.0,
  'TP Similarity': 88.333740234375,
  'Others Similarity Average': 1.6272428408265114,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Size': 2493504,
  'Width': 4080.0,
  'Height': 2296.0,
  'TP Similarity': 99.79627227783203,
  'Others Similarity Average': 0.984981439076364,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Size': 670114,
  'Width': 2040.0,
  'Height': 1148.0,
  'TP Similarity': 99.81233978271484,
  'Others Similarity Average': 0.996704763546586,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Size': 178216,
  'Width': 1020.0,
  'Height': 574.0,
  'TP Similarity': 99.84548950195312,
  'Others Similarity Average': 1.0830789929255844,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Size': 47146,
  'Width': 510.0,
  'Height': 287.0,
  'TP Similarity': 99.86063385009766,
  'Others Similarity Average': 1.5730882443487644,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Size': 13775,
  'Width': 255.0,
  'Height': 143.5,
  'TP Similarity': 99.89300537109375,
  'Others Similarity Average': 1.4632713694125414,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Size': 681421,
  'Width': 1728.0,
  'Height': 2304.0,
  'TP Similarity': 85.77983093261719,
  'Others Similarity Average': 3.140716354548931,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Size': 252306,
  'Width': 864.0,
  'Height': 1152.0,
  'TP Similarity': 84.06564331054688,
  'Others Similarity Average': 3.133885638229549,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Size': 62181,
  'Width': 432.0,
  'Height': 576.0,
  'TP Similarity': 83.77474975585938,
  'Others Similarity Average': 2.734885105676949,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Size': 18347,
  'Width': 216.0,
  'Height': 288.0,
  'TP Similarity': 92.14131927490234,
  'Others Similarity Average': 3.55494830198586,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Size': 6506,
  'Width': 108.0,
  'Height': 144.0,
  'TP Similarity': 96.36589050292969,
  'Others Similarity Average': 3.595156991109252,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Size': 2210501,
  'Width': 1448.0,
  'Height': 2573.0,
  'TP Similarity': 92.47247314453125,
  'Others Similarity Average': 2.2658330842852594,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Size': 563068,
  'Width': 724.0,
  'Height': 1286.5,
  'TP Similarity': 93.20973205566406,
  'Others Similarity Average': 2.2271247915923595,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Size': 147200,
  'Width': 362.0,
  'Height': 643.25,
  'TP Similarity': 95.38751983642578,
  'Others Similarity Average': 3.0485570944845675,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Size': 42406,
  'Width': 181.0,
  'Height': 321.625,
  'TP Similarity': 97.38607025146484,
  'Others Similarity Average': 3.1799647852778437,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Size': 13126,
  'Width': 90.5,
  'Height': 160.8125,
  'TP Similarity': 97.97149658203125,
  'Others Similarity Average': 3.2200575530529023,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 2656147,
  'Width': 1815.0,
  'Height': 2420.0,
  'TP Similarity': 99.97511291503906,
  'Others Similarity Average': 3.2863854110240935,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 678168,
  'Width': 907.5,
  'Height': 1210.0,
  'TP Similarity': 99.98091888427734,
  'Others Similarity Average': 2.926190160214901,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 167717,
  'Width': 453.75,
  'Height': 605.0,
  'TP Similarity': 99.98099517822266,
  'Others Similarity Average': 2.6911584116518497,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 43922,
  'Width': 226.875,
  'Height': 302.5,
  'TP Similarity': 99.98690032958984,
  'Others Similarity Average': 2.7426745750010015,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 12822,
  'Width': 113.4375,
  'Height': 151.25,
  'TP Similarity': 99.97799682617188,
  'Others Similarity Average': 3.360512676090002,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Size': 406218,
  'Width': 960.0,
  'Height': 1280.0,
  'TP Similarity': 99.99837493896484,
  'Others Similarity Average': 2.551242534443736,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Size': 176177,
  'Width': 480.0,
  'Height': 640.0,
  'TP Similarity': 99.99861145019531,
  'Others Similarity Average': 2.5477589923888444,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Size': 46878,
  'Width': 240.0,
  'Height': 320.0,
  'TP Similarity': 99.99935913085938,
  'Others Similarity Average': 2.7223513260483743,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Size': 13242,
  'Width': 120.0,
  'Height': 160.0,
  'TP Similarity': 99.99908447265625,
  'Others Similarity Average': 2.394521977007389,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'EstebanToro',
  'Size': 406528,
  'Width': 1158.0,
  'Height': 1544.0,
  'TP Similarity': 24.850305557250977,
  'Others Similarity Average': 1.6295249566435814,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'EstebanToro',
  'Size': 197793,
  'Width': 579.0,
  'Height': 772.0,
  'TP Similarity': 22.22709083557129,
  'Others Similarity Average': 1.7777831643819808,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'EstebanToro',
  'Size': 61009,
  'Width': 289.5,
  'Height': 386.0,
  'TP Similarity': 37.236732482910156,
  'Others Similarity Average': 2.547832150757313,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'EstebanToro',
  'Size': 18113,
  'Width': 144.75,
  'Height': 193.0,
  'TP Similarity': 45.19512939453125,
  'Others Similarity Average': 3.4809620469808578,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielCastanez',
  'Size': 499704,
  'Width': 1560.0,
  'Height': 2104.0,
  'TP Similarity': 89.65965270996094,
  'Others Similarity Average': 0.6795554909855127,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Size': 218030,
  'Width': 780.0,
  'Height': 1052.0,
  'TP Similarity': 89.09134674072266,
  'Others Similarity Average': 0.6984400000423193,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Size': 65930,
  'Width': 390.0,
  'Height': 526.0,
  'TP Similarity': 96.39202880859375,
  'Others Similarity Average': 0.9810973484069109,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Size': 20995,
  'Width': 195.0,
  'Height': 263.0,
  'TP Similarity': 96.87889862060547,
  'Others Similarity Average': 1.0536024786531926,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Size': 6987,
  'Width': 97.5,
  'Height': 131.5,
  'TP Similarity': 98.50534057617188,
  'Others Similarity Average': 2.249524240940809,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 1753278,
  'Width': 1448.0,
  'Height': 2573.0,
  'TP Similarity': 93.90735626220703,
  'Others Similarity Average': 3.351626097410917,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 501727,
  'Width': 724.0,
  'Height': 1286.5,
  'TP Similarity': 93.73417663574219,
  'Others Similarity Average': 3.297337343543768,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 145044,
  'Width': 362.0,
  'Height': 643.25,
  'TP Similarity': 96.29671478271484,
  'Others Similarity Average': 3.956879562139511,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 43406,
  'Width': 181.0,
  'Height': 321.625,
  'TP Similarity': 97.49388122558594,
  'Others Similarity Average': 5.416099363565445,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 13749,
  'Width': 90.5,
  'Height': 160.8125,
  'TP Similarity': 99.18865203857422,
  'Others Similarity Average': 6.833860555291176,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Size': 153406,
  'Width': 900.0,
  'Height': 1600.0,
  'TP Similarity': 99.82682037353516,
  'Others Similarity Average': 0.9946721397340298,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Size': 76288,
  'Width': 450.0,
  'Height': 800.0,
  'TP Similarity': 99.81456756591797,
  'Others Similarity Average': 1.0598905311897397,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Size': 23481,
  'Width': 225.0,
  'Height': 400.0,
  'TP Similarity': 99.9118423461914,
  'Others Similarity Average': 1.311674016341567,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Size': 7635,
  'Width': 112.5,
  'Height': 200.0,
  'TP Similarity': 99.93370056152344,
  'Others Similarity Average': 1.2462817434221507,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Size': 2494917,
  'Width': 1815.0,
  'Height': 2420.0,
  'TP Similarity': 90.19048309326172,
  'Others Similarity Average': 2.4458005324006082,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Size': 648495,
  'Width': 907.5,
  'Height': 1210.0,
  'TP Similarity': 90.86630249023438,
  'Others Similarity Average': 2.380746265500784,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Size': 168647,
  'Width': 453.75,
  'Height': 605.0,
  'TP Similarity': 89.60079193115234,
  'Others Similarity Average': 2.04707684032619,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Size': 46372,
  'Width': 226.875,
  'Height': 302.5,
  'TP Similarity': 92.52606964111328,
  'Others Similarity Average': 2.8114303547888992,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Size': 14192,
  'Width': 113.4375,
  'Height': 151.25,
  'TP Similarity': 93.95025634765625,
  'Others Similarity Average': 3.326995212212205,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 1309286,
  'Width': 2448.0,
  'Height': 2448.0,
  'TP Similarity': 95.58659362792969,
  'Others Similarity Average': 1.1747006580233574,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 283578,
  'Width': 1224.0,
  'Height': 1224.0,
  'TP Similarity': 95.5313949584961,
  'Others Similarity Average': 1.2101244114339351,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 69647,
  'Width': 612.0,
  'Height': 612.0,
  'TP Similarity': 96.06659698486328,
  'Others Similarity Average': 1.1548402439802885,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 20186,
  'Width': 306.0,
  'Height': 306.0,
  'TP Similarity': 96.77803802490234,
  'Others Similarity Average': 1.2452124807983638,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 6921,
  'Width': 153.0,
  'Height': 153.0,
  'TP Similarity': 98.1744384765625,
  'Others Similarity Average': 1.5943113092333079,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Size': 56945,
  'Width': 709.0,
  'Height': 945.0,
  'TP Similarity': 99.98370361328125,
  'Others Similarity Average': 5.73456010222435,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Size': 28130,
  'Width': 354.5,
  'Height': 472.5,
  'TP Similarity': 99.9823226928711,
  'Others Similarity Average': 6.7587379440665245,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Size': 10527,
  'Width': 177.25,
  'Height': 236.25,
  'TP Similarity': 99.9863052368164,
  'Others Similarity Average': 6.91877695992589,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Size': 4245,
  'Width': 88.625,
  'Height': 118.125,
  'TP Similarity': 99.96077728271484,
  'Others Similarity Average': 7.439024039357901,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaBerdugo',
  'Size': 295314,
  'Width': 682.0,
  'Height': 910.0,
  'TP Similarity': 69.69281768798828,
  'Others Similarity Average': 2.3489835422486065,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MariaBerdugo',
  'Size': 95993,
  'Width': 341.0,
  'Height': 455.0,
  'TP Similarity': 68.46001434326172,
  'Others Similarity Average': 2.4023231767117976,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MariaBerdugo',
  'Size': 28116,
  'Width': 170.5,
  'Height': 227.5,
  'TP Similarity': 75.68578338623047,
  'Others Similarity Average': 2.5406573489308357,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MariaBerdugo',
  'Size': 8931,
  'Width': 85.25,
  'Height': 113.75,
  'TP Similarity': 80.98726654052734,
  'Others Similarity Average': 2.5777169175446035,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 1234792,
  'Width': 2640.0,
  'Height': 1980.0,
  'TP Similarity': 95.8797607421875,
  'Others Similarity Average': 0.8239467702805996,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 320733,
  'Width': 1320.0,
  'Height': 990.0,
  'TP Similarity': 96.8438949584961,
  'Others Similarity Average': 0.8226736757904292,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 88227,
  'Width': 660.0,
  'Height': 495.0,
  'TP Similarity': 96.31365203857422,
  'Others Similarity Average': 0.7712897799909115,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 25172,
  'Width': 330.0,
  'Height': 247.5,
  'TP Similarity': 97.99603271484375,
  'Others Similarity Average': 0.9374587096273899,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 8658,
  'Width': 165.0,
  'Height': 123.75,
  'TP Similarity': 99.00933837890625,
  'Others Similarity Average': 1.043074905127287,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MauricioDeLaHoz',
  'Size': 805111,
  'Width': 1200.0,
  'Height': 1600.0,
  'TP Similarity': 48.60635757446289,
  'Others Similarity Average': 1.4447561271488667,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MauricioDeLaHoz',
  'Size': 307452,
  'Width': 600.0,
  'Height': 800.0,
  'TP Similarity': 48.92091751098633,
  'Others Similarity Average': 1.4664651293307542,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MauricioDeLaHoz',
  'Size': 91433,
  'Width': 300.0,
  'Height': 400.0,
  'TP Similarity': 57.61127471923828,
  'Others Similarity Average': 1.5603970430791378,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MauricioDeLaHoz',
  'Size': 27666,
  'Width': 150.0,
  'Height': 200.0,
  'TP Similarity': 57.38005828857422,
  'Others Similarity Average': 1.6117106508463621,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'SergioFonseca',
  'Size': 1139778,
  'Width': 1440.0,
  'Height': 2559.0,
  'TP Similarity': 99.69623565673828,
  'Others Similarity Average': 3.843785073980689,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 523623,
  'Width': 720.0,
  'Height': 1279.5,
  'TP Similarity': 99.64881134033203,
  'Others Similarity Average': 3.760669694840908,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 146653,
  'Width': 360.0,
  'Height': 639.75,
  'TP Similarity': 99.63636779785156,
  'Others Similarity Average': 3.3368470948189497,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 42367,
  'Width': 180.0,
  'Height': 319.875,
  'TP Similarity': 99.70448303222656,
  'Others Similarity Average': 3.6845561444759367,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 12835,
  'Width': 90.0,
  'Height': 159.9375,
  'TP Similarity': 99.85037231445312,
  'Others Similarity Average': 6.23372248224914,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 974769,
  'Width': 1740.0,
  'Height': 2319.0,
  'TP Similarity': 96.14613342285156,
  'Others Similarity Average': 1.3920975841581822,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 479683,
  'Width': 870.0,
  'Height': 1159.5,
  'TP Similarity': 95.90391540527344,
  'Others Similarity Average': 1.349291417002678,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 139486,
  'Width': 435.0,
  'Height': 579.75,
  'TP Similarity': 96.27154541015625,
  'Others Similarity Average': 1.8351430505514146,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 38439,
  'Width': 217.5,
  'Height': 289.875,
  'TP Similarity': 96.18267822265625,
  'Others Similarity Average': 2.25339385792613,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 11696,
  'Width': 108.75,
  'Height': 144.9375,
  'TP Similarity': 97.1167221069336,
  'Others Similarity Average': 2.5897349663078786,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0}]
    ,
[{'Name': 'AllysonSalom',
  'Size': 1310936,
  'Width': 2576.0,
  'Height': 1932.0,
  'TP Similarity': 98.53770446777344,
  'Others Similarity Average': 1.532307855784893,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Size': 343350,
  'Width': 1288.0,
  'Height': 966.0,
  'TP Similarity': 99.17543029785156,
  'Others Similarity Average': 1.2253220435231924,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Size': 82816,
  'Width': 644.0,
  'Height': 483.0,
  'TP Similarity': 98.78667449951172,
  'Others Similarity Average': 1.0896438490599394,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Size': 20953,
  'Width': 322.0,
  'Height': 241.5,
  'TP Similarity': 98.90978240966797,
  'Others Similarity Average': 1.1025170255452394,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Size': 7051,
  'Width': 161.0,
  'Height': 120.75,
  'TP Similarity': 99.25872039794922,
  'Others Similarity Average': 1.1525273965671659,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Size': 1681946,
  'Width': 2944.0,
  'Height': 2208.0,
  'TP Similarity': 96.99846649169922,
  'Others Similarity Average': 0.5451221261173487,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Size': 434127,
  'Width': 1472.0,
  'Height': 1104.0,
  'TP Similarity': 96.77735900878906,
  'Others Similarity Average': 0.5522511582821608,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Size': 104017,
  'Width': 736.0,
  'Height': 552.0,
  'TP Similarity': 96.34819030761719,
  'Others Similarity Average': 0.5623830519616604,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Size': 26896,
  'Width': 368.0,
  'Height': 276.0,
  'TP Similarity': 97.75481414794922,
  'Others Similarity Average': 0.5737399626523256,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Size': 8386,
  'Width': 184.0,
  'Height': 138.0,
  'TP Similarity': 97.2166976928711,
  'Others Similarity Average': 0.5151427872478962,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 2782796,
  'Width': 1815.0,
  'Height': 2420.0,
  'TP Similarity': 99.99827575683594,
  'Others Similarity Average': 2.358381826430559,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 741937,
  'Width': 907.5,
  'Height': 1210.0,
  'TP Similarity': 99.99786376953125,
  'Others Similarity Average': 2.392451196908951,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 191965,
  'Width': 453.75,
  'Height': 605.0,
  'TP Similarity': 99.99866485595703,
  'Others Similarity Average': 2.2249677181243896,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 54231,
  'Width': 226.875,
  'Height': 302.5,
  'TP Similarity': 99.99864196777344,
  'Others Similarity Average': 3.2388573102653027,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 16898,
  'Width': 113.4375,
  'Height': 151.25,
  'TP Similarity': 99.9977798461914,
  'Others Similarity Average': 3.7815046943724155,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Size': 7952,
  'Width': 120.0,
  'Height': 120.0,
  'TP Similarity': 99.92094421386719,
  'Others Similarity Average': 1.1138332807458937,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DylanDeOro',
  'Size': 1940671,
  'Width': 2598.0,
  'Height': 3464.0,
  'TP Similarity': 99.99288177490234,
  'Others Similarity Average': 4.462024740874767,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DylanDeOro',
  'Size': 614079,
  'Width': 1299.0,
  'Height': 1732.0,
  'TP Similarity': 99.99242401123047,
  'Others Similarity Average': 4.448854621499777,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DylanDeOro',
  'Size': 181059,
  'Width': 649.5,
  'Height': 866.0,
  'TP Similarity': 99.99388122558594,
  'Others Similarity Average': 4.327168222516775,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DylanDeOro',
  'Size': 52010,
  'Width': 324.75,
  'Height': 433.0,
  'TP Similarity': 99.99139404296875,
  'Others Similarity Average': 4.058821618556976,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DylanDeOro',
  'Size': 16628,
  'Width': 162.375,
  'Height': 216.5,
  'TP Similarity': 99.9903793334961,
  'Others Similarity Average': 4.737360790371895,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DylanDeOro',
  'Size': 6105,
  'Width': 81.1875,
  'Height': 108.25,
  'TP Similarity': 99.98680114746094,
  'Others Similarity Average': 5.273266561329365,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 1184660,
  'Width': 2576.0,
  'Height': 1932.0,
  'TP Similarity': 99.83631134033203,
  'Others Similarity Average': 4.345866221934557,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 324270,
  'Width': 1288.0,
  'Height': 966.0,
  'TP Similarity': 99.8289566040039,
  'Others Similarity Average': 4.074006658047438,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 84175,
  'Width': 644.0,
  'Height': 483.0,
  'TP Similarity': 99.76625061035156,
  'Others Similarity Average': 3.9028295055031776,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 23069,
  'Width': 322.0,
  'Height': 241.5,
  'TP Similarity': 99.8050308227539,
  'Others Similarity Average': 3.949507884681225,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 7265,
  'Width': 161.0,
  'Height': 120.75,
  'TP Similarity': 99.9368667602539,
  'Others Similarity Average': 4.126785218715668,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Size': 2498409,
  'Width': 3392.0,
  'Height': 2544.0,
  'TP Similarity': 85.91094970703125,
  'Others Similarity Average': 1.8382618464529514,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Size': 524959,
  'Width': 1696.0,
  'Height': 1272.0,
  'TP Similarity': 77.27352905273438,
  'Others Similarity Average': 1.4621236082166433,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'HabidAbdala',
  'Size': 120010,
  'Width': 848.0,
  'Height': 636.0,
  'TP Similarity': 84.22045135498047,
  'Others Similarity Average': 1.4594266302883625,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Size': 32013,
  'Width': 424.0,
  'Height': 318.0,
  'TP Similarity': 87.52764129638672,
  'Others Similarity Average': 1.579133266583085,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Size': 10355,
  'Width': 212.0,
  'Height': 159.0,
  'TP Similarity': 91.10663604736328,
  'Others Similarity Average': 1.7235442008823156,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanLozano',
  'Size': 710063,
  'Width': 1179.0,
  'Height': 1329.0,
  'TP Similarity': 99.98926544189453,
  'Others Similarity Average': 1.3084623273462057,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanLozano',
  'Size': 244453,
  'Width': 589.5,
  'Height': 664.5,
  'TP Similarity': 99.98858642578125,
  'Others Similarity Average': 1.42935311794281,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanLozano',
  'Size': 72867,
  'Width': 294.75,
  'Height': 332.25,
  'TP Similarity': 99.98822784423828,
  'Others Similarity Average': 1.2455848511308432,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanLozano',
  'Size': 20726,
  'Width': 147.375,
  'Height': 166.125,
  'TP Similarity': 99.98695373535156,
  'Others Similarity Average': 1.2514030300080776,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 2877854,
  'Width': 3088.0,
  'Height': 2316.0,
  'TP Similarity': 99.89158630371094,
  'Others Similarity Average': 1.2641500122845173,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 971144,
  'Width': 1544.0,
  'Height': 1158.0,
  'TP Similarity': 99.9172134399414,
  'Others Similarity Average': 1.451296916231513,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 244020,
  'Width': 772.0,
  'Height': 579.0,
  'TP Similarity': 99.89769744873047,
  'Others Similarity Average': 1.2376359002664685,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 65755,
  'Width': 386.0,
  'Height': 289.5,
  'TP Similarity': 99.8897933959961,
  'Others Similarity Average': 1.1962723135948181,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 18809,
  'Width': 193.0,
  'Height': 144.75,
  'TP Similarity': 99.89588928222656,
  'Others Similarity Average': 1.7561635645106435,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0}]
    ,
[{'Name': 'AllysonSalom',
  'Size': 1310936,
  'Width': 2576.0,
  'Height': 1932.0,
  'TP Similarity': 99.5690689086914,
  'Others Similarity Average': 1.6399994045495987,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Size': 343350,
  'Width': 1288.0,
  'Height': 966.0,
  'TP Similarity': 99.73533630371094,
  'Others Similarity Average': 1.2774476632475853,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Size': 82816,
  'Width': 644.0,
  'Height': 483.0,
  'TP Similarity': 99.53018188476562,
  'Others Similarity Average': 1.0884984396398067,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Size': 20953,
  'Width': 322.0,
  'Height': 241.5,
  'TP Similarity': 99.52098083496094,
  'Others Similarity Average': 1.177279880270362,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Size': 7051,
  'Width': 161.0,
  'Height': 120.75,
  'TP Similarity': 99.72750091552734,
  'Others Similarity Average': 1.193180002272129,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Size': 1681946,
  'Width': 2944.0,
  'Height': 2208.0,
  'TP Similarity': 98.08149719238281,
  'Others Similarity Average': 0.6746870577335358,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Size': 434127,
  'Width': 1472.0,
  'Height': 1104.0,
  'TP Similarity': 98.15304565429688,
  'Others Similarity Average': 0.692503510043025,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Size': 104017,
  'Width': 736.0,
  'Height': 552.0,
  'TP Similarity': 97.63814544677734,
  'Others Similarity Average': 0.6912622097879648,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Size': 26896,
  'Width': 368.0,
  'Height': 276.0,
  'TP Similarity': 98.47135925292969,
  'Others Similarity Average': 0.7379992641508579,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Size': 8386,
  'Width': 184.0,
  'Height': 138.0,
  'TP Similarity': 98.2528076171875,
  'Others Similarity Average': 0.702400928363204,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 2782796,
  'Width': 1815.0,
  'Height': 2420.0,
  'TP Similarity': 99.98949432373047,
  'Others Similarity Average': 1.792025189846754,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 741937,
  'Width': 907.5,
  'Height': 1210.0,
  'TP Similarity': 99.98674011230469,
  'Others Similarity Average': 1.7182643171399832,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 191965,
  'Width': 453.75,
  'Height': 605.0,
  'TP Similarity': 99.98863983154297,
  'Others Similarity Average': 1.52582036703825,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 54231,
  'Width': 226.875,
  'Height': 302.5,
  'TP Similarity': 99.99266815185547,
  'Others Similarity Average': 1.9937574826180935,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 16898,
  'Width': 113.4375,
  'Height': 151.25,
  'TP Similarity': 99.99311828613281,
  'Others Similarity Average': 2.072732660919428,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Size': 7952,
  'Width': 120.0,
  'Height': 120.0,
  'TP Similarity': 99.77826690673828,
  'Others Similarity Average': 0.9854428311809897,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 1184660,
  'Width': 2576.0,
  'Height': 1932.0,
  'TP Similarity': 99.84022521972656,
  'Others Similarity Average': 4.111621972173452,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 324270,
  'Width': 1288.0,
  'Height': 966.0,
  'TP Similarity': 99.87461853027344,
  'Others Similarity Average': 3.8655693605542183,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 84175,
  'Width': 644.0,
  'Height': 483.0,
  'TP Similarity': 99.84859466552734,
  'Others Similarity Average': 3.712099274620414,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 23069,
  'Width': 322.0,
  'Height': 241.5,
  'TP Similarity': 99.89318084716797,
  'Others Similarity Average': 3.617652080953121,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 7265,
  'Width': 161.0,
  'Height': 120.75,
  'TP Similarity': 99.97649383544922,
  'Others Similarity Average': 3.3509560488164425,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Size': 2498409,
  'Width': 3392.0,
  'Height': 2544.0,
  'TP Similarity': 95.19093322753906,
  'Others Similarity Average': 1.5980888232588768,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Size': 524959,
  'Width': 1696.0,
  'Height': 1272.0,
  'TP Similarity': 92.99176025390625,
  'Others Similarity Average': 1.4005082845687866,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Size': 120010,
  'Width': 848.0,
  'Height': 636.0,
  'TP Similarity': 93.7102279663086,
  'Others Similarity Average': 1.4769216999411583,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Size': 32013,
  'Width': 424.0,
  'Height': 318.0,
  'TP Similarity': 94.37623596191406,
  'Others Similarity Average': 1.596305314451456,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Size': 10355,
  'Width': 212.0,
  'Height': 159.0,
  'TP Similarity': 96.19229888916016,
  'Others Similarity Average': 1.8773129545152187,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 2877854,
  'Width': 3088.0,
  'Height': 2316.0,
  'TP Similarity': 99.95989990234375,
  'Others Similarity Average': 0.8743001129478216,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 971144,
  'Width': 1544.0,
  'Height': 1158.0,
  'TP Similarity': 99.96353912353516,
  'Others Similarity Average': 0.9985643681138754,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 244020,
  'Width': 772.0,
  'Height': 579.0,
  'TP Similarity': 99.96077728271484,
  'Others Similarity Average': 0.9766655806452036,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 65755,
  'Width': 386.0,
  'Height': 289.5,
  'TP Similarity': 99.94915771484375,
  'Others Similarity Average': 0.9234097050502896,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 18809,
  'Width': 193.0,
  'Height': 144.75,
  'TP Similarity': 99.93002319335938,
  'Others Similarity Average': 1.4270836226642132,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0}]
    ,
[{'Name': 'AllysonSalom',
  'Size': 1310936,
  'Width': 2576.0,
  'Height': 1932.0,
  'TP Similarity': 99.36978149414062,
  'Others Similarity Average': 3.598295509815216,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Size': 343350,
  'Width': 1288.0,
  'Height': 966.0,
  'TP Similarity': 99.6009521484375,
  'Others Similarity Average': 2.5193273769691586,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Size': 82816,
  'Width': 644.0,
  'Height': 483.0,
  'TP Similarity': 99.19178009033203,
  'Others Similarity Average': 2.3043125988915563,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Size': 20953,
  'Width': 322.0,
  'Height': 241.5,
  'TP Similarity': 99.41763305664062,
  'Others Similarity Average': 2.3808038104325533,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Size': 7051,
  'Width': 161.0,
  'Height': 120.75,
  'TP Similarity': 99.49372100830078,
  'Others Similarity Average': 2.5135992113500834,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Size': 1681946,
  'Width': 2944.0,
  'Height': 2208.0,
  'TP Similarity': 98.37953186035156,
  'Others Similarity Average': 0.46988178975880146,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Size': 434127,
  'Width': 1472.0,
  'Height': 1104.0,
  'TP Similarity': 98.35993957519531,
  'Others Similarity Average': 0.48527762293815613,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Size': 104017,
  'Width': 736.0,
  'Height': 552.0,
  'TP Similarity': 97.88182830810547,
  'Others Similarity Average': 0.4479565005749464,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Size': 26896,
  'Width': 368.0,
  'Height': 276.0,
  'TP Similarity': 98.75167083740234,
  'Others Similarity Average': 0.49741969630122185,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Size': 8386,
  'Width': 184.0,
  'Height': 138.0,
  'TP Similarity': 98.62452697753906,
  'Others Similarity Average': 0.44729529321193695,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 2782796,
  'Width': 1815.0,
  'Height': 2420.0,
  'TP Similarity': 99.98242950439453,
  'Others Similarity Average': 1.653609935194254,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 741937,
  'Width': 907.5,
  'Height': 1210.0,
  'TP Similarity': 99.97850799560547,
  'Others Similarity Average': 1.6111019253730774,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 191965,
  'Width': 453.75,
  'Height': 605.0,
  'TP Similarity': 99.98760223388672,
  'Others Similarity Average': 1.4619624819606543,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 54231,
  'Width': 226.875,
  'Height': 302.5,
  'TP Similarity': 99.98538970947266,
  'Others Similarity Average': 1.8435204289853573,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 16898,
  'Width': 113.4375,
  'Height': 151.25,
  'TP Similarity': 99.98257446289062,
  'Others Similarity Average': 1.931056696921587,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Size': 7952,
  'Width': 120.0,
  'Height': 120.0,
  'TP Similarity': 98.99430084228516,
  'Others Similarity Average': 0.888177671469748,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 1184660,
  'Width': 2576.0,
  'Height': 1932.0,
  'TP Similarity': 99.41552734375,
  'Others Similarity Average': 3.178672529757023,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 324270,
  'Width': 1288.0,
  'Height': 966.0,
  'TP Similarity': 99.515380859375,
  'Others Similarity Average': 3.028181169182062,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 84175,
  'Width': 644.0,
  'Height': 483.0,
  'TP Similarity': 99.39329528808594,
  'Others Similarity Average': 2.9494052287191153,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 23069,
  'Width': 322.0,
  'Height': 241.5,
  'TP Similarity': 99.57501983642578,
  'Others Similarity Average': 2.862315049394965,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 7265,
  'Width': 161.0,
  'Height': 120.75,
  'TP Similarity': 99.88545227050781,
  'Others Similarity Average': 3.153392294421792,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Size': 2498409,
  'Width': 3392.0,
  'Height': 2544.0,
  'TP Similarity': 96.71986389160156,
  'Others Similarity Average': 1.792000649496913,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Size': 524959,
  'Width': 1696.0,
  'Height': 1272.0,
  'TP Similarity': 95.44287872314453,
  'Others Similarity Average': 1.5647470261901617,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Size': 120010,
  'Width': 848.0,
  'Height': 636.0,
  'TP Similarity': 96.0565414428711,
  'Others Similarity Average': 1.6111675705760717,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Size': 32013,
  'Width': 424.0,
  'Height': 318.0,
  'TP Similarity': 96.80117797851562,
  'Others Similarity Average': 1.635359052568674,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Size': 10355,
  'Width': 212.0,
  'Height': 159.0,
  'TP Similarity': 97.53350067138672,
  'Others Similarity Average': 1.7933253794908524,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 2877854,
  'Width': 3088.0,
  'Height': 2316.0,
  'TP Similarity': 99.9587631225586,
  'Others Similarity Average': 0.7074432701338083,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 971144,
  'Width': 1544.0,
  'Height': 1158.0,
  'TP Similarity': 99.96744537353516,
  'Others Similarity Average': 0.7994621025864035,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 244020,
  'Width': 772.0,
  'Height': 579.0,
  'TP Similarity': 99.95999145507812,
  'Others Similarity Average': 0.7274830301757902,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 65755,
  'Width': 386.0,
  'Height': 289.5,
  'TP Similarity': 99.95407104492188,
  'Others Similarity Average': 0.6762099196203053,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 18809,
  'Width': 193.0,
  'Height': 144.75,
  'TP Similarity': 99.9568862915039,
  'Others Similarity Average': 0.9881189530715346,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0}]
    ,
[{'Name': 'AliRada',
  'Size': 2132873,
  'Width': 1448.0,
  'Height': 2573.0,
  'TP Similarity': 37.06013488769531,
  'Others Similarity Average': 2.102227704051663,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Size': 637436,
  'Width': 724.0,
  'Height': 1286.5,
  'TP Similarity': 39.31591033935547,
  'Others Similarity Average': 2.1523709551376453,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Size': 170814,
  'Width': 362.0,
  'Height': 643.25,
  'TP Similarity': 54.744083404541016,
  'Others Similarity Average': 2.787888348540839,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Size': 47021,
  'Width': 181.0,
  'Height': 321.625,
  'TP Similarity': 54.34162139892578,
  'Others Similarity Average': 2.7980010044925354,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Size': 14231,
  'Width': 90.5,
  'Height': 160.8125,
  'TP Similarity': 49.167274475097656,
  'Others Similarity Average': 2.6944815941593228,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'AndreaQuintero',
  'Size': 2976953,
  'Width': 1815.0,
  'Height': 2420.0,
  'TP Similarity': 88.99156188964844,
  'Others Similarity Average': 0.7444834163522019,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndreaQuintero',
  'Size': 768913,
  'Width': 907.5,
  'Height': 1210.0,
  'TP Similarity': 89.86370086669922,
  'Others Similarity Average': 0.8283438857863931,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndreaQuintero',
  'Size': 205485,
  'Width': 453.75,
  'Height': 605.0,
  'TP Similarity': 91.15558624267578,
  'Others Similarity Average': 0.7088894258965465,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndreaQuintero',
  'Size': 58234,
  'Width': 226.875,
  'Height': 302.5,
  'TP Similarity': 97.26351165771484,
  'Others Similarity Average': 0.7698947651421323,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndreaQuintero',
  'Size': 17652,
  'Width': 113.4375,
  'Height': 151.25,
  'TP Similarity': 98.4051513671875,
  'Others Similarity Average': 0.793364343836027,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Size': 2011376,
  'Width': 3088.0,
  'Height': 2316.0,
  'TP Similarity': 99.00015258789062,
  'Others Similarity Average': 3.1725732710431602,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Size': 553610,
  'Width': 1544.0,
  'Height': 1158.0,
  'TP Similarity': 99.02788543701172,
  'Others Similarity Average': 3.3528166452751442,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Size': 140949,
  'Width': 772.0,
  'Height': 579.0,
  'TP Similarity': 99.24468231201172,
  'Others Similarity Average': 3.1397248865926968,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Size': 41053,
  'Width': 386.0,
  'Height': 289.5,
  'TP Similarity': 99.34745788574219,
  'Others Similarity Average': 2.968832210144576,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Size': 13404,
  'Width': 193.0,
  'Height': 144.75,
  'TP Similarity': 99.54102325439453,
  'Others Similarity Average': 2.7326948949519325,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 2479887,
  'Width': 4032.0,
  'Height': 3024.0,
  'TP Similarity': 98.34449005126953,
  'Others Similarity Average': 1.7929686281610937,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 847561,
  'Width': 2016.0,
  'Height': 1512.0,
  'TP Similarity': 98.45365905761719,
  'Others Similarity Average': 1.7055673402021914,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 243913,
  'Width': 1008.0,
  'Height': 756.0,
  'TP Similarity': 98.90862274169922,
  'Others Similarity Average': 1.7593612776083105,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 65817,
  'Width': 504.0,
  'Height': 378.0,
  'TP Similarity': 98.76470947265625,
  'Others Similarity Average': 1.5432880792547674,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 19392,
  'Width': 252.0,
  'Height': 189.0,
  'TP Similarity': 98.57579803466797,
  'Others Similarity Average': 1.6264603383400862,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 6438,
  'Width': 126.0,
  'Height': 94.5,
  'TP Similarity': 98.24251556396484,
  'Others Similarity Average': 1.9442300551077898,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Size': 2792151,
  'Width': 3120.0,
  'Height': 4208.0,
  'TP Similarity': 96.55563354492188,
  'Others Similarity Average': 1.302703916149981,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Size': 536684,
  'Width': 1560.0,
  'Height': 2104.0,
  'TP Similarity': 96.35408782958984,
  'Others Similarity Average': 1.3875885351615793,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Size': 143366,
  'Width': 780.0,
  'Height': 1052.0,
  'TP Similarity': 95.04216003417969,
  'Others Similarity Average': 1.2554203946800793,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Size': 43131,
  'Width': 390.0,
  'Height': 526.0,
  'TP Similarity': 94.46643829345703,
  'Others Similarity Average': 1.2359398862018305,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Size': 13853,
  'Width': 195.0,
  'Height': 263.0,
  'TP Similarity': 94.71343994140625,
  'Others Similarity Average': 1.277218709973728,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Size': 5200,
  'Width': 97.5,
  'Height': 131.5,
  'TP Similarity': 96.94876098632812,
  'Others Similarity Average': 1.4079913571476936,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Size': 7952,
  'Width': 120.0,
  'Height': 120.0,
  'TP Similarity': 98.81391143798828,
  'Others Similarity Average': 1.428397532333346,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Size': 4160186,
  'Width': 5184.0,
  'Height': 3880.0,
  'TP Similarity': 98.8939437866211,
  'Others Similarity Average': 2.3239248132004455,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Size': 1216079,
  'Width': 2592.0,
  'Height': 1940.0,
  'TP Similarity': 98.79019165039062,
  'Others Similarity Average': 2.2816638753694645,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Size': 314274,
  'Width': 1296.0,
  'Height': 970.0,
  'TP Similarity': 98.65238952636719,
  'Others Similarity Average': 2.2991230338811874,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Size': 75787,
  'Width': 648.0,
  'Height': 485.0,
  'TP Similarity': 98.85625457763672,
  'Others Similarity Average': 2.2094896760057,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Size': 21346,
  'Width': 324.0,
  'Height': 242.5,
  'TP Similarity': 98.88610076904297,
  'Others Similarity Average': 2.574554434593986,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Size': 7465,
  'Width': 162.0,
  'Height': 121.25,
  'TP Similarity': 98.7540283203125,
  'Others Similarity Average': 3.045873184414471,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielaBecerra',
  'Size': 1971600,
  'Width': 3088.0,
  'Height': 2316.0,
  'TP Similarity': 44.632171630859375,
  'Others Similarity Average': 1.8086722484406303,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Size': 550810,
  'Width': 1544.0,
  'Height': 1158.0,
  'TP Similarity': 50.89549255371094,
  'Others Similarity Average': 1.476154530311332,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Size': 143644,
  'Width': 772.0,
  'Height': 579.0,
  'TP Similarity': 47.5759391784668,
  'Others Similarity Average': 1.4745643498266445,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Size': 42706,
  'Width': 386.0,
  'Height': 289.5,
  'TP Similarity': 54.30563735961914,
  'Others Similarity Average': 1.5808330686653362,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Size': 13437,
  'Width': 193.0,
  'Height': 144.75,
  'TP Similarity': 66.2890396118164,
  'Others Similarity Average': 1.732126442825093,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'JuanHernandez',
  'Size': 2247431,
  'Width': 1816.0,
  'Height': 2420.0,
  'TP Similarity': 97.55355834960938,
  'Others Similarity Average': 1.2334066547234268,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanHernandez',
  'Size': 603616,
  'Width': 908.0,
  'Height': 1210.0,
  'TP Similarity': 97.27867889404297,
  'Others Similarity Average': 1.1509141135303413,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanHernandez',
  'Size': 168566,
  'Width': 454.0,
  'Height': 605.0,
  'TP Similarity': 97.44697570800781,
  'Others Similarity Average': 1.2220837596584768,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanHernandez',
  'Size': 50295,
  'Width': 227.0,
  'Height': 302.5,
  'TP Similarity': 98.38319396972656,
  'Others Similarity Average': 1.6659615651211317,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanHernandez',
  'Size': 15403,
  'Width': 113.5,
  'Height': 151.25,
  'TP Similarity': 98.38579559326172,
  'Others Similarity Average': 1.9243760845240425,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Size': 1896132,
  'Width': 3000.0,
  'Height': 3000.0,
  'TP Similarity': 98.77051544189453,
  'Others Similarity Average': 1.8607854282154757,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Size': 522081,
  'Width': 1500.0,
  'Height': 1500.0,
  'TP Similarity': 98.63788604736328,
  'Others Similarity Average': 1.7904345217873068,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Size': 172424,
  'Width': 750.0,
  'Height': 750.0,
  'TP Similarity': 98.91877746582031,
  'Others Similarity Average': 1.906785076155382,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Size': 56926,
  'Width': 375.0,
  'Height': 375.0,
  'TP Similarity': 98.46672058105469,
  'Others Similarity Average': 2.165530206526027,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Size': 18143,
  'Width': 187.5,
  'Height': 187.5,
  'TP Similarity': 96.02798461914062,
  'Others Similarity Average': 2.4111796968123493,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Size': 5733,
  'Width': 93.75,
  'Height': 93.75,
  'TP Similarity': 88.488525390625,
  'Others Similarity Average': 3.1845390012159065,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Size': 2262354,
  'Width': 3088.0,
  'Height': 2320.0,
  'TP Similarity': 99.99517822265625,
  'Others Similarity Average': 2.03015497239197,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Size': 852943,
  'Width': 1544.0,
  'Height': 1160.0,
  'TP Similarity': 99.99465942382812,
  'Others Similarity Average': 1.9517960741239435,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Size': 246461,
  'Width': 772.0,
  'Height': 580.0,
  'TP Similarity': 99.99610900878906,
  'Others Similarity Average': 1.965901674593196,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Size': 69068,
  'Width': 386.0,
  'Height': 290.0,
  'TP Similarity': 99.9961166381836,
  'Others Similarity Average': 2.2224409326034436,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Size': 20797,
  'Width': 193.0,
  'Height': 145.0,
  'TP Similarity': 99.9914321899414,
  'Others Similarity Average': 2.6002318197313476,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'LucianaDeLaRosa',
  'Size': 2150747,
  'Width': 3088.0,
  'Height': 2316.0,
  'TP Similarity': 62.41984939575195,
  'Others Similarity Average': 0.748034856556093,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Size': 598968,
  'Width': 1544.0,
  'Height': 1158.0,
  'TP Similarity': 62.44054412841797,
  'Others Similarity Average': 0.639917861451121,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Size': 151609,
  'Width': 772.0,
  'Height': 579.0,
  'TP Similarity': 57.817466735839844,
  'Others Similarity Average': 0.8074475951931056,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Size': 42603,
  'Width': 386.0,
  'Height': 289.5,
  'TP Similarity': 63.11839294433594,
  'Others Similarity Average': 0.8360661899342257,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Size': 13023,
  'Width': 193.0,
  'Height': 144.75,
  'TP Similarity': 81.5477523803711,
  'Others Similarity Average': 1.0573603791349075,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Size': 2812489,
  'Width': 3264.0,
  'Height': 2448.0,
  'TP Similarity': 94.92184448242188,
  'Others Similarity Average': 1.6695729292490904,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Size': 563386,
  'Width': 1632.0,
  'Height': 1224.0,
  'TP Similarity': 94.31363677978516,
  'Others Similarity Average': 1.5591947594109703,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Size': 128300,
  'Width': 816.0,
  'Height': 612.0,
  'TP Similarity': 92.8443374633789,
  'Others Similarity Average': 1.5608657738741707,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Size': 36729,
  'Width': 408.0,
  'Height': 306.0,
  'TP Similarity': 94.3400650024414,
  'Others Similarity Average': 1.6216411643168505,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Size': 11668,
  'Width': 204.0,
  'Height': 153.0,
  'TP Similarity': 95.26931762695312,
  'Others Similarity Average': 1.8827218281872131,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Size': 344995,
  'Width': 1500.0,
  'Height': 1500.0,
  'TP Similarity': 99.9660415649414,
  'Others Similarity Average': 0.6899145232403979,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Size': 170978,
  'Width': 750.0,
  'Height': 750.0,
  'TP Similarity': 99.94721984863281,
  'Others Similarity Average': 0.932023377102964,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Size': 58318,
  'Width': 375.0,
  'Height': 375.0,
  'TP Similarity': 99.91081237792969,
  'Others Similarity Average': 1.5751591500113993,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Size': 19355,
  'Width': 187.5,
  'Height': 187.5,
  'TP Similarity': 99.84212493896484,
  'Others Similarity Average': 2.2177240042125477,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Size': 6332,
  'Width': 93.75,
  'Height': 93.75,
  'TP Similarity': 95.52224731445312,
  'Others Similarity Average': 2.4531385276247475,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Size': 365365,
  'Width': 720.0,
  'Height': 1280.0,
  'TP Similarity': 99.8544921875,
  'Others Similarity Average': 1.4455808085553787,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Size': 162022,
  'Width': 360.0,
  'Height': 640.0,
  'TP Similarity': 99.88819885253906,
  'Others Similarity Average': 1.4169458063209759,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Size': 46603,
  'Width': 180.0,
  'Height': 320.0,
  'TP Similarity': 99.87699890136719,
  'Others Similarity Average': 1.57132585171391,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Size': 13804,
  'Width': 90.0,
  'Height': 160.0,
  'TP Similarity': 99.8595199584961,
  'Others Similarity Average': 1.526002650751787,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Size': 1904711,
  'Width': 1448.0,
  'Height': 2573.0,
  'TP Similarity': 98.64669799804688,
  'Others Similarity Average': 3.4411073626183413,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Size': 502221,
  'Width': 724.0,
  'Height': 1286.5,
  'TP Similarity': 99.02549743652344,
  'Others Similarity Average': 3.79567141962402,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Size': 133134,
  'Width': 362.0,
  'Height': 643.25,
  'TP Similarity': 99.41433715820312,
  'Others Similarity Average': 4.466567715739503,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Size': 38031,
  'Width': 181.0,
  'Height': 321.625,
  'TP Similarity': 99.44883728027344,
  'Others Similarity Average': 4.594810701468411,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Size': 11740,
  'Width': 90.5,
  'Height': 160.8125,
  'TP Similarity': 99.55448150634766,
  'Others Similarity Average': 5.452496769673684,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0}]
    ,
[{'Name': 'AliRada',
  'Size': 2132873,
  'Width': 1448.0,
  'Height': 2573.0,
  'TP Similarity': 61.33567810058594,
  'Others Similarity Average': 2.367114483896229,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Size': 637436,
  'Width': 724.0,
  'Height': 1286.5,
  'TP Similarity': 62.49993896484375,
  'Others Similarity Average': 2.414755016565323,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Size': 170814,
  'Width': 362.0,
  'Height': 643.25,
  'TP Similarity': 68.85172271728516,
  'Others Similarity Average': 2.8447299930784435,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Size': 47021,
  'Width': 181.0,
  'Height': 321.625,
  'TP Similarity': 63.204689025878906,
  'Others Similarity Average': 2.8023876949316926,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Size': 14231,
  'Width': 90.5,
  'Height': 160.8125,
  'TP Similarity': 61.27073669433594,
  'Others Similarity Average': 2.588735514630874,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'AndreaQuintero',
  'Size': 2976953,
  'Width': 1815.0,
  'Height': 2420.0,
  'TP Similarity': 89.40705871582031,
  'Others Similarity Average': 0.6296498691663146,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndreaQuintero',
  'Size': 768913,
  'Width': 907.5,
  'Height': 1210.0,
  'TP Similarity': 90.66566467285156,
  'Others Similarity Average': 0.7414012102203237,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndreaQuintero',
  'Size': 205485,
  'Width': 453.75,
  'Height': 605.0,
  'TP Similarity': 89.35252380371094,
  'Others Similarity Average': 0.5389337563473318,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndreaQuintero',
  'Size': 58234,
  'Width': 226.875,
  'Height': 302.5,
  'TP Similarity': 95.50077056884766,
  'Others Similarity Average': 0.6466536184565889,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndreaQuintero',
  'Size': 17652,
  'Width': 113.4375,
  'Height': 151.25,
  'TP Similarity': 97.2132339477539,
  'Others Similarity Average': 0.8573765682263507,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Size': 2011376,
  'Width': 3088.0,
  'Height': 2316.0,
  'TP Similarity': 99.1168212890625,
  'Others Similarity Average': 1.770919004248248,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Size': 553610,
  'Width': 1544.0,
  'Height': 1158.0,
  'TP Similarity': 99.22639465332031,
  'Others Similarity Average': 1.740290156669087,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Size': 140949,
  'Width': 772.0,
  'Height': 579.0,
  'TP Similarity': 99.38243865966797,
  'Others Similarity Average': 1.6096617380777996,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Size': 41053,
  'Width': 386.0,
  'Height': 289.5,
  'TP Similarity': 99.40299224853516,
  'Others Similarity Average': 1.6167419387234583,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Size': 13404,
  'Width': 193.0,
  'Height': 144.75,
  'TP Similarity': 99.66764831542969,
  'Others Similarity Average': 1.6685461493002043,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 2479887,
  'Width': 4032.0,
  'Height': 3024.0,
  'TP Similarity': 99.59708404541016,
  'Others Similarity Average': 2.8617605728407702,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 847561,
  'Width': 2016.0,
  'Height': 1512.0,
  'TP Similarity': 99.48284912109375,
  'Others Similarity Average': 2.843349119855298,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 243913,
  'Width': 1008.0,
  'Height': 756.0,
  'TP Similarity': 99.63429260253906,
  'Others Similarity Average': 2.6986657877763114,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 65817,
  'Width': 504.0,
  'Height': 378.0,
  'TP Similarity': 99.42101287841797,
  'Others Similarity Average': 2.236489296787315,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 19392,
  'Width': 252.0,
  'Height': 189.0,
  'TP Similarity': 99.5682601928711,
  'Others Similarity Average': 2.4540358094705477,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 6438,
  'Width': 126.0,
  'Height': 94.5,
  'TP Similarity': 99.60309600830078,
  'Others Similarity Average': 2.6087447148230343,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Size': 2792151,
  'Width': 3120.0,
  'Height': 4208.0,
  'TP Similarity': 92.07698822021484,
  'Others Similarity Average': 0.7610486381583743,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Size': 536684,
  'Width': 1560.0,
  'Height': 2104.0,
  'TP Similarity': 90.95264434814453,
  'Others Similarity Average': 0.729096882045269,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Size': 143366,
  'Width': 780.0,
  'Height': 1052.0,
  'TP Similarity': 87.9625473022461,
  'Others Similarity Average': 0.6245342087414529,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Size': 43131,
  'Width': 390.0,
  'Height': 526.0,
  'TP Similarity': 87.1390609741211,
  'Others Similarity Average': 0.6481020541654693,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Size': 13853,
  'Width': 195.0,
  'Height': 263.0,
  'TP Similarity': 89.21670532226562,
  'Others Similarity Average': 0.6475533329778247,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Size': 5200,
  'Width': 97.5,
  'Height': 131.5,
  'TP Similarity': 91.72611236572266,
  'Others Similarity Average': 0.826569194595019,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Size': 7952,
  'Width': 120.0,
  'Height': 120.0,
  'TP Similarity': 99.8436279296875,
  'Others Similarity Average': 2.1857605307466454,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Size': 4160186,
  'Width': 5184.0,
  'Height': 3880.0,
  'TP Similarity': 99.93456268310547,
  'Others Similarity Average': 3.1509495311313205,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Size': 1216079,
  'Width': 2592.0,
  'Height': 1940.0,
  'TP Similarity': 99.94371795654297,
  'Others Similarity Average': 3.1539018038246365,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Size': 314274,
  'Width': 1296.0,
  'Height': 970.0,
  'TP Similarity': 99.9256591796875,
  'Others Similarity Average': 3.071884679297606,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Size': 75787,
  'Width': 648.0,
  'Height': 485.0,
  'TP Similarity': 99.907958984375,
  'Others Similarity Average': 3.171787101775408,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Size': 21346,
  'Width': 324.0,
  'Height': 242.5,
  'TP Similarity': 99.87446594238281,
  'Others Similarity Average': 3.816358936329683,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Size': 7465,
  'Width': 162.0,
  'Height': 121.25,
  'TP Similarity': 99.8131332397461,
  'Others Similarity Average': 4.330604129367405,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielaBecerra',
  'Size': 1971600,
  'Width': 3088.0,
  'Height': 2316.0,
  'TP Similarity': 79.52253723144531,
  'Others Similarity Average': 1.0394336962037616,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Size': 550810,
  'Width': 1544.0,
  'Height': 1158.0,
  'TP Similarity': 72.10108947753906,
  'Others Similarity Average': 0.9317781565090021,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Size': 143644,
  'Width': 772.0,
  'Height': 579.0,
  'TP Similarity': 73.0598373413086,
  'Others Similarity Average': 0.9648766467968622,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Size': 42706,
  'Width': 386.0,
  'Height': 289.5,
  'TP Similarity': 74.98086547851562,
  'Others Similarity Average': 1.0938008237216208,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Size': 13437,
  'Width': 193.0,
  'Height': 144.75,
  'TP Similarity': 79.50297546386719,
  'Others Similarity Average': 1.1764926811059315,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'JuanHernandez',
  'Size': 2247431,
  'Width': 1816.0,
  'Height': 2420.0,
  'TP Similarity': 98.06483459472656,
  'Others Similarity Average': 0.9495677285724216,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanHernandez',
  'Size': 603616,
  'Width': 908.0,
  'Height': 1210.0,
  'TP Similarity': 97.23268127441406,
  'Others Similarity Average': 1.0154192447662354,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanHernandez',
  'Size': 168566,
  'Width': 454.0,
  'Height': 605.0,
  'TP Similarity': 97.37200164794922,
  'Others Similarity Average': 0.9244137451880508,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanHernandez',
  'Size': 50295,
  'Width': 227.0,
  'Height': 302.5,
  'TP Similarity': 98.43804168701172,
  'Others Similarity Average': 1.1650454166034858,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanHernandez',
  'Size': 15403,
  'Width': 113.5,
  'Height': 151.25,
  'TP Similarity': 98.1948471069336,
  'Others Similarity Average': 1.4021399161881871,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Size': 1896132,
  'Width': 3000.0,
  'Height': 3000.0,
  'TP Similarity': 99.98037719726562,
  'Others Similarity Average': 2.3395167373948627,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Size': 522081,
  'Width': 1500.0,
  'Height': 1500.0,
  'TP Similarity': 99.98145294189453,
  'Others Similarity Average': 2.224568718009525,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Size': 172424,
  'Width': 750.0,
  'Height': 750.0,
  'TP Similarity': 99.97811889648438,
  'Others Similarity Average': 2.3831990443997912,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Size': 56926,
  'Width': 375.0,
  'Height': 375.0,
  'TP Similarity': 99.96946716308594,
  'Others Similarity Average': 2.9004442137148647,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Size': 18143,
  'Width': 187.5,
  'Height': 187.5,
  'TP Similarity': 99.90835571289062,
  'Others Similarity Average': 3.1847219690680504,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Size': 5733,
  'Width': 93.75,
  'Height': 93.75,
  'TP Similarity': 99.2308349609375,
  'Others Similarity Average': 2.8598232972953053,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Size': 2262354,
  'Width': 3088.0,
  'Height': 2320.0,
  'TP Similarity': 99.9950942993164,
  'Others Similarity Average': 2.884959561129411,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Size': 852943,
  'Width': 1544.0,
  'Height': 1160.0,
  'TP Similarity': 99.99486541748047,
  'Others Similarity Average': 2.5441685956385403,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Size': 246461,
  'Width': 772.0,
  'Height': 580.0,
  'TP Similarity': 99.99630737304688,
  'Others Similarity Average': 2.773871161871486,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Size': 69068,
  'Width': 386.0,
  'Height': 290.0,
  'TP Similarity': 99.99620056152344,
  'Others Similarity Average': 2.7334693322579064,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Size': 20797,
  'Width': 193.0,
  'Height': 145.0,
  'TP Similarity': 99.99178314208984,
  'Others Similarity Average': 3.0263732837306128,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'LucianaDeLaRosa',
  'Size': 2150747,
  'Width': 3088.0,
  'Height': 2316.0,
  'TP Similarity': 34.896121978759766,
  'Others Similarity Average': 1.9296186798148685,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Size': 598968,
  'Width': 1544.0,
  'Height': 1158.0,
  'TP Similarity': 37.239036560058594,
  'Others Similarity Average': 1.491688559245732,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Size': 151609,
  'Width': 772.0,
  'Height': 579.0,
  'TP Similarity': 29.636388778686523,
  'Others Similarity Average': 1.9830947344501813,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Size': 42603,
  'Width': 386.0,
  'Height': 289.5,
  'TP Similarity': 33.80421829223633,
  'Others Similarity Average': 2.073008799304565,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Size': 13023,
  'Width': 193.0,
  'Height': 144.75,
  'TP Similarity': 58.9327392578125,
  'Others Similarity Average': 2.7002707686689167,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'MariaFerrer',
  'Size': 2812489,
  'Width': 3264.0,
  'Height': 2448.0,
  'TP Similarity': 98.91039276123047,
  'Others Similarity Average': 0.8734370085100333,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Size': 563386,
  'Width': 1632.0,
  'Height': 1224.0,
  'TP Similarity': 99.03958129882812,
  'Others Similarity Average': 0.8410102424936162,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Size': 128300,
  'Width': 816.0,
  'Height': 612.0,
  'TP Similarity': 98.82952880859375,
  'Others Similarity Average': 0.8608032518790828,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Size': 36729,
  'Width': 408.0,
  'Height': 306.0,
  'TP Similarity': 98.94998168945312,
  'Others Similarity Average': 0.8673346717324522,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Size': 11668,
  'Width': 204.0,
  'Height': 153.0,
  'TP Similarity': 99.2539291381836,
  'Others Similarity Average': 1.0309709534049034,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Size': 344995,
  'Width': 1500.0,
  'Height': 1500.0,
  'TP Similarity': 99.99407196044922,
  'Others Similarity Average': 0.7927997306817107,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Size': 170978,
  'Width': 750.0,
  'Height': 750.0,
  'TP Similarity': 99.99053192138672,
  'Others Similarity Average': 0.8306254649327861,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Size': 58318,
  'Width': 375.0,
  'Height': 375.0,
  'TP Similarity': 99.94522094726562,
  'Others Similarity Average': 1.1795126704706087,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Size': 19355,
  'Width': 187.5,
  'Height': 187.5,
  'TP Similarity': 99.8031997680664,
  'Others Similarity Average': 1.358548881693019,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Size': 6332,
  'Width': 93.75,
  'Height': 93.75,
  'TP Similarity': 92.30693817138672,
  'Others Similarity Average': 1.4922211021184921,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Size': 365365,
  'Width': 720.0,
  'Height': 1280.0,
  'TP Similarity': 99.9174575805664,
  'Others Similarity Average': 0.9441517504553,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Size': 162022,
  'Width': 360.0,
  'Height': 640.0,
  'TP Similarity': 99.94355773925781,
  'Others Similarity Average': 0.9766216195291944,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Size': 46603,
  'Width': 180.0,
  'Height': 320.0,
  'TP Similarity': 99.95821380615234,
  'Others Similarity Average': 1.1893030276729002,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Size': 13804,
  'Width': 90.0,
  'Height': 160.0,
  'TP Similarity': 99.96471405029297,
  'Others Similarity Average': 1.302849763383468,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Size': 1904711,
  'Width': 1448.0,
  'Height': 2573.0,
  'TP Similarity': 97.7334213256836,
  'Others Similarity Average': 2.3863274347451,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Size': 502221,
  'Width': 724.0,
  'Height': 1286.5,
  'TP Similarity': 98.21468353271484,
  'Others Similarity Average': 2.634743983960814,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Size': 133134,
  'Width': 362.0,
  'Height': 643.25,
  'TP Similarity': 98.53775787353516,
  'Others Similarity Average': 3.195830113357968,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Size': 38031,
  'Width': 181.0,
  'Height': 321.625,
  'TP Similarity': 98.57135772705078,
  'Others Similarity Average': 3.2862528098954096,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Size': 11740,
  'Width': 90.5,
  'Height': 160.8125,
  'TP Similarity': 98.80339813232422,
  'Others Similarity Average': 3.8350171293649407,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0}]
    ,
[{'Name': 'AllysonSalom',
  'Size': 1310936,
  'Width': 2576.0,
  'Height': 1932.0,
  'TP Similarity': 70.47772216796875,
  'Others Similarity Average': 1.443195465952158,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'AllysonSalom',
  'Size': 343350,
  'Width': 1288.0,
  'Height': 966.0,
  'TP Similarity': 72.08467102050781,
  'Others Similarity Average': 1.1027839872986078,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'AllysonSalom',
  'Size': 82816,
  'Width': 644.0,
  'Height': 483.0,
  'TP Similarity': 72.78466796875,
  'Others Similarity Average': 1.1188347674906254,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'AllysonSalom',
  'Size': 20953,
  'Width': 322.0,
  'Height': 241.5,
  'TP Similarity': 76.94189453125,
  'Others Similarity Average': 1.2357152453623712,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'AllysonSalom',
  'Size': 7051,
  'Width': 161.0,
  'Height': 120.75,
  'TP Similarity': 71.85690307617188,
  'Others Similarity Average': 1.3876345613971353,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'AngieValencia',
  'Size': 1681946,
  'Width': 2944.0,
  'Height': 2208.0,
  'TP Similarity': 10.440814018249512,
  'Others Similarity Average': 0.928038876503706,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'AngieValencia',
  'Size': 434127,
  'Width': 1472.0,
  'Height': 1104.0,
  'TP Similarity': 8.998136520385742,
  'Others Similarity Average': 0.8715083636343479,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'AngieValencia',
  'Size': 104017,
  'Width': 736.0,
  'Height': 552.0,
  'TP Similarity': 8.674696922302246,
  'Others Similarity Average': 0.9770966172218323,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'AngieValencia',
  'Size': 26896,
  'Width': 368.0,
  'Height': 276.0,
  'TP Similarity': 8.609228134155273,
  'Others Similarity Average': 0.9152459986507893,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'AngieValencia',
  'Size': 8386,
  'Width': 184.0,
  'Height': 138.0,
  'TP Similarity': 6.593143939971924,
  'Others Similarity Average': 0.8696340247988701,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'DanielDelgado',
  'Size': 2782796,
  'Width': 1815.0,
  'Height': 2420.0,
  'TP Similarity': 99.95332336425781,
  'Others Similarity Average': 1.3282098155468702,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 741937,
  'Width': 907.5,
  'Height': 1210.0,
  'TP Similarity': 99.94843292236328,
  'Others Similarity Average': 1.230876088142395,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 191965,
  'Width': 453.75,
  'Height': 605.0,
  'TP Similarity': 99.9356689453125,
  'Others Similarity Average': 1.083416709676385,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 54231,
  'Width': 226.875,
  'Height': 302.5,
  'TP Similarity': 99.94548797607422,
  'Others Similarity Average': 1.1965131014585495,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 16898,
  'Width': 113.4375,
  'Height': 151.25,
  'TP Similarity': 99.95323181152344,
  'Others Similarity Average': 1.3068920243531466,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Size': 7952,
  'Width': 120.0,
  'Height': 120.0,
  'TP Similarity': 95.58522033691406,
  'Others Similarity Average': 0.8959888853132725,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DylanDeOro',
  'Size': 1940671,
  'Width': 2598.0,
  'Height': 3464.0,
  'TP Similarity': 49.13496017456055,
  'Others Similarity Average': 3.396141640841961,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'DylanDeOro',
  'Size': 614079,
  'Width': 1299.0,
  'Height': 1732.0,
  'TP Similarity': 47.17201614379883,
  'Others Similarity Average': 3.384124882519245,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'DylanDeOro',
  'Size': 181059,
  'Width': 649.5,
  'Height': 866.0,
  'TP Similarity': 48.507957458496094,
  'Others Similarity Average': 3.1940123066306114,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'DylanDeOro',
  'Size': 52010,
  'Width': 324.75,
  'Height': 433.0,
  'TP Similarity': 40.1319694519043,
  'Others Similarity Average': 3.137999899685383,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'DylanDeOro',
  'Size': 16628,
  'Width': 162.375,
  'Height': 216.5,
  'TP Similarity': 29.586517333984375,
  'Others Similarity Average': 3.918106973171234,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'DylanDeOro',
  'Size': 6105,
  'Width': 81.1875,
  'Height': 108.25,
  'TP Similarity': 21.204212188720703,
  'Others Similarity Average': 4.230882450938225,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'FelipeOspino',
  'Size': 1184660,
  'Width': 2576.0,
  'Height': 1932.0,
  'TP Similarity': 98.97916412353516,
  'Others Similarity Average': 1.0787325650453568,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 324270,
  'Width': 1288.0,
  'Height': 966.0,
  'TP Similarity': 99.0031509399414,
  'Others Similarity Average': 1.0590943656861782,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 84175,
  'Width': 644.0,
  'Height': 483.0,
  'TP Similarity': 98.79683685302734,
  'Others Similarity Average': 0.9412950612604618,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 23069,
  'Width': 322.0,
  'Height': 241.5,
  'TP Similarity': 99.16751098632812,
  'Others Similarity Average': 0.961676262319088,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 7265,
  'Width': 161.0,
  'Height': 120.75,
  'TP Similarity': 99.72689056396484,
  'Others Similarity Average': 1.020977009087801,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Size': 2498409,
  'Width': 3392.0,
  'Height': 2544.0,
  'TP Similarity': 47.71977233886719,
  'Others Similarity Average': 2.5660893246531487,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'HabidAbdala',
  'Size': 524959,
  'Width': 1696.0,
  'Height': 1272.0,
  'TP Similarity': 40.587806701660156,
  'Others Similarity Average': 2.352803535759449,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'HabidAbdala',
  'Size': 120010,
  'Width': 848.0,
  'Height': 636.0,
  'TP Similarity': 46.63041305541992,
  'Others Similarity Average': 2.570849224925041,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'HabidAbdala',
  'Size': 32013,
  'Width': 424.0,
  'Height': 318.0,
  'TP Similarity': 53.94526290893555,
  'Others Similarity Average': 3.31789468228817,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'HabidAbdala',
  'Size': 10355,
  'Width': 212.0,
  'Height': 159.0,
  'TP Similarity': 69.7373275756836,
  'Others Similarity Average': 3.8141427971422672,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'JuanLozano',
  'Size': 710063,
  'Width': 1179.0,
  'Height': 1329.0,
  'TP Similarity': 97.84983825683594,
  'Others Similarity Average': 1.173785200342536,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanLozano',
  'Size': 244453,
  'Width': 589.5,
  'Height': 664.5,
  'TP Similarity': 98.0440673828125,
  'Others Similarity Average': 1.228166837245226,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanLozano',
  'Size': 72867,
  'Width': 294.75,
  'Height': 332.25,
  'TP Similarity': 97.97882080078125,
  'Others Similarity Average': 1.2526857163757086,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanLozano',
  'Size': 20726,
  'Width': 147.375,
  'Height': 166.125,
  'TP Similarity': 97.99236297607422,
  'Others Similarity Average': 1.3977173622697592,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 2877854,
  'Width': 3088.0,
  'Height': 2316.0,
  'TP Similarity': 99.21290588378906,
  'Others Similarity Average': 0.6690060831606388,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 971144,
  'Width': 1544.0,
  'Height': 1158.0,
  'TP Similarity': 99.47515869140625,
  'Others Similarity Average': 0.7126277014613152,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 244020,
  'Width': 772.0,
  'Height': 579.0,
  'TP Similarity': 99.42998504638672,
  'Others Similarity Average': 0.6812547296285629,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 65755,
  'Width': 386.0,
  'Height': 289.5,
  'TP Similarity': 99.37183380126953,
  'Others Similarity Average': 0.6400647275149822,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 18809,
  'Width': 193.0,
  'Height': 144.75,
  'TP Similarity': 99.86974334716797,
  'Others Similarity Average': 1.2809365317225456,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0}]
    ,
[{'Name': 'AliRada',
  'Size': 2132873,
  'Width': 1448.0,
  'Height': 2573.0,
  'TP Similarity': 42.891544342041016,
  'Others Similarity Average': 1.120036377105862,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Size': 637436,
  'Width': 724.0,
  'Height': 1286.5,
  'TP Similarity': 43.04743576049805,
  'Others Similarity Average': 1.1185841264668852,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Size': 170814,
  'Width': 362.0,
  'Height': 643.25,
  'TP Similarity': 53.840576171875,
  'Others Similarity Average': 1.4926776285283267,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Size': 47021,
  'Width': 181.0,
  'Height': 321.625,
  'TP Similarity': 52.54452133178711,
  'Others Similarity Average': 1.9391431545373052,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Size': 14231,
  'Width': 90.5,
  'Height': 160.8125,
  'TP Similarity': 55.1875114440918,
  'Others Similarity Average': 1.9024500655941665,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AndreaQuintero',
  'Size': 2976953,
  'Width': 1815.0,
  'Height': 2420.0,
  'TP Similarity': 44.91169738769531,
  'Others Similarity Average': 0.5571200340054929,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AndreaQuintero',
  'Size': 768913,
  'Width': 907.5,
  'Height': 1210.0,
  'TP Similarity': 43.292808532714844,
  'Others Similarity Average': 0.5704279951751232,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AndreaQuintero',
  'Size': 205485,
  'Width': 453.75,
  'Height': 605.0,
  'TP Similarity': 49.65502166748047,
  'Others Similarity Average': 0.5706520390231162,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AndreaQuintero',
  'Size': 58234,
  'Width': 226.875,
  'Height': 302.5,
  'TP Similarity': 72.50198364257812,
  'Others Similarity Average': 0.9264598013833165,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AndreaQuintero',
  'Size': 17652,
  'Width': 113.4375,
  'Height': 151.25,
  'TP Similarity': 82.84883117675781,
  'Others Similarity Average': 1.2294125952757895,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Size': 2011376,
  'Width': 3088.0,
  'Height': 2316.0,
  'TP Similarity': 82.22405242919922,
  'Others Similarity Average': 1.2287443964742124,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Size': 553610,
  'Width': 1544.0,
  'Height': 1158.0,
  'TP Similarity': 83.6656723022461,
  'Others Similarity Average': 1.1904992070049047,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Size': 140949,
  'Width': 772.0,
  'Height': 579.0,
  'TP Similarity': 84.38101196289062,
  'Others Similarity Average': 1.2416807776317,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Size': 41053,
  'Width': 386.0,
  'Height': 289.5,
  'TP Similarity': 92.4544448852539,
  'Others Similarity Average': 1.22067391872406,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Size': 13404,
  'Width': 193.0,
  'Height': 144.75,
  'TP Similarity': 93.0855484008789,
  'Others Similarity Average': 1.2037355545908213,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 2479887,
  'Width': 4032.0,
  'Height': 3024.0,
  'TP Similarity': 96.02197265625,
  'Others Similarity Average': 2.552891136147082,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 847561,
  'Width': 2016.0,
  'Height': 1512.0,
  'TP Similarity': 94.37042999267578,
  'Others Similarity Average': 2.73548142099753,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 243913,
  'Width': 1008.0,
  'Height': 756.0,
  'TP Similarity': 96.24175262451172,
  'Others Similarity Average': 2.401618833653629,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 65817,
  'Width': 504.0,
  'Height': 378.0,
  'TP Similarity': 94.54741668701172,
  'Others Similarity Average': 1.8613166068680584,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 19392,
  'Width': 252.0,
  'Height': 189.0,
  'TP Similarity': 95.71537780761719,
  'Others Similarity Average': 2.437487599439919,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 6438,
  'Width': 126.0,
  'Height': 94.5,
  'TP Similarity': 96.58467102050781,
  'Others Similarity Average': 2.386577314697206,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Size': 2792151,
  'Width': 3120.0,
  'Height': 4208.0,
  'TP Similarity': 56.03097152709961,
  'Others Similarity Average': 0.7814590223133564,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'BrayanRubiano',
  'Size': 536684,
  'Width': 1560.0,
  'Height': 2104.0,
  'TP Similarity': 46.51444625854492,
  'Others Similarity Average': 0.7528514107689261,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'BrayanRubiano',
  'Size': 143366,
  'Width': 780.0,
  'Height': 1052.0,
  'TP Similarity': 39.95817184448242,
  'Others Similarity Average': 0.6379954349249601,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'BrayanRubiano',
  'Size': 43131,
  'Width': 390.0,
  'Height': 526.0,
  'TP Similarity': 45.31818771362305,
  'Others Similarity Average': 0.6557275103405118,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'BrayanRubiano',
  'Size': 13853,
  'Width': 195.0,
  'Height': 263.0,
  'TP Similarity': 46.36511993408203,
  'Others Similarity Average': 0.7174110095947981,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'BrayanRubiano',
  'Size': 5200,
  'Width': 97.5,
  'Height': 131.5,
  'TP Similarity': 55.280120849609375,
  'Others Similarity Average': 0.8844851413741708,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'DiegoGomez',
  'Size': 7952,
  'Width': 120.0,
  'Height': 120.0,
  'TP Similarity': 98.19999694824219,
  'Others Similarity Average': 0.6766462130472064,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielaBecerra',
  'Size': 1971600,
  'Width': 3088.0,
  'Height': 2316.0,
  'TP Similarity': 27.609220504760742,
  'Others Similarity Average': 0.9221827061846852,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Size': 550810,
  'Width': 1544.0,
  'Height': 1158.0,
  'TP Similarity': 27.270355224609375,
  'Others Similarity Average': 0.8659686180762947,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Size': 143644,
  'Width': 772.0,
  'Height': 579.0,
  'TP Similarity': 27.0175838470459,
  'Others Similarity Average': 0.9244034513831139,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Size': 42706,
  'Width': 386.0,
  'Height': 289.5,
  'TP Similarity': 25.46268653869629,
  'Others Similarity Average': 0.9254001034423709,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Size': 13437,
  'Width': 193.0,
  'Height': 144.75,
  'TP Similarity': 38.628013610839844,
  'Others Similarity Average': 1.0720488065853715,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Size': 2150747,
  'Width': 3088.0,
  'Height': 2316.0,
  'TP Similarity': 13.655082702636719,
  'Others Similarity Average': 0.44758571998681873,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Size': 598968,
  'Width': 1544.0,
  'Height': 1158.0,
  'TP Similarity': 16.68793487548828,
  'Others Similarity Average': 0.4166840666439384,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Size': 151609,
  'Width': 772.0,
  'Height': 579.0,
  'TP Similarity': 21.674131393432617,
  'Others Similarity Average': 0.46829255705233663,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Size': 42603,
  'Width': 386.0,
  'Height': 289.5,
  'TP Similarity': 19.988054275512695,
  'Others Similarity Average': 0.5572963407612406,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Size': 13023,
  'Width': 193.0,
  'Height': 144.75,
  'TP Similarity': 28.02359390258789,
  'Others Similarity Average': 0.7078586139250547,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'MariaFerrer',
  'Size': 2812489,
  'Width': 3264.0,
  'Height': 2448.0,
  'TP Similarity': 94.7603530883789,
  'Others Similarity Average': 1.7291973009705544,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Size': 563386,
  'Width': 1632.0,
  'Height': 1224.0,
  'TP Similarity': 93.8847885131836,
  'Others Similarity Average': 1.694209907669574,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Size': 128300,
  'Width': 816.0,
  'Height': 612.0,
  'TP Similarity': 92.40508270263672,
  'Others Similarity Average': 1.837861861102283,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Size': 36729,
  'Width': 408.0,
  'Height': 306.0,
  'TP Similarity': 94.33631896972656,
  'Others Similarity Average': 1.8179184785112739,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Size': 11668,
  'Width': 204.0,
  'Height': 153.0,
  'TP Similarity': 94.36502075195312,
  'Others Similarity Average': 2.1383713860996068,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Size': 344995,
  'Width': 1500.0,
  'Height': 1500.0,
  'TP Similarity': 99.96528625488281,
  'Others Similarity Average': 0.6975560099817812,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Size': 170978,
  'Width': 750.0,
  'Height': 750.0,
  'TP Similarity': 99.95478057861328,
  'Others Similarity Average': 1.115491934120655,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Size': 58318,
  'Width': 375.0,
  'Height': 375.0,
  'TP Similarity': 99.89872741699219,
  'Others Similarity Average': 1.8585737589746714,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Size': 19355,
  'Width': 187.5,
  'Height': 187.5,
  'TP Similarity': 99.77790832519531,
  'Others Similarity Average': 1.8819007137790322,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Size': 6332,
  'Width': 93.75,
  'Height': 93.75,
  'TP Similarity': 90.11188507080078,
  'Others Similarity Average': 2.7425854625180364,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Size': 365365,
  'Width': 720.0,
  'Height': 1280.0,
  'TP Similarity': 92.85736846923828,
  'Others Similarity Average': 0.8608262008056045,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Size': 162022,
  'Width': 360.0,
  'Height': 640.0,
  'TP Similarity': 93.68726348876953,
  'Others Similarity Average': 0.9043132856022567,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Size': 46603,
  'Width': 180.0,
  'Height': 320.0,
  'TP Similarity': 95.63734436035156,
  'Others Similarity Average': 1.1009649073239416,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Size': 13804,
  'Width': 90.0,
  'Height': 160.0,
  'TP Similarity': 94.93598175048828,
  'Others Similarity Average': 1.324639149941504,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Size': 1904711,
  'Width': 1448.0,
  'Height': 2573.0,
  'TP Similarity': 98.12653350830078,
  'Others Similarity Average': 0.9574354549404234,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Size': 502221,
  'Width': 724.0,
  'Height': 1286.5,
  'TP Similarity': 98.43992614746094,
  'Others Similarity Average': 1.0338331214152277,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Size': 133134,
  'Width': 362.0,
  'Height': 643.25,
  'TP Similarity': 98.84510803222656,
  'Others Similarity Average': 1.2275023572146893,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Size': 38031,
  'Width': 181.0,
  'Height': 321.625,
  'TP Similarity': 98.92985534667969,
  'Others Similarity Average': 1.2932035611011088,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Size': 11740,
  'Width': 90.5,
  'Height': 160.8125,
  'TP Similarity': 99.25614166259766,
  'Others Similarity Average': 1.5398996006697416,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0}]
    ,
[{'Name': 'AlbaQuiroz',
  'Size': 1480511,
  'Width': 2640.0,
  'Height': 1980.0,
  'TP Similarity': 52.032691955566406,
  'Others Similarity Average': 1.1335056978277862,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AlbaQuiroz',
  'Size': 411929,
  'Width': 1320.0,
  'Height': 990.0,
  'TP Similarity': 51.188514709472656,
  'Others Similarity Average': 1.0950422207824886,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AlbaQuiroz',
  'Size': 109834,
  'Width': 660.0,
  'Height': 495.0,
  'TP Similarity': 48.64827346801758,
  'Others Similarity Average': 1.1309407451190054,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AlbaQuiroz',
  'Size': 31881,
  'Width': 330.0,
  'Height': 247.5,
  'TP Similarity': 66.56327056884766,
  'Others Similarity Average': 1.4832819811999798,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AlbaQuiroz',
  'Size': 10426,
  'Width': 165.0,
  'Height': 123.75,
  'TP Similarity': 74.11644744873047,
  'Others Similarity Average': 1.7114190869033337,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'CarlosBornacelly',
  'Size': 2656147,
  'Width': 1815.0,
  'Height': 2420.0,
  'TP Similarity': 99.96061706542969,
  'Others Similarity Average': 2.4877804941497743,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 678168,
  'Width': 907.5,
  'Height': 1210.0,
  'TP Similarity': 99.95845794677734,
  'Others Similarity Average': 2.2892434410750866,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 167717,
  'Width': 453.75,
  'Height': 605.0,
  'TP Similarity': 99.9660873413086,
  'Others Similarity Average': 2.4379043760709465,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 43922,
  'Width': 226.875,
  'Height': 302.5,
  'TP Similarity': 99.97856903076172,
  'Others Similarity Average': 2.4275219002738595,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 12822,
  'Width': 113.4375,
  'Height': 151.25,
  'TP Similarity': 99.9790267944336,
  'Others Similarity Average': 2.631549533456564,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Size': 406218,
  'Width': 960.0,
  'Height': 1280.0,
  'TP Similarity': 99.71452331542969,
  'Others Similarity Average': 0.9159824112430215,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Size': 176177,
  'Width': 480.0,
  'Height': 640.0,
  'TP Similarity': 99.55833435058594,
  'Others Similarity Average': 0.9973501791246235,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Size': 46878,
  'Width': 240.0,
  'Height': 320.0,
  'TP Similarity': 99.57971954345703,
  'Others Similarity Average': 1.4116801479831338,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Size': 13242,
  'Width': 120.0,
  'Height': 160.0,
  'TP Similarity': 99.60648345947266,
  'Others Similarity Average': 1.4767442736774683,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Size': 499704,
  'Width': 1560.0,
  'Height': 2104.0,
  'TP Similarity': 65.35408020019531,
  'Others Similarity Average': 1.4947621640749276,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielCastanez',
  'Size': 218030,
  'Width': 780.0,
  'Height': 1052.0,
  'TP Similarity': 65.63046264648438,
  'Others Similarity Average': 1.5068966997787356,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielCastanez',
  'Size': 65930,
  'Width': 390.0,
  'Height': 526.0,
  'TP Similarity': 87.1346206665039,
  'Others Similarity Average': 1.8537437859922647,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Size': 20995,
  'Width': 195.0,
  'Height': 263.0,
  'TP Similarity': 90.38933563232422,
  'Others Similarity Average': 1.9272237052209675,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Size': 6987,
  'Width': 97.5,
  'Height': 131.5,
  'TP Similarity': 97.11119842529297,
  'Others Similarity Average': 2.49263354530558,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 1753278,
  'Width': 1448.0,
  'Height': 2573.0,
  'TP Similarity': 84.16940307617188,
  'Others Similarity Average': 2.7104345681145787,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 501727,
  'Width': 724.0,
  'Height': 1286.5,
  'TP Similarity': 84.59636688232422,
  'Others Similarity Average': 2.7372102048248053,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 145044,
  'Width': 362.0,
  'Height': 643.25,
  'TP Similarity': 90.53956604003906,
  'Others Similarity Average': 3.216217180714011,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 43406,
  'Width': 181.0,
  'Height': 321.625,
  'TP Similarity': 91.56636047363281,
  'Others Similarity Average': 5.065481286495924,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 13749,
  'Width': 90.5,
  'Height': 160.8125,
  'TP Similarity': 96.21534729003906,
  'Others Similarity Average': 6.93691205047071,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Size': 153406,
  'Width': 900.0,
  'Height': 1600.0,
  'TP Similarity': 98.92111206054688,
  'Others Similarity Average': 1.3175145196728408,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Size': 76288,
  'Width': 450.0,
  'Height': 800.0,
  'TP Similarity': 99.23614501953125,
  'Others Similarity Average': 1.6106272325851023,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Size': 23481,
  'Width': 225.0,
  'Height': 400.0,
  'TP Similarity': 99.66078186035156,
  'Others Similarity Average': 1.8605390889570117,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Size': 7635,
  'Width': 112.5,
  'Height': 200.0,
  'TP Similarity': 99.77400970458984,
  'Others Similarity Average': 2.1596696032211185,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Size': 2494917,
  'Width': 1815.0,
  'Height': 2420.0,
  'TP Similarity': 77.93065643310547,
  'Others Similarity Average': 2.1064315610565245,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'JoseVilla',
  'Size': 648495,
  'Width': 907.5,
  'Height': 1210.0,
  'TP Similarity': 79.1728515625,
  'Others Similarity Average': 2.0519534265622497,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'JoseVilla',
  'Size': 168647,
  'Width': 453.75,
  'Height': 605.0,
  'TP Similarity': 77.66632080078125,
  'Others Similarity Average': 2.044434029608965,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'JoseVilla',
  'Size': 46372,
  'Width': 226.875,
  'Height': 302.5,
  'TP Similarity': 83.11155700683594,
  'Others Similarity Average': 2.8935644165612757,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Size': 14192,
  'Width': 113.4375,
  'Height': 151.25,
  'TP Similarity': 87.10655975341797,
  'Others Similarity Average': 3.3701292499899864,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 1309286,
  'Width': 2448.0,
  'Height': 2448.0,
  'TP Similarity': 93.02420043945312,
  'Others Similarity Average': 2.01569077023305,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 283578,
  'Width': 1224.0,
  'Height': 1224.0,
  'TP Similarity': 92.03022766113281,
  'Others Similarity Average': 2.207980146398768,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 69647,
  'Width': 612.0,
  'Height': 612.0,
  'TP Similarity': 91.31702423095703,
  'Others Similarity Average': 1.9561412546318024,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 20186,
  'Width': 306.0,
  'Height': 306.0,
  'TP Similarity': 92.0831527709961,
  'Others Similarity Average': 2.1273418294731528,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 6921,
  'Width': 153.0,
  'Height': 153.0,
  'TP Similarity': 93.23113250732422,
  'Others Similarity Average': 2.7957943056244403,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Size': 56945,
  'Width': 709.0,
  'Height': 945.0,
  'TP Similarity': 23.685335159301758,
  'Others Similarity Average': 1.9644473302178085,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'ManuelRangel',
  'Size': 28130,
  'Width': 354.5,
  'Height': 472.5,
  'TP Similarity': 33.97922134399414,
  'Others Similarity Average': 2.7053330773487687,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'ManuelRangel',
  'Size': 10527,
  'Width': 177.25,
  'Height': 236.25,
  'TP Similarity': 36.170555114746094,
  'Others Similarity Average': 2.8104626378044486,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'ManuelRangel',
  'Size': 4245,
  'Width': 88.625,
  'Height': 118.125,
  'TP Similarity': 28.779865264892578,
  'Others Similarity Average': 3.2030022302642465,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'MariaLopez',
  'Size': 1234792,
  'Width': 2640.0,
  'Height': 1980.0,
  'TP Similarity': 99.54537200927734,
  'Others Similarity Average': 0.5203173724003136,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 320733,
  'Width': 1320.0,
  'Height': 990.0,
  'TP Similarity': 99.59290313720703,
  'Others Similarity Average': 0.49778187181800604,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 88227,
  'Width': 660.0,
  'Height': 495.0,
  'TP Similarity': 99.67862701416016,
  'Others Similarity Average': 0.5035302871838212,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 25172,
  'Width': 330.0,
  'Height': 247.5,
  'TP Similarity': 99.85299682617188,
  'Others Similarity Average': 0.564939078874886,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 8658,
  'Width': 165.0,
  'Height': 123.75,
  'TP Similarity': 99.92970275878906,
  'Others Similarity Average': 0.6374569768086076,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 1139778,
  'Width': 1440.0,
  'Height': 2559.0,
  'TP Similarity': 99.6398696899414,
  'Others Similarity Average': 2.7129330867901444,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 523623,
  'Width': 720.0,
  'Height': 1279.5,
  'TP Similarity': 99.6124496459961,
  'Others Similarity Average': 2.687226727604866,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 146653,
  'Width': 360.0,
  'Height': 639.75,
  'TP Similarity': 99.45877075195312,
  'Others Similarity Average': 2.500909158959985,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 42367,
  'Width': 180.0,
  'Height': 319.875,
  'TP Similarity': 99.59358978271484,
  'Others Similarity Average': 2.5680986316874623,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 12835,
  'Width': 90.0,
  'Height': 159.9375,
  'TP Similarity': 99.80890655517578,
  'Others Similarity Average': 3.9531316170468926,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 974769,
  'Width': 1740.0,
  'Height': 2319.0,
  'TP Similarity': 87.67218780517578,
  'Others Similarity Average': 0.9881745353341103,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 479683,
  'Width': 870.0,
  'Height': 1159.5,
  'TP Similarity': 85.98136138916016,
  'Others Similarity Average': 0.9662217665463686,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 139486,
  'Width': 435.0,
  'Height': 579.75,
  'TP Similarity': 86.30917358398438,
  'Others Similarity Average': 1.2783358991146088,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 38439,
  'Width': 217.5,
  'Height': 289.875,
  'TP Similarity': 88.53654479980469,
  'Others Similarity Average': 1.5541371842846274,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 11696,
  'Width': 108.75,
  'Height': 144.9375,
  'TP Similarity': 92.17350769042969,
  'Others Similarity Average': 1.7932442035526037,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0}]
]

In [ ]:
biggestResolution=0
leastResolution=99999999
numberOfElements=0

#Obtain number of elements, greatest and lowest resolution value in order to apply Larson's rule
for element in resolutionControlResults:
    for object in element:
        if(object["Size"]>biggestResolution):
            biggestResolution=object["Size"]
        if(object["Size"]<leastResolution):
            leastResolution=object["Size"]
        numberOfElements=numberOfElements+1


numberOfIntervals=math.ceil(1+3.332*np.log10(numberOfElements))
step=math.ceil((biggestResolution-leastResolution)/(numberOfIntervals))
print(f"Biggest {biggestResolution} and least {leastResolution}")
print(f"number of intervals: {numberOfIntervals}")
print(f"step : {step}")



Classify using quartiles and obtain performance indexes

In [33]:
objectGroup = []
for element in resolutionControlResults:
    objectGroup.extend(element)

#Sort elements taking into account the key "Size"
objectGroup.sort(key=lambda obj: obj["Size"])

#Compute quartile size
quartileSize = len(objectGroup) // 4
resultsClassified = [[], [], [], []]

#Distribute objects in quartiles. IF the number isn't divisible by four, the objects remaining or missing are deducted or added to the fourth array
for i, obj in enumerate(objectGroup):
    if i < quartileSize:
        resultsClassified[0].append(obj)
    elif i < 2 * quartileSize:
        resultsClassified[1].append(obj)
    elif i < 3 * quartileSize:
        resultsClassified[2].append(obj)
    else:
        resultsClassified[3].append(obj)



In [35]:

for element in resultsClassified:
   TPSimilarityTotal=0
   OthersSimilarityTotal=0
   faceCounter=0
   TP=TN=FP=FN=0
   for i,object in enumerate(element):
      TPSimilarityTotal=TPSimilarityTotal+object["TP Similarity"]
      TP=TP+object["TP"]
      TN=TN+object["TN"]
      FP=FP+object["FP"]
      FN=FN+object["FN"]

      faceCounter=faceCounter+object["TP"]+object["TN"]+object["FP"]+object["FN"]-1
      OthersSimilarityTotal=OthersSimilarityTotal+(object["Others Similarity Average"])*(object["TP"]+object["TN"]+object["FP"]+object["FN"]-1)
   SNS=TP/(TP+FN)
   PRC=TP/(TP+FP)
   print(f"Resolution between {element[0]["Size"]/1000}kb and {element[-1]["Size"]/1000}kb ------- Elements in quartile {i+1}")
   print(f"TP similarity average was : {TPSimilarityTotal/(i+1)}")
   print(f"Other faces similarity average was : {OthersSimilarityTotal/faceCounter} ")
   print(f"F1-Score was {2*((SNS*PRC)/(SNS+PRC))}")
   print(f"TP:{TP} TN:{TN} FP: {FP} FN: {FN}")
   print("")

Resolution between 4.245kb and 20.953kb ------- Elements in quartile 148
TP similarity average was : 91.05189779964653
Other faces similarity average was : 2.5070251127766205 
F1-Score was 0.9314079422382672
TP:129 TN:2312 FP: 0 FN: 19

Resolution between 20.953kb and 104.017kb ------- Elements in quartile 148
TP similarity average was : 89.54347084664009
Other faces similarity average was : 2.07902388406487 
F1-Score was 0.9077490774907748
TP:123 TN:2328 FP: 0 FN: 25

Resolution between 104.017kb and 524.959kb ------- Elements in quartile 148
TP similarity average was : 87.25257496576052
Other faces similarity average was : 1.7985032574680049 
F1-Score was 0.8955223880597014
TP:120 TN:2381 FP: 0 FN: 28

Resolution between 524.959kb and 4160.186kb ------- Elements in quartile 150
TP similarity average was : 86.1447571627299
Other faces similarity average was : 1.8174695273158712 
F1-Score was 0.8805970149253731
TP:118 TN:2280 FP: 0 FN: 32



Brigthness control

Calculate brightness of an image converting it to black and white array and obtaining the mean of the pixels

In [83]:
def calculate_brightness_simple(image_path):
    img = Image.open(image_path)
    gray_img = img.convert('L')
    pixel_array = np.array(gray_img)
    average_brightness = np.mean(pixel_array)
    normalized_brightness = average_brightness / 255

    return math.ceil(normalized_brightness*100)


In [ ]:
selectedCourse="Robotics"
student_photos_path=os.path.join("..","ImagesRekognition",f"Students{selectedCourse}")
camera_frame=os.path.join("..","ImagesRekognition","RealClassroomPhotos","classroom11100.jpg")
camera_photo= Image.open(camera_frame)
cameraWidth,cameraHeight=camera_photo.size

with open(camera_frame, 'rb') as image_file:
    camera_frame_bytes = image_file.read()
results=[]

i=0
for student in os.listdir(student_photos_path):
    if student.split(".")[0] in groundTruthNames:
        if student.split(".")[1]=="jpeg":
            imageQuality=100
        elif student.split(".")[1]=="jpg":
            imageQuality=95
        #Define path to individual photo
        student_id_photo=os.path.join("..","imagesRekognition",f"Students{selectedCourse}",student)
        #Obtain image's dimensions in order to know if its size is valid for the minimal rekognition's requirements
        baseImage=Image.open(student_id_photo)
        initialBox=True

        brigthnessValuesArray=[0.1, 0.5, 1, 1.5, 3]
        for brightnessValue in brigthnessValuesArray:
            #Take base image, resize and save it according to a multiplier
            studentIdPhotoBrightnessModified=baseImage
            enhancer=ImageEnhance.Brightness(studentIdPhotoBrightnessModified)
            enhanced_img=enhancer.enhance(brightnessValue)
            enhanced_img.save(rf"modifiedBrightnessImages\{brightnessValue}_{student}")

            studentIdPhotoBrightnessModifiedPath=rf"modifiedBrightnessImages\{brightnessValue}_{student}"
            realBrightness=calculate_brightness_simple(rf"modifiedBrightnessImages\{brightnessValue}_{student}")
            TPSimilarityAverage=0
            FPSimilarityAverage=0
            FPCounter=0
            TP=TN=FN=FP=0
            #Convert to bytes the resixed image
            with open(studentIdPhotoBrightnessModifiedPath, 'rb') as image_file:
                studentIdPhotoBrightnessModifiedPathBytes = image_file.read()
            #Look for the resized students photo in the group photo, reduce threshold to obtain similarity info about all faces detected
            responseRecognition=client.compare_faces(
                        SourceImage={'Bytes': studentIdPhotoBrightnessModifiedPathBytes},
                        TargetImage={'Bytes': camera_frame_bytes},
                        SimilarityThreshold=0
                    )
            
            #Save the corresponding face gallery coordinates in order to compare
            if (initialBox):
                for element in tuples[0]:
                    if element[0]==student.split(".")[0]:
                        trueBoundingBoxWithId=(student,element[1]/cameraWidth,element[2]/cameraHeight,
                                               element[3]/cameraWidth,element[4]/cameraHeight)
                        break

            initialBox=False

            print(f"Results for brightness {brightnessValue}")

            for face in responseRecognition["FaceMatches"]:
                partialBB=face["Face"]["BoundingBox"]
                IoU=calculate_iou(trueBoundingBoxWithId, ("Name",partialBB["Left"],partialBB["Top"],partialBB["Width"],partialBB["Height"]))
                if (IoU>=0.5):
                    TPSimilarityAverage=TPSimilarityAverage+float(face["Similarity"])
                    if(face["Similarity"]>80):
                        TP=TP+1
                    print(f"{student.split(".")[0]} recognized in {partialBB} with a similarity of {face["Similarity"]}")
                else:
                    FPSimilarityAverage=FPSimilarityAverage+float(face["Similarity"])
                    FPCounter=FPCounter+1
                    if face["Similarity"]>80:
                        FP=FP+1
                        print(f"Error occurred, the model mismatched a face in {partialBB} with {face["Similarity"]}")
                    else:
                        TN=TN+1
            if (TP==0):
                FN=1
            results.append({"Name":student.split(".")[0],"Brightness Modifying Factor":brightnessValue, "Real Measured Brightness": realBrightness , "TP Similarity": TPSimilarityAverage,
                            "Others Similarity Average": FPSimilarityAverage/FPCounter,"TP":TP,"TN":TN,"FP": FP,"FN":FN})
    else:
        print(f"{student.split(".")[0]} not in ground truth array ")

    i=i+1


Store data

In [78]:
brightnessControlResults=[
[{'Name': 'AlbaQuiroz',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 88.01058197021484,
  'Others Similarity Average': 2.1969918753951787,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlbaQuiroz',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 24,
  'TP Similarity': 88.20014953613281,
  'Others Similarity Average': 1.8983517281711102,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlbaQuiroz',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 48,
  'TP Similarity': 87.35999298095703,
  'Others Similarity Average': 2.0486047476530076,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlbaQuiroz',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 68,
  'TP Similarity': 87.50442504882812,
  'Others Similarity Average': 2.3855127837508916,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlbaQuiroz',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 87,
  'TP Similarity': 77.08657836914062,
  'Others Similarity Average': 2.2052392676472663,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'AlejandroCasallas',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 99.5874252319336,
  'Others Similarity Average': 1.1364270502701401,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 34,
  'TP Similarity': 99.61628723144531,
  'Others Similarity Average': 0.9606295792385936,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 67,
  'TP Similarity': 99.6025619506836,
  'Others Similarity Average': 1.0072110334411264,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 93,
  'TP Similarity': 99.41980743408203,
  'Others Similarity Average': 0.9612692285329103,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 97,
  'TP Similarity': 91.68767547607422,
  'Others Similarity Average': 1.0099311149679124,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 99.90065002441406,
  'Others Similarity Average': 1.585069509781897,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 28,
  'TP Similarity': 99.89723205566406,
  'Others Similarity Average': 1.048238249309361,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 55,
  'TP Similarity': 99.83711242675781,
  'Others Similarity Average': 0.9902661461383104,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 76,
  'TP Similarity': 99.83284759521484,
  'Others Similarity Average': 0.9059900196269155,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 91,
  'TP Similarity': 99.84764099121094,
  'Others Similarity Average': 1.2998270755633712,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 92.56886291503906,
  'Others Similarity Average': 3.267968349158764,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 34,
  'TP Similarity': 94.02654266357422,
  'Others Similarity Average': 3.259969189018011,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 67,
  'TP Similarity': 95.1724853515625,
  'Others Similarity Average': 3.1305185072124004,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 85,
  'TP Similarity': 92.94146728515625,
  'Others Similarity Average': 3.386878427118063,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 93,
  'TP Similarity': 86.89830017089844,
  'Others Similarity Average': 2.069833429157734,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 84.31819915771484,
  'Others Similarity Average': 1.8538203455507756,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 24,
  'TP Similarity': 88.84516906738281,
  'Others Similarity Average': 1.8374723881483077,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 47,
  'TP Similarity': 91.47819519042969,
  'Others Similarity Average': 1.8632595747709275,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 65,
  'TP Similarity': 92.30393981933594,
  'Others Similarity Average': 1.6756079241633415,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 82,
  'TP Similarity': 78.9549331665039,
  'Others Similarity Average': 1.4334005046635867,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'CarlosBornacelly',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 99.97984313964844,
  'Others Similarity Average': 4.555502036213875,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 30,
  'TP Similarity': 99.98910522460938,
  'Others Similarity Average': 3.8505742132663725,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 60,
  'TP Similarity': 99.98979187011719,
  'Others Similarity Average': 3.5141441985964774,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 82,
  'TP Similarity': 99.97977447509766,
  'Others Similarity Average': 3.583403731137514,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 94,
  'TP Similarity': 99.91567993164062,
  'Others Similarity Average': 6.88985390663147,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 99.97991943359375,
  'Others Similarity Average': 1.3559506569057702,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 32,
  'TP Similarity': 99.95281219482422,
  'Others Similarity Average': 1.8501320749521255,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 63,
  'TP Similarity': 99.97116088867188,
  'Others Similarity Average': 1.6291546270251274,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 78,
  'TP Similarity': 99.97628021240234,
  'Others Similarity Average': 1.715571514517069,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 86,
  'TP Similarity': 99.81299591064453,
  'Others Similarity Average': 1.485349639505148,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'EstebanToro',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 48.53325271606445,
  'Others Similarity Average': 2.7246740356087686,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'EstebanToro',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 26,
  'TP Similarity': 49.53548812866211,
  'Others Similarity Average': 2.267951089143753,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'EstebanToro',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 52,
  'TP Similarity': 46.55854797363281,
  'Others Similarity Average': 2.299416425824165,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'EstebanToro',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 69,
  'TP Similarity': 58.182437896728516,
  'Others Similarity Average': 1.8683922123163939,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'EstebanToro',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 80,
  'TP Similarity': 30.125534057617188,
  'Others Similarity Average': 2.6571992810815575,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielCastanez',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 95.32503509521484,
  'Others Similarity Average': 1.2434487283229827,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 36,
  'TP Similarity': 93.80210876464844,
  'Others Similarity Average': 0.999935993924737,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 72,
  'TP Similarity': 89.58196258544922,
  'Others Similarity Average': 0.9302847081795335,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 93,
  'TP Similarity': 94.20719146728516,
  'Others Similarity Average': 0.9882729848846793,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 99,
  'TP Similarity': 94.60247039794922,
  'Others Similarity Average': 1.1900492012500763,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 98.93363952636719,
  'Others Similarity Average': 3.5290984816849233,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 29,
  'TP Similarity': 98.43285369873047,
  'Others Similarity Average': 2.6955130331218244,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 59,
  'TP Similarity': 97.7962875366211,
  'Others Similarity Average': 3.019160672277212,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 80,
  'TP Similarity': 98.52165985107422,
  'Others Similarity Average': 2.705001349002123,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 94,
  'TP Similarity': 98.60499572753906,
  'Others Similarity Average': 2.7233989045023916,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 99.74689483642578,
  'Others Similarity Average': 1.053610473126173,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 32,
  'TP Similarity': 99.7249526977539,
  'Others Similarity Average': 0.8842603323981166,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 63,
  'TP Similarity': 99.6865234375,
  'Others Similarity Average': 1.0130176421254873,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 87,
  'TP Similarity': 99.8038101196289,
  'Others Similarity Average': 1.0729636332020163,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 95,
  'TP Similarity': 99.49944305419922,
  'Others Similarity Average': 0.8708102442324162,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 97.2891616821289,
  'Others Similarity Average': 2.7103342916816473,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 29,
  'TP Similarity': 95.5040512084961,
  'Others Similarity Average': 2.7353203585371375,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 57,
  'TP Similarity': 95.7163314819336,
  'Others Similarity Average': 2.422147117368877,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 80,
  'TP Similarity': 94.73522186279297,
  'Others Similarity Average': 2.5878288727253675,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 94,
  'TP Similarity': 89.71369171142578,
  'Others Similarity Average': 1.707136263139546,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 87.45349884033203,
  'Others Similarity Average': 2.6973644509911536,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 32,
  'TP Similarity': 87.8018798828125,
  'Others Similarity Average': 2.3161489522084593,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 64,
  'TP Similarity': 92.86930084228516,
  'Others Similarity Average': 1.8919164406135678,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 90,
  'TP Similarity': 88.68183135986328,
  'Others Similarity Average': 2.299023671448231,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 99,
  'TP Similarity': 69.93314361572266,
  'Others Similarity Average': 2.6518351513892413,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'ManuelRangel',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 8,
  'TP Similarity': 99.93527221679688,
  'Others Similarity Average': 4.346984792873263,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 37,
  'TP Similarity': 99.91902160644531,
  'Others Similarity Average': 3.908978560566902,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 73,
  'TP Similarity': 99.91717529296875,
  'Others Similarity Average': 3.784017212688923,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 88,
  'TP Similarity': 99.87142944335938,
  'Others Similarity Average': 3.3938391588628294,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 96,
  'TP Similarity': 98.30668640136719,
  'Others Similarity Average': 2.521917455736548,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaBerdugo',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 85.8554916381836,
  'Others Similarity Average': 2.8346631832420828,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaBerdugo',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 30,
  'TP Similarity': 84.49813079833984,
  'Others Similarity Average': 2.6274335484951736,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaBerdugo',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 59,
  'TP Similarity': 89.5107650756836,
  'Others Similarity Average': 3.3991156943142413,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaBerdugo',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 75,
  'TP Similarity': 87.74154663085938,
  'Others Similarity Average': 2.8062998823821546,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaBerdugo',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 86,
  'TP Similarity': 84.43879699707031,
  'Others Similarity Average': 1.990517020970583,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 99.27285766601562,
  'Others Similarity Average': 1.782124679815024,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 32,
  'TP Similarity': 98.71829986572266,
  'Others Similarity Average': 1.2005760760046542,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 64,
  'TP Similarity': 99.11638641357422,
  'Others Similarity Average': 1.2658425320871174,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 89,
  'TP Similarity': 99.06282043457031,
  'Others Similarity Average': 1.5570255869999527,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 99,
  'TP Similarity': 95.58234405517578,
  'Others Similarity Average': 1.601969309709966,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MauricioDeLaHoz',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 4,
  'TP Similarity': 82.57122802734375,
  'Others Similarity Average': 1.2647012013942003,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MauricioDeLaHoz',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 21,
  'TP Similarity': 68.89955139160156,
  'Others Similarity Average': 1.1582052946090697,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MauricioDeLaHoz',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 42,
  'TP Similarity': 76.35456085205078,
  'Others Similarity Average': 1.3674629025161267,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MauricioDeLaHoz',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 60,
  'TP Similarity': 72.98904418945312,
  'Others Similarity Average': 1.358123305812478,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MauricioDeLaHoz',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 83,
  'TP Similarity': 61.867313385009766,
  'Others Similarity Average': 1.8496808156371116,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'SergioFonseca',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 99.49628448486328,
  'Others Similarity Average': 2.047520363330841,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 34,
  'TP Similarity': 99.62598419189453,
  'Others Similarity Average': 2.606442754715681,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 67,
  'TP Similarity': 99.54402160644531,
  'Others Similarity Average': 2.3292657585814593,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 85,
  'TP Similarity': 99.49909210205078,
  'Others Similarity Average': 2.0139601696282625,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 98,
  'TP Similarity': 90.35474395751953,
  'Others Similarity Average': 1.9362657815217972,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 97.89749145507812,
  'Others Similarity Average': 0.98249436840415,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 24,
  'TP Similarity': 98.79549407958984,
  'Others Similarity Average': 1.1270706325769424,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 48,
  'TP Similarity': 98.94715881347656,
  'Others Similarity Average': 1.1727693067863583,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 65,
  'TP Similarity': 99.02716827392578,
  'Others Similarity Average': 1.3798190891742705,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 83,
  'TP Similarity': 86.25108337402344,
  'Others Similarity Average': 1.0903073767200113,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0}]
,
[{'Name': 'AlbaQuiroz',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 74.5663070678711,
  'Others Similarity Average': 1.005218437872827,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'AlbaQuiroz',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 24,
  'TP Similarity': 78.34637451171875,
  'Others Similarity Average': 0.9221500352025032,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'AlbaQuiroz',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 48,
  'TP Similarity': 79.85643768310547,
  'Others Similarity Average': 0.9912954699248075,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'AlbaQuiroz',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 68,
  'TP Similarity': 79.53730010986328,
  'Others Similarity Average': 1.1666507488116622,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'AlbaQuiroz',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 87,
  'TP Similarity': 59.736228942871094,
  'Others Similarity Average': 0.9164359427988529,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'AlejandroCasallas',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 82.80094146728516,
  'Others Similarity Average': 1.0571303639560938,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 34,
  'TP Similarity': 82.20821380615234,
  'Others Similarity Average': 1.0232938297092915,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 67,
  'TP Similarity': 85.80200958251953,
  'Others Similarity Average': 1.1137323319911956,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 93,
  'TP Similarity': 84.87799072265625,
  'Others Similarity Average': 1.0276214707642795,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 97,
  'TP Similarity': 60.399131774902344,
  'Others Similarity Average': 0.8536014646291733,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'AlejandroVelasco',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 99.75849151611328,
  'Others Similarity Average': 1.3709403734654189,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 28,
  'TP Similarity': 99.75609588623047,
  'Others Similarity Average': 0.9855959098786116,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 55,
  'TP Similarity': 99.7657699584961,
  'Others Similarity Average': 0.9778900315985084,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 76,
  'TP Similarity': 99.77227783203125,
  'Others Similarity Average': 0.8779998825863004,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 91,
  'TP Similarity': 99.43981170654297,
  'Others Similarity Average': 1.0675977632403373,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 82.81414794921875,
  'Others Similarity Average': 3.4136287586763503,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 34,
  'TP Similarity': 84.042724609375,
  'Others Similarity Average': 3.1845361830666663,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 67,
  'TP Similarity': 85.96520233154297,
  'Others Similarity Average': 3.078850771673024,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 85,
  'TP Similarity': 84.40591430664062,
  'Others Similarity Average': 3.1460376208648086,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 93,
  'TP Similarity': 69.63329315185547,
  'Others Similarity Average': 2.2791628492996097,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'AndresAnillo',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 83.51392364501953,
  'Others Similarity Average': 2.092092467099428,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 24,
  'TP Similarity': 88.34542846679688,
  'Others Similarity Average': 2.0551652811467647,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 47,
  'TP Similarity': 91.48632049560547,
  'Others Similarity Average': 2.29701404646039,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 65,
  'TP Similarity': 94.13457489013672,
  'Others Similarity Average': 2.121199209243059,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 82,
  'TP Similarity': 81.2099838256836,
  'Others Similarity Average': 1.4939185258001089,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 99.9623794555664,
  'Others Similarity Average': 3.374993256106973,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 30,
  'TP Similarity': 99.98467254638672,
  'Others Similarity Average': 3.153124178946018,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 60,
  'TP Similarity': 99.98246765136719,
  'Others Similarity Average': 2.7596656568348408,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 82,
  'TP Similarity': 99.96366119384766,
  'Others Similarity Average': 2.878652698919177,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 94,
  'TP Similarity': 99.85601043701172,
  'Others Similarity Average': 4.969975094497204,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 99.99666595458984,
  'Others Similarity Average': 2.082003529369831,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 32,
  'TP Similarity': 99.9970703125,
  'Others Similarity Average': 2.736656844243407,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 63,
  'TP Similarity': 99.9983901977539,
  'Others Similarity Average': 2.5572612468153237,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 78,
  'TP Similarity': 99.99845886230469,
  'Others Similarity Average': 2.496662601083517,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 86,
  'TP Similarity': 99.96273040771484,
  'Others Similarity Average': 2.367504256032407,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'EstebanToro',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 23.000972747802734,
  'Others Similarity Average': 1.854222732037306,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'EstebanToro',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 26,
  'TP Similarity': 22.594648361206055,
  'Others Similarity Average': 1.6235562454909087,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'EstebanToro',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 52,
  'TP Similarity': 24.16159439086914,
  'Others Similarity Average': 1.6524500973522662,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'EstebanToro',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 69,
  'TP Similarity': 21.470407485961914,
  'Others Similarity Average': 1.4022385641932487,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'EstebanToro',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 80,
  'TP Similarity': 7.07533597946167,
  'Others Similarity Average': 1.588840689510107,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielCastanez',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 87.90132904052734,
  'Others Similarity Average': 0.9032652314752341,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 36,
  'TP Similarity': 90.20160675048828,
  'Others Similarity Average': 0.6546931620687246,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 72,
  'TP Similarity': 88.95003509521484,
  'Others Similarity Average': 0.673047487437725,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 93,
  'TP Similarity': 91.89338684082031,
  'Others Similarity Average': 0.7021664548665285,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 99,
  'TP Similarity': 63.26514434814453,
  'Others Similarity Average': 0.7029921807348728,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'GiohanOlivares',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 96.66763305664062,
  'Others Similarity Average': 3.892719651013613,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 29,
  'TP Similarity': 94.46363067626953,
  'Others Similarity Average': 2.9434641383588316,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 59,
  'TP Similarity': 93.43003845214844,
  'Others Similarity Average': 3.189732679724693,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 80,
  'TP Similarity': 94.83776092529297,
  'Others Similarity Average': 3.038669714331627,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 94,
  'TP Similarity': 97.11970520019531,
  'Others Similarity Average': 3.039774616807699,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 99.84325408935547,
  'Others Similarity Average': 0.8837755771353841,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 32,
  'TP Similarity': 99.7923583984375,
  'Others Similarity Average': 0.8849699718877673,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 63,
  'TP Similarity': 99.80581665039062,
  'Others Similarity Average': 0.9710116410627961,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 87,
  'TP Similarity': 99.84487915039062,
  'Others Similarity Average': 1.0795750331133604,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 95,
  'TP Similarity': 99.6285171508789,
  'Others Similarity Average': 0.9219930704683066,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 91.45002746582031,
  'Others Similarity Average': 2.391481027752161,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 29,
  'TP Similarity': 88.14852905273438,
  'Others Similarity Average': 2.4682904321700336,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 57,
  'TP Similarity': 91.1860580444336,
  'Others Similarity Average': 2.346254600957036,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 80,
  'TP Similarity': 89.84803771972656,
  'Others Similarity Average': 2.3905782517045737,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 94,
  'TP Similarity': 84.2878189086914,
  'Others Similarity Average': 1.3996314726769925,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 87.10272216796875,
  'Others Similarity Average': 1.4643566926941276,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 32,
  'TP Similarity': 93.85466766357422,
  'Others Similarity Average': 1.1645700505003334,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 64,
  'TP Similarity': 95.17949676513672,
  'Others Similarity Average': 1.0633814785629512,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 90,
  'TP Similarity': 94.88046264648438,
  'Others Similarity Average': 1.3009448196738957,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 99,
  'TP Similarity': 67.3231201171875,
  'Others Similarity Average': 1.6487240873277187,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'ManuelRangel',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 8,
  'TP Similarity': 99.99028015136719,
  'Others Similarity Average': 4.958468579500914,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 37,
  'TP Similarity': 99.98872375488281,
  'Others Similarity Average': 5.486388131231069,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 73,
  'TP Similarity': 99.98037719726562,
  'Others Similarity Average': 5.576162705570459,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 88,
  'TP Similarity': 99.9759750366211,
  'Others Similarity Average': 4.920691722631455,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 96,
  'TP Similarity': 99.90704345703125,
  'Others Similarity Average': 4.1410790123045444,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaBerdugo',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 36.89340591430664,
  'Others Similarity Average': 2.3880215503275393,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MariaBerdugo',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 30,
  'TP Similarity': 62.47700881958008,
  'Others Similarity Average': 2.399203757196665,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MariaBerdugo',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 59,
  'TP Similarity': 66.96556854248047,
  'Others Similarity Average': 2.4237869784235953,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MariaBerdugo',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 75,
  'TP Similarity': 54.76073455810547,
  'Others Similarity Average': 2.1724979914724827,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MariaBerdugo',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 86,
  'TP Similarity': 43.183170318603516,
  'Others Similarity Average': 1.259925878420472,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MariaLopez',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 95.00772857666016,
  'Others Similarity Average': 0.793061102554202,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 32,
  'TP Similarity': 92.7301254272461,
  'Others Similarity Average': 0.7007740017026662,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 64,
  'TP Similarity': 96.71992492675781,
  'Others Similarity Average': 0.7637720350176096,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 89,
  'TP Similarity': 94.39450073242188,
  'Others Similarity Average': 0.8468008801341057,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 99,
  'TP Similarity': 72.8812484741211,
  'Others Similarity Average': 0.9420293673872948,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MauricioDeLaHoz',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 4,
  'TP Similarity': 38.34391403198242,
  'Others Similarity Average': 1.1299863379448651,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MauricioDeLaHoz',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 21,
  'TP Similarity': 25.78199577331543,
  'Others Similarity Average': 1.0470409601926804,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MauricioDeLaHoz',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 42,
  'TP Similarity': 36.3138542175293,
  'Others Similarity Average': 1.2259764846414327,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MauricioDeLaHoz',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 60,
  'TP Similarity': 31.315866470336914,
  'Others Similarity Average': 1.2263694431632757,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MauricioDeLaHoz',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 83,
  'TP Similarity': 18.764904022216797,
  'Others Similarity Average': 1.4819452613592148,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'SergioFonseca',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 99.76094055175781,
  'Others Similarity Average': 3.4563798416405915,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 34,
  'TP Similarity': 99.68158721923828,
  'Others Similarity Average': 3.950433214753866,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 67,
  'TP Similarity': 99.69081115722656,
  'Others Similarity Average': 3.918403945490718,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 85,
  'TP Similarity': 99.68998718261719,
  'Others Similarity Average': 3.5419018886983396,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 98,
  'TP Similarity': 97.01907348632812,
  'Others Similarity Average': 3.49512426443398,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 94.01709747314453,
  'Others Similarity Average': 1.364824216067791,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 24,
  'TP Similarity': 95.09034729003906,
  'Others Similarity Average': 1.4256251931190491,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 48,
  'TP Similarity': 96.29116821289062,
  'Others Similarity Average': 1.3835722923278808,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 65,
  'TP Similarity': 96.90060424804688,
  'Others Similarity Average': 1.5781916379928589,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 83,
  'TP Similarity': 46.122589111328125,
  'Others Similarity Average': 0.9182126186788082,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1}]
,
[{'Name': 'AllysonSalom',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 98.53809356689453,
  'Others Similarity Average': 1.829109886661172,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 30,
  'TP Similarity': 98.75013732910156,
  'Others Similarity Average': 1.2692551333457232,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 59,
  'TP Similarity': 99.15478515625,
  'Others Similarity Average': 1.5014729928225279,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 76,
  'TP Similarity': 98.3895263671875,
  'Others Similarity Average': 1.2720602210611105,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 89,
  'TP Similarity': 91.40753936767578,
  'Others Similarity Average': 3.8978457432240248,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 96.59619903564453,
  'Others Similarity Average': 0.4521115217357874,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 28,
  'TP Similarity': 96.91472625732422,
  'Others Similarity Average': 0.5656287111341953,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 55,
  'TP Similarity': 96.80204772949219,
  'Others Similarity Average': 0.5912975873798132,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 71,
  'TP Similarity': 96.49905395507812,
  'Others Similarity Average': 0.7041579987853765,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 84,
  'TP Similarity': 92.94527435302734,
  'Others Similarity Average': 0.42133802361786366,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 99.99571228027344,
  'Others Similarity Average': 2.170890599489212,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 28,
  'TP Similarity': 99.99822998046875,
  'Others Similarity Average': 2.2610618323087692,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 56,
  'TP Similarity': 99.99815368652344,
  'Others Similarity Average': 2.326705366373062,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 76,
  'TP Similarity': 99.99671936035156,
  'Others Similarity Average': 2.3441918566823006,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 91,
  'TP Similarity': 99.97411346435547,
  'Others Similarity Average': 1.3587259724736214,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 4,
  'TP Similarity': 99.57173156738281,
  'Others Similarity Average': 0.9336015712469816,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 20,
  'TP Similarity': 99.88565063476562,
  'Others Similarity Average': 1.2998011326417327,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 40,
  'TP Similarity': 99.95091247558594,
  'Others Similarity Average': 1.2655978063121438,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 50,
  'TP Similarity': 99.92023468017578,
  'Others Similarity Average': 1.242448864504695,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 65,
  'TP Similarity': 99.90959930419922,
  'Others Similarity Average': 1.372305820696056,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DylanDeOro',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 99.9880599975586,
  'Others Similarity Average': 3.518435202538967,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DylanDeOro',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 32,
  'TP Similarity': 99.98484802246094,
  'Others Similarity Average': 4.178720813244581,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DylanDeOro',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 65,
  'TP Similarity': 99.99186706542969,
  'Others Similarity Average': 4.483760043978691,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DylanDeOro',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 70,
  'TP Similarity': 99.98783874511719,
  'Others Similarity Average': 3.881165847182274,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DylanDeOro',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 76,
  'TP Similarity': 96.13954162597656,
  'Others Similarity Average': 3.331793213263154,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 99.71528625488281,
  'Others Similarity Average': 3.3232192993164062,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 25,
  'TP Similarity': 99.68473052978516,
  'Others Similarity Average': 3.392502186819911,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 50,
  'TP Similarity': 99.7753677368164,
  'Others Similarity Average': 4.3533663004636765,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 74,
  'TP Similarity': 99.77745056152344,
  'Others Similarity Average': 3.5844663493335247,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 90,
  'TP Similarity': 97.48261260986328,
  'Others Similarity Average': 1.7214275253936648,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 72.54345703125,
  'Others Similarity Average': 1.3034116439521313,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'HabidAbdala',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 28,
  'TP Similarity': 79.493408203125,
  'Others Similarity Average': 1.6744536589831114,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'HabidAbdala',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 56,
  'TP Similarity': 82.11431121826172,
  'Others Similarity Average': 1.6522499360144138,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 78,
  'TP Similarity': 85.60923767089844,
  'Others Similarity Average': 1.4981691930443048,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 93,
  'TP Similarity': 72.22544860839844,
  'Others Similarity Average': 1.3379376754164696,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'JuanLozano',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 99.96952056884766,
  'Others Similarity Average': 1.1391082033514977,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanLozano',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 25,
  'TP Similarity': 99.9856185913086,
  'Others Similarity Average': 1.217697718180716,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanLozano',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 50,
  'TP Similarity': 99.98599243164062,
  'Others Similarity Average': 1.3044462651014328,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanLozano',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 73,
  'TP Similarity': 99.99124145507812,
  'Others Similarity Average': 1.3681305758655071,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanLozano',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 87,
  'TP Similarity': 99.98978424072266,
  'Others Similarity Average': 1.2931485883891582,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 99.51856231689453,
  'Others Similarity Average': 1.0926092052832246,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 26,
  'TP Similarity': 99.7427978515625,
  'Others Similarity Average': 0.8945727376267314,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 51,
  'TP Similarity': 99.8836669921875,
  'Others Similarity Average': 1.2580478861927986,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 72,
  'TP Similarity': 99.88809204101562,
  'Others Similarity Average': 1.103275048546493,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 84,
  'TP Similarity': 99.35282897949219,
  'Others Similarity Average': 0.9749432150274515,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0}]
,
[{'Name': 'AllysonSalom',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 99.40182495117188,
  'Others Similarity Average': 1.85300356708467,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 30,
  'TP Similarity': 99.5317611694336,
  'Others Similarity Average': 1.5233557112514973,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 59,
  'TP Similarity': 99.72551727294922,
  'Others Similarity Average': 1.6730364225804806,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 76,
  'TP Similarity': 99.51342010498047,
  'Others Similarity Average': 1.45421726629138,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 89,
  'TP Similarity': 94.40270233154297,
  'Others Similarity Average': 5.491343937814236,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 97.97297668457031,
  'Others Similarity Average': 0.5798797886818647,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 28,
  'TP Similarity': 98.17655181884766,
  'Others Similarity Average': 0.7523220609873533,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 55,
  'TP Similarity': 98.18424224853516,
  'Others Similarity Average': 0.7080531679093838,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 71,
  'TP Similarity': 97.48056030273438,
  'Others Similarity Average': 0.8296205010265112,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 84,
  'TP Similarity': 95.09803771972656,
  'Others Similarity Average': 0.6551215685904026,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 99.98524475097656,
  'Others Similarity Average': 1.7303606420755386,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 28,
  'TP Similarity': 99.9850082397461,
  'Others Similarity Average': 1.8036228884011507,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 56,
  'TP Similarity': 99.990234375,
  'Others Similarity Average': 1.7142794877290726,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 76,
  'TP Similarity': 99.98904418945312,
  'Others Similarity Average': 1.7736976891756058,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 91,
  'TP Similarity': 99.95265197753906,
  'Others Similarity Average': 1.1926895827054977,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 4,
  'TP Similarity': 98.94214630126953,
  'Others Similarity Average': 1.3917095582000911,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 20,
  'TP Similarity': 99.72447204589844,
  'Others Similarity Average': 1.4633749113418162,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 40,
  'TP Similarity': 99.84524536132812,
  'Others Similarity Average': 1.2137379222549498,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 50,
  'TP Similarity': 99.74907684326172,
  'Others Similarity Average': 1.1342328675091267,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 65,
  'TP Similarity': 99.76673126220703,
  'Others Similarity Average': 1.2081989669241011,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 99.84639739990234,
  'Others Similarity Average': 3.680484563112259,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 25,
  'TP Similarity': 99.77649688720703,
  'Others Similarity Average': 3.543222626671195,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 50,
  'TP Similarity': 99.82421875,
  'Others Similarity Average': 4.283228313550353,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 74,
  'TP Similarity': 99.87376403808594,
  'Others Similarity Average': 3.6669969260692596,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 90,
  'TP Similarity': 99.13074493408203,
  'Others Similarity Average': 1.9807391641661525,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 92.01134490966797,
  'Others Similarity Average': 1.2019858788698912,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 28,
  'TP Similarity': 94.23568725585938,
  'Others Similarity Average': 1.4574579522013664,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 56,
  'TP Similarity': 93.60900115966797,
  'Others Similarity Average': 1.5116543509066105,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 78,
  'TP Similarity': 91.96234130859375,
  'Others Similarity Average': 1.748791879042983,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 93,
  'TP Similarity': 92.37832641601562,
  'Others Similarity Average': 2.0249971505254507,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 99.82742309570312,
  'Others Similarity Average': 0.8416733955964446,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 26,
  'TP Similarity': 99.91200256347656,
  'Others Similarity Average': 0.7414470184594393,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 51,
  'TP Similarity': 99.95941925048828,
  'Others Similarity Average': 0.867758322507143,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 72,
  'TP Similarity': 99.95655059814453,
  'Others Similarity Average': 0.7891464587301016,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 84,
  'TP Similarity': 99.7653579711914,
  'Others Similarity Average': 0.7965148044750094,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0}]
,
[{'Name': 'AllysonSalom',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 99.45267486572266,
  'Others Similarity Average': 3.4918361715972424,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 30,
  'TP Similarity': 99.29071044921875,
  'Others Similarity Average': 3.3301315465942025,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 59,
  'TP Similarity': 99.5538558959961,
  'Others Similarity Average': 3.1894785556942225,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 76,
  'TP Similarity': 99.16223907470703,
  'Others Similarity Average': 2.9531085332855582,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 89,
  'TP Similarity': 96.64427947998047,
  'Others Similarity Average': 8.773234151303768,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 98.54341125488281,
  'Others Similarity Average': 0.4261716650798917,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 28,
  'TP Similarity': 98.43852233886719,
  'Others Similarity Average': 0.5049397787079215,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 55,
  'TP Similarity': 98.4059829711914,
  'Others Similarity Average': 0.5101536782458425,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 71,
  'TP Similarity': 98.35557556152344,
  'Others Similarity Average': 0.5861158017069101,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 84,
  'TP Similarity': 97.22884368896484,
  'Others Similarity Average': 0.4467185717076063,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 99.97972106933594,
  'Others Similarity Average': 1.646271701902151,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 28,
  'TP Similarity': 99.97891235351562,
  'Others Similarity Average': 1.6590362042188644,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 56,
  'TP Similarity': 99.98382568359375,
  'Others Similarity Average': 1.5845682490617037,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 76,
  'TP Similarity': 99.98604583740234,
  'Others Similarity Average': 1.6320837549865246,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 91,
  'TP Similarity': 99.9475326538086,
  'Others Similarity Average': 0.8771318234503269,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 4,
  'TP Similarity': 96.39479064941406,
  'Others Similarity Average': 1.2748436299152672,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 20,
  'TP Similarity': 98.80697631835938,
  'Others Similarity Average': 1.3756391052156687,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 40,
  'TP Similarity': 99.35296630859375,
  'Others Similarity Average': 1.0948957819491625,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 50,
  'TP Similarity': 98.99596405029297,
  'Others Similarity Average': 1.024805748835206,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 65,
  'TP Similarity': 98.88663482666016,
  'Others Similarity Average': 1.166659719310701,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 99.44154357910156,
  'Others Similarity Average': 2.942508105188608,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 25,
  'TP Similarity': 99.24964141845703,
  'Others Similarity Average': 2.9565869998186827,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 50,
  'TP Similarity': 99.2288818359375,
  'Others Similarity Average': 3.280583241954446,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 74,
  'TP Similarity': 99.4161148071289,
  'Others Similarity Average': 3.0184924695640802,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 90,
  'TP Similarity': 94.16246032714844,
  'Others Similarity Average': 1.7238133382052183,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 93.53055572509766,
  'Others Similarity Average': 1.3908745758235455,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 28,
  'TP Similarity': 96.42936706542969,
  'Others Similarity Average': 1.6176293846219778,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 56,
  'TP Similarity': 96.02462005615234,
  'Others Similarity Average': 1.6628906335681677,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 78,
  'TP Similarity': 95.49121856689453,
  'Others Similarity Average': 1.8172947689890862,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 93,
  'TP Similarity': 95.66250610351562,
  'Others Similarity Average': 1.6875299345701933,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 99.80447387695312,
  'Others Similarity Average': 0.7186266968492419,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 26,
  'TP Similarity': 99.92880249023438,
  'Others Similarity Average': 0.5860070737544447,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 51,
  'TP Similarity': 99.95974731445312,
  'Others Similarity Average': 0.6668948750011623,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 72,
  'TP Similarity': 99.95232391357422,
  'Others Similarity Average': 0.6252435566857457,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 84,
  'TP Similarity': 99.59783935546875,
  'Others Similarity Average': 0.7682922461535782,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0}]
,
[{'Name': 'AliRada',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 2,
  'TP Similarity': 27.550020217895508,
  'Others Similarity Average': 1.816616229712963,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 11,
  'TP Similarity': 31.281173706054688,
  'Others Similarity Average': 2.0646008932415176,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 22,
  'TP Similarity': 37.42267990112305,
  'Others Similarity Average': 2.0274264716050205,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 32,
  'TP Similarity': 42.3597412109375,
  'Others Similarity Average': 1.918477054904489,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 53,
  'TP Similarity': 13.392382621765137,
  'Others Similarity Average': 3.809286101776011,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'AndreaQuintero',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 91.7555160522461,
  'Others Similarity Average': 0.783118460984791,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndreaQuintero',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 32,
  'TP Similarity': 91.1017074584961,
  'Others Similarity Average': 0.76205353614162,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndreaQuintero',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 64,
  'TP Similarity': 90.9809341430664,
  'Others Similarity Average': 0.8814706002526423,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndreaQuintero',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 82,
  'TP Similarity': 93.0732192993164,
  'Others Similarity Average': 0.8672900622820153,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndreaQuintero',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 92,
  'TP Similarity': 91.02416229248047,
  'Others Similarity Average': 0.7801216937163297,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 98.8546142578125,
  'Others Similarity Average': 3.7607805742060436,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 30,
  'TP Similarity': 98.6980972290039,
  'Others Similarity Average': 2.859019412275623,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 60,
  'TP Similarity': 98.87459564208984,
  'Others Similarity Average': 3.03892352081397,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 77,
  'TP Similarity': 99.0721435546875,
  'Others Similarity Average': 3.1967561788418712,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 93,
  'TP Similarity': 99.06759643554688,
  'Others Similarity Average': 3.2508590177578083,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 98.70367431640625,
  'Others Similarity Average': 1.9056943343842732,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 28,
  'TP Similarity': 97.94190216064453,
  'Others Similarity Average': 1.7831507272580092,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 56,
  'TP Similarity': 98.43746948242188,
  'Others Similarity Average': 1.7416104437673794,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 77,
  'TP Similarity': 98.7347640991211,
  'Others Similarity Average': 1.745059650610475,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 90,
  'TP Similarity': 98.02775573730469,
  'Others Similarity Average': 1.299754733548445,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 96.64961242675781,
  'Others Similarity Average': 1.1532133984215118,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 33,
  'TP Similarity': 95.36695861816406,
  'Others Similarity Average': 1.3992513619801576,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 67,
  'TP Similarity': 95.56987762451172,
  'Others Similarity Average': 1.2388239814078106,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 82,
  'TP Similarity': 95.45856475830078,
  'Others Similarity Average': 1.0294968157130129,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 98,
  'TP Similarity': 63.31157684326172,
  'Others Similarity Average': 0.4788195898427683,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 4,
  'TP Similarity': 97.89405059814453,
  'Others Similarity Average': 0.815384533475427,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 20,
  'TP Similarity': 98.72679901123047,
  'Others Similarity Average': 1.4585221324335127,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 40,
  'TP Similarity': 99.26193237304688,
  'Others Similarity Average': 1.7082959654576637,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 50,
  'TP Similarity': 98.770751953125,
  'Others Similarity Average': 1.2453064738827593,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 65,
  'TP Similarity': 98.51448822021484,
  'Others Similarity Average': 1.2817865890615128,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 99.20560455322266,
  'Others Similarity Average': 3.0859069061629913,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 32,
  'TP Similarity': 98.85468292236328,
  'Others Similarity Average': 2.559763389475205,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 63,
  'TP Similarity': 98.96647644042969,
  'Others Similarity Average': 2.3324328818741966,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 73,
  'TP Similarity': 98.13899993896484,
  'Others Similarity Average': 2.015912163783522,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 82,
  'TP Similarity': 96.95953369140625,
  'Others Similarity Average': 1.8698309502180885,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielaBecerra',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 65.82658386230469,
  'Others Similarity Average': 1.4412481136181776,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 27,
  'TP Similarity': 71.22661590576172,
  'Others Similarity Average': 1.9877366984591764,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 54,
  'TP Similarity': 49.96076202392578,
  'Others Similarity Average': 1.694888133336516,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 73,
  'TP Similarity': 57.653594970703125,
  'Others Similarity Average': 1.5426626529763727,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 84,
  'TP Similarity': 67.371826171875,
  'Others Similarity Average': 1.4544496829895412,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'JuanHernandez',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 96.1058349609375,
  'Others Similarity Average': 1.5573841526227838,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanHernandez',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 29,
  'TP Similarity': 96.30870819091797,
  'Others Similarity Average': 1.420093973769861,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanHernandez',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 57,
  'TP Similarity': 97.93284606933594,
  'Others Similarity Average': 1.2475979832165383,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanHernandez',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 75,
  'TP Similarity': 97.25757598876953,
  'Others Similarity Average': 1.0486901959075647,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanHernandez',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 94,
  'TP Similarity': 93.21819305419922,
  'Others Similarity Average': 1.1525748192387468,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 99.21244049072266,
  'Others Similarity Average': 1.4867036802803768,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 27,
  'TP Similarity': 98.79637145996094,
  'Others Similarity Average': 1.6716272305039799,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 55,
  'TP Similarity': 98.88557434082031,
  'Others Similarity Average': 1.7413329929113388,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 77,
  'TP Similarity': 99.02843475341797,
  'Others Similarity Average': 1.7878941586788963,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 97,
  'TP Similarity': 98.21614837646484,
  'Others Similarity Average': 2.0921347474350647,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 99.99586486816406,
  'Others Similarity Average': 1.9895818373736214,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 29,
  'TP Similarity': 99.9953384399414,
  'Others Similarity Average': 1.6695245793637108,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 58,
  'TP Similarity': 99.99524688720703,
  'Others Similarity Average': 1.903458717991324,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 80,
  'TP Similarity': 99.99475860595703,
  'Others Similarity Average': 1.8691879116437013,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 96,
  'TP Similarity': 99.9855728149414,
  'Others Similarity Average': 1.7044661982971079,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'LucianaDeLaRosa',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 62.90346908569336,
  'Others Similarity Average': 0.9102006615961299,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 31,
  'TP Similarity': 62.11105728149414,
  'Others Similarity Average': 0.8251557779662749,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 62,
  'TP Similarity': 70.83712768554688,
  'Others Similarity Average': 0.7655029820606989,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 84,
  'TP Similarity': 68.71955871582031,
  'Others Similarity Average': 0.6161271114559734,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 96,
  'TP Similarity': 68.61097717285156,
  'Others Similarity Average': 0.9906143611844849,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'MariaFerrer',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 93.27005767822266,
  'Others Similarity Average': 1.334353144554531,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 24,
  'TP Similarity': 93.28968811035156,
  'Others Similarity Average': 1.7281439751386642,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 49,
  'TP Similarity': 95.52566528320312,
  'Others Similarity Average': 1.564881468520445,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 69,
  'TP Similarity': 95.05774688720703,
  'Others Similarity Average': 1.5841111759929096,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 89,
  'TP Similarity': 90.20153045654297,
  'Others Similarity Average': 1.486288135542589,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 99.93501281738281,
  'Others Similarity Average': 0.7642424080301734,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 23,
  'TP Similarity': 99.95480346679688,
  'Others Similarity Average': 0.7225839083685595,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 45,
  'TP Similarity': 99.9745101928711,
  'Others Similarity Average': 0.7319272436639842,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 64,
  'TP Similarity': 99.95894622802734,
  'Others Similarity Average': 0.7403453518362606,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 83,
  'TP Similarity': 99.86565399169922,
  'Others Similarity Average': 0.5961829520323697,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 4,
  'TP Similarity': 99.92753601074219,
  'Others Similarity Average': 1.264900489765055,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 19,
  'TP Similarity': 99.88971710205078,
  'Others Similarity Average': 1.318168183459955,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 38,
  'TP Similarity': 99.8543930053711,
  'Others Similarity Average': 1.506772675058421,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 53,
  'TP Similarity': 99.93317413330078,
  'Others Similarity Average': 1.6171116592252957,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 73,
  'TP Similarity': 99.87012481689453,
  'Others Similarity Average': 1.978909258456791,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 98.08599090576172,
  'Others Similarity Average': 3.3337691471199777,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 31,
  'TP Similarity': 97.62423706054688,
  'Others Similarity Average': 3.522613862748532,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 61,
  'TP Similarity': 97.96041870117188,
  'Others Similarity Average': 3.20056434622144,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 85,
  'TP Similarity': 98.276611328125,
  'Others Similarity Average': 3.747842894100091,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 97,
  'TP Similarity': 94.82477569580078,
  'Others Similarity Average': 2.8637251058483826,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0}]
,
[{'Name': 'AliRada',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 2,
  'TP Similarity': 46.70623779296875,
  'Others Similarity Average': 2.1371108848187657,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 11,
  'TP Similarity': 60.69416427612305,
  'Others Similarity Average': 2.308414332154724,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 22,
  'TP Similarity': 59.06488800048828,
  'Others Similarity Average': 2.220264555679427,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 32,
  'TP Similarity': 63.99409866333008,
  'Others Similarity Average': 2.277814149028725,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 53,
  'TP Similarity': 60.55272674560547,
  'Others Similarity Average': 3.5161660545402103,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'AndreaQuintero',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 94.65632629394531,
  'Others Similarity Average': 0.7591844797134399,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndreaQuintero',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 32,
  'TP Similarity': 89.27011108398438,
  'Others Similarity Average': 0.6604068413790729,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndreaQuintero',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 64,
  'TP Similarity': 91.5013198852539,
  'Others Similarity Average': 0.7455072916216321,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndreaQuintero',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 82,
  'TP Similarity': 90.82877349853516,
  'Others Similarity Average': 0.6420875684254699,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndreaQuintero',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 92,
  'TP Similarity': 91.22822570800781,
  'Others Similarity Average': 1.1636987398895953,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 99.07129669189453,
  'Others Similarity Average': 1.9745442188448377,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 30,
  'TP Similarity': 98.90357208251953,
  'Others Similarity Average': 1.4352467995550897,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 60,
  'TP Similarity': 99.06097412109375,
  'Others Similarity Average': 1.7365448690123029,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 77,
  'TP Similarity': 99.37906646728516,
  'Others Similarity Average': 1.6798657286498282,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 93,
  'TP Similarity': 98.96676635742188,
  'Others Similarity Average': 1.6985692638489935,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 99.3384017944336,
  'Others Similarity Average': 2.2630650190015635,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 28,
  'TP Similarity': 99.43388366699219,
  'Others Similarity Average': 2.550970537794961,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 56,
  'TP Similarity': 99.5790023803711,
  'Others Similarity Average': 2.6403275641302266,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 77,
  'TP Similarity': 99.4731216430664,
  'Others Similarity Average': 2.5949744838807316,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 90,
  'TP Similarity': 99.70491027832031,
  'Others Similarity Average': 2.381198055214352,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 92.48432159423828,
  'Others Similarity Average': 0.8165776369472345,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 33,
  'TP Similarity': 88.09709167480469,
  'Others Similarity Average': 0.7052515546480814,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 67,
  'TP Similarity': 88.65816497802734,
  'Others Similarity Average': 0.6967483576801088,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 82,
  'TP Similarity': 79.12020111083984,
  'Others Similarity Average': 0.6014684587717056,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'BrayanRubiano',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 98,
  'TP Similarity': 34.16565704345703,
  'Others Similarity Average': 0.5969808203064733,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 4,
  'TP Similarity': 99.84151458740234,
  'Others Similarity Average': 1.7694282457749877,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 20,
  'TP Similarity': 99.85286712646484,
  'Others Similarity Average': 2.465040665740768,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 40,
  'TP Similarity': 99.94434356689453,
  'Others Similarity Average': 2.484226282996436,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 50,
  'TP Similarity': 99.8537826538086,
  'Others Similarity Average': 2.369424554415875,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 65,
  'TP Similarity': 99.82577514648438,
  'Others Similarity Average': 1.9012516898413498,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 99.95919036865234,
  'Others Similarity Average': 3.5171702106793723,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 32,
  'TP Similarity': 99.94046020507812,
  'Others Similarity Average': 3.4803195579184427,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 63,
  'TP Similarity': 99.94161987304688,
  'Others Similarity Average': 3.198676092757119,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 73,
  'TP Similarity': 99.90003967285156,
  'Others Similarity Average': 2.729042095442613,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 82,
  'TP Similarity': 99.84367370605469,
  'Others Similarity Average': 1.9620286011033587,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielaBecerra',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 58.94449234008789,
  'Others Similarity Average': 1.14537177566025,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 27,
  'TP Similarity': 74.5053482055664,
  'Others Similarity Average': 1.1709225119815931,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 54,
  'TP Similarity': 74.18505096435547,
  'Others Similarity Average': 1.0473293869031801,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 73,
  'TP Similarity': 68.36034393310547,
  'Others Similarity Average': 1.039387635058827,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 84,
  'TP Similarity': 22.796096801757812,
  'Others Similarity Average': 1.1939068333142333,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'JuanHernandez',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 95.90581512451172,
  'Others Similarity Average': 0.824887857461969,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanHernandez',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 29,
  'TP Similarity': 97.02032470703125,
  'Others Similarity Average': 1.0057757670680683,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanHernandez',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 57,
  'TP Similarity': 97.30013275146484,
  'Others Similarity Average': 1.060443223764499,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanHernandez',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 75,
  'TP Similarity': 97.45325469970703,
  'Others Similarity Average': 0.9385538962152269,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanHernandez',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 94,
  'TP Similarity': 89.53607940673828,
  'Others Similarity Average': 0.8742001286397377,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 99.96450805664062,
  'Others Similarity Average': 1.6339125591847632,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 27,
  'TP Similarity': 99.98507690429688,
  'Others Similarity Average': 2.1759955088297525,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 55,
  'TP Similarity': 99.98274230957031,
  'Others Similarity Average': 2.226177658471796,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 77,
  'TP Similarity': 99.98002624511719,
  'Others Similarity Average': 2.0482491445210247,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 97,
  'TP Similarity': 99.96129608154297,
  'Others Similarity Average': 2.2111117194096246,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 99.99519348144531,
  'Others Similarity Average': 2.8552690661615796,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 29,
  'TP Similarity': 99.99543762207031,
  'Others Similarity Average': 2.371054123673174,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 58,
  'TP Similarity': 99.99549102783203,
  'Others Similarity Average': 2.6634365626507335,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 80,
  'TP Similarity': 99.99331665039062,
  'Others Similarity Average': 2.558636197613345,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 96,
  'TP Similarity': 99.982177734375,
  'Others Similarity Average': 1.8739770150019064,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'LucianaDeLaRosa',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 28.37514877319336,
  'Others Similarity Average': 2.0653756120138698,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 31,
  'TP Similarity': 30.651763916015625,
  'Others Similarity Average': 1.8263909951266315,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 62,
  'TP Similarity': 43.514068603515625,
  'Others Similarity Average': 1.9314214469244082,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 84,
  'TP Similarity': 40.48908233642578,
  'Others Similarity Average': 1.663923914440804,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 96,
  'TP Similarity': 32.40959167480469,
  'Others Similarity Average': 2.109596104671558,
  'TP': 0,
  'TN': 18,
  'FP': 0,
  'FN': 1},
 {'Name': 'MariaFerrer',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 99.27674102783203,
  'Others Similarity Average': 0.851792652780811,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 24,
  'TP Similarity': 98.91574096679688,
  'Others Similarity Average': 0.9377465026660098,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 49,
  'TP Similarity': 99.2775650024414,
  'Others Similarity Average': 0.89247569317619,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 69,
  'TP Similarity': 99.01487731933594,
  'Others Similarity Average': 0.884792243440946,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 89,
  'TP Similarity': 97.4568099975586,
  'Others Similarity Average': 1.0669645513925288,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 99.9825439453125,
  'Others Similarity Average': 0.8208864749305778,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 23,
  'TP Similarity': 99.99468231201172,
  'Others Similarity Average': 0.771922391321924,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 45,
  'TP Similarity': 99.9935302734375,
  'Others Similarity Average': 0.8126723936034573,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 64,
  'TP Similarity': 99.99369812011719,
  'Others Similarity Average': 0.8140773951179452,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 83,
  'TP Similarity': 99.98571014404297,
  'Others Similarity Average': 0.8336114585399628,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 4,
  'TP Similarity': 99.921630859375,
  'Others Similarity Average': 0.9674094008902708,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 19,
  'TP Similarity': 99.93647766113281,
  'Others Similarity Average': 0.9508887268602848,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 38,
  'TP Similarity': 99.91573333740234,
  'Others Similarity Average': 0.9822758005725013,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 53,
  'TP Similarity': 99.94719696044922,
  'Others Similarity Average': 1.1362827331241634,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 73,
  'TP Similarity': 99.91339874267578,
  'Others Similarity Average': 0.9840375176734395,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 96.83959197998047,
  'Others Similarity Average': 2.4726630159550242,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 31,
  'TP Similarity': 96.60602569580078,
  'Others Similarity Average': 2.354372078759803,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 61,
  'TP Similarity': 96.72438049316406,
  'Others Similarity Average': 2.2101338774793677,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 85,
  'TP Similarity': 97.25032043457031,
  'Others Similarity Average': 2.5421495172712536,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 97,
  'TP Similarity': 91.0044174194336,
  'Others Similarity Average': 2.13871320295665,
  'TP': 1,
  'TN': 18,
  'FP': 0,
  'FN': 0}]
,
[{'Name': 'AllysonSalom',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 82.65355682373047,
  'Others Similarity Average': 1.8424663124606013,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 30,
  'TP Similarity': 72.1763687133789,
  'Others Similarity Average': 1.3486814154312015,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'AllysonSalom',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 59,
  'TP Similarity': 74.20136260986328,
  'Others Similarity Average': 1.402872615493834,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'AllysonSalom',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 76,
  'TP Similarity': 80.3000717163086,
  'Others Similarity Average': 1.5356129622086883,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 89,
  'TP Similarity': 44.195220947265625,
  'Others Similarity Average': 4.3145793080329895,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'AngieValencia',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 14.522992134094238,
  'Others Similarity Average': 0.8170417193323374,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'AngieValencia',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 28,
  'TP Similarity': 7.888208866119385,
  'Others Similarity Average': 0.8794789128005505,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'AngieValencia',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 55,
  'TP Similarity': 11.3575439453125,
  'Others Similarity Average': 0.8995814472436905,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'AngieValencia',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 71,
  'TP Similarity': 9.796089172363281,
  'Others Similarity Average': 0.973224051296711,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'AngieValencia',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 84,
  'TP Similarity': 3.8114845752716064,
  'Others Similarity Average': 0.9562348909676075,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'DanielDelgado',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 99.93083953857422,
  'Others Similarity Average': 1.3647318426519632,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 28,
  'TP Similarity': 99.95247650146484,
  'Others Similarity Average': 1.3088625371456146,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 56,
  'TP Similarity': 99.95960235595703,
  'Others Similarity Average': 1.2337983306497335,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 76,
  'TP Similarity': 99.94009399414062,
  'Others Similarity Average': 1.2235800828784704,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 91,
  'TP Similarity': 99.31826782226562,
  'Others Similarity Average': 0.9300482897087932,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 4,
  'TP Similarity': 98.37503814697266,
  'Others Similarity Average': 1.3523441744036973,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 20,
  'TP Similarity': 97.07007598876953,
  'Others Similarity Average': 1.267873789416626,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 40,
  'TP Similarity': 96.83516693115234,
  'Others Similarity Average': 1.2475125915370882,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 50,
  'TP Similarity': 96.12223815917969,
  'Others Similarity Average': 1.0494548762217164,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 65,
  'TP Similarity': 97.36092376708984,
  'Others Similarity Average': 1.4304261896759272,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DylanDeOro',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 42.35308837890625,
  'Others Similarity Average': 2.5593683794140816,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'DylanDeOro',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 32,
  'TP Similarity': 52.60070037841797,
  'Others Similarity Average': 3.3610803484916687,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'DylanDeOro',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 65,
  'TP Similarity': 47.36221694946289,
  'Others Similarity Average': 3.363679125905037,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'DylanDeOro',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 70,
  'TP Similarity': 40.607566833496094,
  'Others Similarity Average': 3.466375306248665,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'DylanDeOro',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 76,
  'TP Similarity': 30.677383422851562,
  'Others Similarity Average': 2.40562005341053,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'FelipeOspino',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 99.01644134521484,
  'Others Similarity Average': 0.9966385923326015,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 25,
  'TP Similarity': 98.78271484375,
  'Others Similarity Average': 1.0867482014000416,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 50,
  'TP Similarity': 99.01407623291016,
  'Others Similarity Average': 1.1410747021436691,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 74,
  'TP Similarity': 99.11012268066406,
  'Others Similarity Average': 1.0036781691014767,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 90,
  'TP Similarity': 97.1445083618164,
  'Others Similarity Average': 0.6340370904654264,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 51.62701416015625,
  'Others Similarity Average': 1.965866893529892,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'HabidAbdala',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 28,
  'TP Similarity': 50.837589263916016,
  'Others Similarity Average': 2.225810669362545,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'HabidAbdala',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 56,
  'TP Similarity': 43.45123291015625,
  'Others Similarity Average': 2.7091728020459414,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'HabidAbdala',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 78,
  'TP Similarity': 44.62739181518555,
  'Others Similarity Average': 3.3662274293601513,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'HabidAbdala',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 93,
  'TP Similarity': 50.452911376953125,
  'Others Similarity Average': 3.6284715989604592,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'JuanLozano',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 98.00874328613281,
  'Others Similarity Average': 1.1534462813287973,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanLozano',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 25,
  'TP Similarity': 97.7108154296875,
  'Others Similarity Average': 1.1055992376059294,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanLozano',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 50,
  'TP Similarity': 97.91923522949219,
  'Others Similarity Average': 1.2344261649996042,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanLozano',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 73,
  'TP Similarity': 98.13214874267578,
  'Others Similarity Average': 1.2128341812640429,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanLozano',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 87,
  'TP Similarity': 97.763427734375,
  'Others Similarity Average': 1.2328985761851072,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 99.30842590332031,
  'Others Similarity Average': 0.7552906721830368,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 26,
  'TP Similarity': 99.10773468017578,
  'Others Similarity Average': 0.6556607410311699,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 51,
  'TP Similarity': 99.42860412597656,
  'Others Similarity Average': 0.6556923873722553,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 72,
  'TP Similarity': 99.38301086425781,
  'Others Similarity Average': 0.6820560023188591,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 84,
  'TP Similarity': 99.38421630859375,
  'Others Similarity Average': 0.6171690504997969,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0}]
,
[{'Name': 'AliRada',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 2,
  'TP Similarity': 34.93126678466797,
  'Others Similarity Average': 1.1812574565410614,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 11,
  'TP Similarity': 37.479286193847656,
  'Others Similarity Average': 1.1302873212844133,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 22,
  'TP Similarity': 41.062984466552734,
  'Others Similarity Average': 1.0135458512231708,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 32,
  'TP Similarity': 47.95368194580078,
  'Others Similarity Average': 1.023737926967442,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 53,
  'TP Similarity': 16.197309494018555,
  'Others Similarity Average': 1.4175290036946535,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AndreaQuintero',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 49.80790710449219,
  'Others Similarity Average': 0.6723115732893348,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AndreaQuintero',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 32,
  'TP Similarity': 45.63249588012695,
  'Others Similarity Average': 0.5462267957627773,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AndreaQuintero',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 64,
  'TP Similarity': 47.09027099609375,
  'Others Similarity Average': 0.6125038028694689,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AndreaQuintero',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 82,
  'TP Similarity': 54.34757995605469,
  'Others Similarity Average': 0.5125240369234234,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AndreaQuintero',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 92,
  'TP Similarity': 39.64013671875,
  'Others Similarity Average': 1.0237516956403852,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AndresFabregas',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 65.94017791748047,
  'Others Similarity Average': 1.365183292888105,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AndresFabregas',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 30,
  'TP Similarity': 75.76734924316406,
  'Others Similarity Average': 1.2044298788532615,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AndresFabregas',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 60,
  'TP Similarity': 80.37979888916016,
  'Others Similarity Average': 1.1452988861128688,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 77,
  'TP Similarity': 77.3922348022461,
  'Others Similarity Average': 1.1776250023394823,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AndresFabregas',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 93,
  'TP Similarity': 63.04100036621094,
  'Others Similarity Average': 1.133121351711452,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AndresNarvaez',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 92.13346099853516,
  'Others Similarity Average': 1.7136936853639781,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 28,
  'TP Similarity': 94.4728775024414,
  'Others Similarity Average': 2.504156390670687,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 56,
  'TP Similarity': 96.18318176269531,
  'Others Similarity Average': 2.3697099941782653,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 77,
  'TP Similarity': 95.29589080810547,
  'Others Similarity Average': 2.7127410946413875,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 90,
  'TP Similarity': 93.64911651611328,
  'Others Similarity Average': 2.1804998363368213,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 46.40507888793945,
  'Others Similarity Average': 0.7760722767561674,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'BrayanRubiano',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 33,
  'TP Similarity': 42.36942672729492,
  'Others Similarity Average': 0.7071233317255974,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'BrayanRubiano',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 67,
  'TP Similarity': 47.404136657714844,
  'Others Similarity Average': 0.7021705666556954,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'BrayanRubiano',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 82,
  'TP Similarity': 37.2209358215332,
  'Others Similarity Average': 0.6390055380761623,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'BrayanRubiano',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 98,
  'TP Similarity': 18.013093948364258,
  'Others Similarity Average': 0.6359628075733781,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 4,
  'TP Similarity': 98.98471069335938,
  'Others Similarity Average': 0.5121002336964011,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 20,
  'TP Similarity': 98.49761962890625,
  'Others Similarity Average': 0.6907390737906098,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 40,
  'TP Similarity': 98.3026123046875,
  'Others Similarity Average': 0.7629749686457217,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 50,
  'TP Similarity': 98.4876480102539,
  'Others Similarity Average': 0.686122125480324,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 65,
  'TP Similarity': 98.84598541259766,
  'Others Similarity Average': 0.6273855781182647,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielaBecerra',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 32.38569259643555,
  'Others Similarity Average': 1.1480389242060483,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 27,
  'TP Similarity': 31.793224334716797,
  'Others Similarity Average': 0.9693101751618087,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 54,
  'TP Similarity': 28.13216209411621,
  'Others Similarity Average': 0.9056082763709128,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 73,
  'TP Similarity': 26.76864242553711,
  'Others Similarity Average': 0.9798077875748277,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 84,
  'TP Similarity': 35.141605377197266,
  'Others Similarity Average': 1.6820924361236393,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 13.626729965209961,
  'Others Similarity Average': 0.5438498633448035,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 31,
  'TP Similarity': 12.959529876708984,
  'Others Similarity Average': 0.45454930304549634,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 62,
  'TP Similarity': 19.828479766845703,
  'Others Similarity Average': 0.4373787307413295,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 84,
  'TP Similarity': 19.26041030883789,
  'Others Similarity Average': 0.4722583501134068,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 96,
  'TP Similarity': 12.177565574645996,
  'Others Similarity Average': 0.6682159539777786,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'MariaFerrer',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 90.25926208496094,
  'Others Similarity Average': 1.364048876799643,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 24,
  'TP Similarity': 91.40741729736328,
  'Others Similarity Average': 1.512731796130538,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 49,
  'TP Similarity': 95.6847915649414,
  'Others Similarity Average': 1.7101524085737765,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 69,
  'TP Similarity': 94.2417221069336,
  'Others Similarity Average': 1.6273749228566885,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 89,
  'TP Similarity': 74.22172546386719,
  'Others Similarity Average': 1.9249442303553224,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'MayraCarreno',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 99.92578887939453,
  'Others Similarity Average': 0.7677798112854362,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 23,
  'TP Similarity': 99.96966552734375,
  'Others Similarity Average': 0.774573159404099,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 45,
  'TP Similarity': 99.97005462646484,
  'Others Similarity Average': 0.734415375161916,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 64,
  'TP Similarity': 99.9663314819336,
  'Others Similarity Average': 0.7609717966988683,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 83,
  'TP Similarity': 99.93472290039062,
  'Others Similarity Average': 0.6834306204691529,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 4,
  'TP Similarity': 91.59226989746094,
  'Others Similarity Average': 0.9398075914941728,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 19,
  'TP Similarity': 92.94015502929688,
  'Others Similarity Average': 0.8020394397899508,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 38,
  'TP Similarity': 93.05726623535156,
  'Others Similarity Average': 0.8712828301358968,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 53,
  'TP Similarity': 95.60877227783203,
  'Others Similarity Average': 0.9241046863608062,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 73,
  'TP Similarity': 91.35908508300781,
  'Others Similarity Average': 0.9768891823478043,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 97.58425903320312,
  'Others Similarity Average': 0.9206094616092741,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 31,
  'TP Similarity': 97.01908874511719,
  'Others Similarity Average': 0.8160931274760514,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 61,
  'TP Similarity': 97.4174575805664,
  'Others Similarity Average': 0.8241497855633497,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 85,
  'TP Similarity': 97.61265563964844,
  'Others Similarity Average': 0.9048627726733685,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 97,
  'TP Similarity': 93.67012023925781,
  'Others Similarity Average': 0.9829434021376073,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0}]
,
[{'Name': 'AlbaQuiroz',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 36.291046142578125,
  'Others Similarity Average': 1.0294278310611844,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AlbaQuiroz',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 24,
  'TP Similarity': 45.0855712890625,
  'Others Similarity Average': 1.1114144679158926,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AlbaQuiroz',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 48,
  'TP Similarity': 46.14303207397461,
  'Others Similarity Average': 1.1505906539969146,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AlbaQuiroz',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 68,
  'TP Similarity': 49.04893493652344,
  'Others Similarity Average': 1.137192019727081,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'AlbaQuiroz',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 87,
  'TP Similarity': 38.33700942993164,
  'Others Similarity Average': 1.1109192045405507,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'CarlosBornacelly',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 99.91883087158203,
  'Others Similarity Average': 2.202529752161354,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 30,
  'TP Similarity': 99.97955322265625,
  'Others Similarity Average': 2.4068350004963577,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 60,
  'TP Similarity': 99.97174835205078,
  'Others Similarity Average': 2.1971261396538466,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 82,
  'TP Similarity': 99.9636459350586,
  'Others Similarity Average': 2.9848977057263255,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 94,
  'TP Similarity': 99.86034393310547,
  'Others Similarity Average': 4.050767396576703,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 99.44646453857422,
  'Others Similarity Average': 1.1352817015722394,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 32,
  'TP Similarity': 99.59828186035156,
  'Others Similarity Average': 1.0085317315533757,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 63,
  'TP Similarity': 99.7153091430664,
  'Others Similarity Average': 0.9228788539767265,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 78,
  'TP Similarity': 99.70230102539062,
  'Others Similarity Average': 1.021601753309369,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 86,
  'TP Similarity': 98.07310485839844,
  'Others Similarity Average': 0.8773900903761387,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 66.96182250976562,
  'Others Similarity Average': 1.3212584722787142,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielCastanez',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 36,
  'TP Similarity': 81.19284057617188,
  'Others Similarity Average': 1.4183476341422647,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 72,
  'TP Similarity': 65.58101654052734,
  'Others Similarity Average': 1.4686025721020997,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielCastanez',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 93,
  'TP Similarity': 71.58012390136719,
  'Others Similarity Average': 1.7815856891684234,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielCastanez',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 99,
  'TP Similarity': 62.67463684082031,
  'Others Similarity Average': 1.1573170823976398,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'GiohanOlivares',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 88.70281219482422,
  'Others Similarity Average': 3.4583246679976583,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 29,
  'TP Similarity': 83.38661193847656,
  'Others Similarity Average': 2.3910486875101924,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 59,
  'TP Similarity': 82.19293975830078,
  'Others Similarity Average': 2.6503410432487726,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 80,
  'TP Similarity': 86.96684265136719,
  'Others Similarity Average': 2.539420736953616,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 94,
  'TP Similarity': 87.3205337524414,
  'Others Similarity Average': 2.657095848582685,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 98.79532623291016,
  'Others Similarity Average': 1.209529172629118,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 32,
  'TP Similarity': 98.89508819580078,
  'Others Similarity Average': 1.2085596364922822,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 63,
  'TP Similarity': 98.86714935302734,
  'Others Similarity Average': 1.362074822653085,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 87,
  'TP Similarity': 99.0028305053711,
  'Others Similarity Average': 1.5673868446610868,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 95,
  'TP Similarity': 95.94502258300781,
  'Others Similarity Average': 1.0881824544630945,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 6,
  'TP Similarity': 84.9532699584961,
  'Others Similarity Average': 2.101088587194681,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 29,
  'TP Similarity': 78.07902526855469,
  'Others Similarity Average': 2.0453062439337373,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'JoseVilla',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 57,
  'TP Similarity': 81.51009368896484,
  'Others Similarity Average': 2.073117097839713,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 80,
  'TP Similarity': 80.75821685791016,
  'Others Similarity Average': 1.9701515808701515,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 94,
  'TP Similarity': 85.02582550048828,
  'Others Similarity Average': 1.4605908589437604,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 80.92790222167969,
  'Others Similarity Average': 2.414098793407902,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 32,
  'TP Similarity': 89.98580932617188,
  'Others Similarity Average': 1.757125984178856,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 64,
  'TP Similarity': 92.4250717163086,
  'Others Similarity Average': 1.823334680404514,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 90,
  'TP Similarity': 89.63633728027344,
  'Others Similarity Average': 2.143288924358785,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 99,
  'TP Similarity': 28.984092712402344,
  'Others Similarity Average': 3.4776525595225394,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'ManuelRangel',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 8,
  'TP Similarity': 11.35711669921875,
  'Others Similarity Average': 2.427173249423504,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'ManuelRangel',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 37,
  'TP Similarity': 23.22471809387207,
  'Others Similarity Average': 2.2232837434858084,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'ManuelRangel',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 73,
  'TP Similarity': 23.85824203491211,
  'Others Similarity Average': 1.898551378864795,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'ManuelRangel',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 88,
  'TP Similarity': 18.506608963012695,
  'Others Similarity Average': 1.7882902945857495,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'ManuelRangel',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 96,
  'TP Similarity': 9.907978057861328,
  'Others Similarity Average': 1.7385821319185197,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'MariaLopez',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 99.3152847290039,
  'Others Similarity Average': 0.5928332377225161,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 32,
  'TP Similarity': 99.26599884033203,
  'Others Similarity Average': 0.45780559442937374,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 64,
  'TP Similarity': 99.63204193115234,
  'Others Similarity Average': 0.504434575792402,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 89,
  'TP Similarity': 99.4451904296875,
  'Others Similarity Average': 0.5632496974430978,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 99,
  'TP Similarity': 94.61133575439453,
  'Others Similarity Average': 0.6142183225601912,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 7,
  'TP Similarity': 99.85235595703125,
  'Others Similarity Average': 2.2383891311474144,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 34,
  'TP Similarity': 99.63939666748047,
  'Others Similarity Average': 3.17880314681679,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 67,
  'TP Similarity': 99.6180419921875,
  'Others Similarity Average': 2.7561524864286184,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 85,
  'TP Similarity': 99.42272186279297,
  'Others Similarity Average': 2.078938608057797,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 98,
  'TP Similarity': 97.72429656982422,
  'Others Similarity Average': 2.6087846625596285,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Brightness Modifying Factor': 0.1,
  'Real Measured Brightness': 5,
  'TP Similarity': 70.8116683959961,
  'Others Similarity Average': 0.9694075891748071,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1},
 {'Name': 'YordiRochel',
  'Brightness Modifying Factor': 0.5,
  'Real Measured Brightness': 24,
  'TP Similarity': 80.64549255371094,
  'Others Similarity Average': 0.9455300318077207,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Brightness Modifying Factor': 1,
  'Real Measured Brightness': 48,
  'TP Similarity': 88.11612701416016,
  'Others Similarity Average': 0.9874949818477035,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Brightness Modifying Factor': 1.5,
  'Real Measured Brightness': 65,
  'TP Similarity': 83.60577392578125,
  'Others Similarity Average': 0.9843406137079,
  'TP': 1,
  'TN': 16,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Brightness Modifying Factor': 3,
  'Real Measured Brightness': 83,
  'TP Similarity': 61.080116271972656,
  'Others Similarity Average': 0.8245697673410177,
  'TP': 0,
  'TN': 16,
  'FP': 0,
  'FN': 1}]
]

In [109]:


TP=[0,0,0,0,0]
TN=[0,0,0,0,0]
FP=[0,0,0,0,0]
FN=[0,0,0,0,0]

TPAverageArray=[0,0,0,0,0]
OthersAverageArray=[0,0,0,0,0]
numberOfOthers=[0,0,0,0,0]
TPCounter=[0,0,0,0,0]

for element in brightnessControlResults:
    counter=0
    for object in element:
        TPAverageArray[counter]=TPAverageArray[counter]+object["TP Similarity"]
        TPCounter[counter]=TPCounter[counter]+1

        TP[counter]=TP[counter]+object["TP"]
        TN[counter]=TN[counter]+object["TN"]
        FP[counter]=FP[counter]+object["FP"]
        FN[counter]=FN[counter]+object["FN"]


        OthersAverageArray[counter]=OthersAverageArray[counter]+object["Others Similarity Average"]*((object["TP"]+object["TN"]+object["FP"]+object["FN"]-1))
        numberOfOthers[counter]=numberOfOthers[counter]+(object["TP"]+object["TN"]+object["FP"]+object["FN"]-1)
        counter=counter+1
        counter = 0 if counter == 5 else counter


for i,result in enumerate(OthersAverageArray):
    SNS=TP[i]/(TP[i]+FN[i])
    PRC=TP[i]/(TP[i]+FP[i])
    print(f" {brigthnessValuesArray[i]} brightness results -- TP Similarity {TPAverageArray[i]/TPCounter[i]} -- Others Similarity Average {(OthersAverageArray[i])/(numberOfOthers[i])} -- F1-Score : {2*((SNS*PRC)/(SNS+PRC))}")



 0.1 brightness results -- TP Similarity 86.07511418206352 -- Others Similarity Average 1.8124936849444306 -- F1-Score : 0.8898678414096917
 0.5 brightness results -- TP Similarity 86.83777839796883 -- Others Similarity Average 1.767558952487473 -- F1-Score : 0.8849557522123894
 1 brightness results -- TP Similarity 87.48094520871601 -- Others Similarity Average 1.7775832202152482 -- F1-Score : 0.8947368421052632
 1.5 brightness results -- TP Similarity 87.32589496128143 -- Others Similarity Average 1.747209071540939 -- F1-Score : 0.8898678414096917
 3 brightness results -- TP Similarity 80.29912716055674 -- Others Similarity Average 1.7785487508600843 -- F1-Score : 0.8113207547169812


In [110]:
TN

[1960, 1960, 1960, 1960, 1960]

In [79]:
objectGroup = []
for element in brightnessControlResults:
    objectGroup.extend(element)

#Sort elements taking into account the key "Size"
objectGroup.sort(key=lambda obj: obj["Real Measured Brightness"])

#Compute quartile size
quartileSize = len(objectGroup) // 4
resultsClassifiedBrightness = [[], [], [], []]

#Distribute objects in quartiles. IF the number isn't divisible by four, the objects remaining or missing are deducted or added to the fourth array
for i, obj in enumerate(objectGroup):
    if i < quartileSize:
        resultsClassifiedBrightness[0].append(obj)
    elif i < 2 * quartileSize:
        resultsClassifiedBrightness[1].append(obj)
    elif i < 3 * quartileSize:
        resultsClassifiedBrightness[2].append(obj)
    else:
        resultsClassifiedBrightness[3].append(obj)



In [82]:
for element in resultsClassifiedBrightness:
   TPSimilarityTotal=0
   OthersSimilarityTotal=0
   faceCounter=0
   TP=TN=FP=FN=0
   for i,object in enumerate(element):
      TPSimilarityTotal=TPSimilarityTotal+object["TP Similarity"]
      TP=TP+object["TP"]
      TN=TN+object["TN"]
      FP=FP+object["FP"]
      FN=FN+object["FN"]

      faceCounter=faceCounter+object["TP"]+object["TN"]+object["FP"]+object["FN"]-1
      OthersSimilarityTotal=OthersSimilarityTotal+(object["Others Similarity Average"])*(object["TP"]+object["TN"]+object["FP"]+object["FN"]-1)
   SNS=TP/(TP+FN)
   PRC=TP/(TP+FP)
   print(f"Brightness between {element[0]["Real Measured Brightness"]} and {element[-1]["Real Measured Brightness"]} ------- Elements in quartile {i+1}")
   print(f"TP similarity average was : {TPSimilarityTotal/(i+1)}")
   print(f"Other faces similarity average was : {OthersSimilarityTotal/faceCounter} ")
   print(f"F1-Score was {2*((SNS*PRC)/(SNS+PRC))}")
   print(f"TP:{TP} TN:{TN} FP: {FP} FN: {FN}")
   print("")

Brightness between 2 and 24 ------- Elements in quartile 157
TP similarity average was : 85.08740415998325
Other faces similarity average was : 1.7259238584267491 
F1-Score was 0.8745519713261649
TP:122 TN:2474 FP: 0 FN: 35

Brightness between 24 and 55 ------- Elements in quartile 157
TP similarity average was : 86.52461841303831
Other faces similarity average was : 1.7522043815262704 
F1-Score was 0.8785714285714286
TP:123 TN:2372 FP: 0 FN: 34

Brightness between 55 and 78 ------- Elements in quartile 157
TP similarity average was : 88.61608148076732
Other faces similarity average was : 1.8045507920659187 
F1-Score was 0.9097222222222222
TP:131 TN:2340 FP: 0 FN: 26

Brightness between 78 and 99 ------- Elements in quartile 159
TP similarity average was : 82.22996234144055
Other faces similarity average was : 1.8219733514659842 
F1-Score was 0.8351648351648352
TP:114 TN:2614 FP: 0 FN: 45



In [ ]:

roboticsFolderPath=os.path.join("..",'ImagesRekognition',"StudentsRobotics")

i=0
brightness_counter=0
for image in os.listdir(roboticsFolderPath):
    image_path=os.path.join(roboticsFolderPath,image)
    brightness=calculate_brightness_simple(image_path)
    brightness_counter=brightness_counter+brightness
    i=i+1
    print(f"Initial brightness of {image.split(".")[0]} is {brightness}")

    imageOpened=Image.open(image_path)
    enhancer=ImageEnhance.Brightness(imageOpened)
    enhanced_img=enhancer.enhance(0.05)
    enhanced_img.save(rf"modifiedBrightnessImages\{image}")
    newBrightness=calculate_brightness_simple(rf"modifiedBrightnessImages\{image}")
    print(f"Final brightness of {image.split(".")[0]} is {newBrightness}")




print(f"Average brightness is {float(brightness_counter/i)}")


